In [1]:
import pulp as p
import numpy as np
import math
import pandas as pd

from scipy import stats

import random

import math 

In [2]:
class weber_problem():
    
    def __init__(self, index, weight, x_coor, y_coor):
        self.index = np.array(index)
        self.weight = np.array(weight)
        self.x_coor = np.array(x_coor)
        self.y_coor = np.array(y_coor)
        
    def totalweight(self):
        return sum(self.weight)
    
    def weighteddist(self, x,y):
        return sum([self.weight[i]*math.sqrt((x-self.x_coor[i])**2+(y-self.y_coor[i])**2) for i in range(len(self.index))])
        
    def find_cog(self):
        x_cog = sum([self.weight[i]*self.x_coor[i]/sum(self.weight) for i in range(len(self.index))])
        y_cog = sum([self.weight[i]*self.y_coor[i]/sum(self.weight) for i in range(len(self.index))])
        cost_cog = self.weighteddist(x_cog, y_cog)/sum(self.weight)
        print('Center of Gravity x coordinate: ',x_cog)
        print('Center of Gravity y coordinate: ',y_cog)
        print('Center of Gravity avg. weighted dist: ',cost_cog)
        return x_cog, y_cog, cost_cog
    
    def weightedx(self): 
        return sum([self.weight[i]*self.x_coor[i]/sum(self.weight) for i in range(len(self.index))])
    
    def weightedy(self): 
        return sum([self.weight[i]*self.y_coor[i]/sum(self.weight) for i in range(len(self.index))])
    
    def find_weber(self):
        minX = self.x_coor.min()
        minY = self.y_coor.min()
        costmin = self.weighteddist(minX,minY)
    
        maxX = self.x_coor.max()
        maxY = self.y_coor.max()

        for i in np.arange(minX, maxX, 0.01):
            for j in np.arange(minY, maxY, 0.01):
                cost = self.weighteddist(i,j)
                if cost <= costmin:
                    x = i
                    y = j
                    costmin = cost
        costmin = costmin/sum(self.weight)
        
        print('weber x coordinate: ',x)
        print('weber y coordinate: ',y)
        print('weber avg. weighted dist: ',costmin)
        return x,y,costmin

In [17]:
#QQ3

#Using the same model for SandyCo, suppose the demand for each region is changed to 80 tons.

#What would the outcome be when you optimize this model?

x1a = p.LpVariable('1_to_a', lowBound = 0, cat = 'Integer')
x1b = p.LpVariable('1_to_b', lowBound = 0, cat = 'Integer')
x1c = p.LpVariable('1_to_c', lowBound = 0, cat = 'Integer')
x2a = p.LpVariable('2_to_a', lowBound = 0, cat = 'Integer')
x2b = p.LpVariable('2_to_b', lowBound = 0, cat = 'Integer')
x2c = p.LpVariable('2_to_c', lowBound = 0, cat = 'Integer')

sand = p.LpProblem('Sand Transportation', p.LpMinimize)

sand += 250*x1a + 325*x1b + 445*x1c + 275*x2a + 260*x2b + 460*x2c
sand += x1a + x1b + x1c <=100
sand += x2a + x2b + x2c <=125
sand += x1a + x2a >=80
sand += x1b + x2b >=80
sand += x1c + x2c >=80

sand.solve()

print('Optimal Solution: ', p.value(sand.objective))
print("Solution Status:", p.LpStatus[sand.status])

for item in sand.variables():
    print(item.name,' = ', item.varValue)


Optimal Solution:  77300.0
Solution Status: Infeasible
1_to_a  =  80.0
1_to_b  =  0.0
1_to_c  =  20.0
2_to_a  =  0.0
2_to_b  =  80.0
2_to_c  =  60.0


<b>Practice Problem 2: SteelCo</b>

SteelCo is a small steel distributor in Northern USA. They have two distribution centers (DCs), one in Allentown, PA, and one in Toledo, OH.

SteelCo has just received a large order from a construction contractor and will be supplying steel to four construction projects in the Northeast: one in Buffalo, one in Albany, one in Philadelphia, and one in Cleveland.

The contractor has ordered the following amounts to be shipped each week:

Buffalo: 40,500 pounds

Albany: 22,230 pounds

Philadelphia: 85,200 pounds

Cleveland: 47,500 pounds

Both DCs can each handle an additional 100,000 pounds of steel per week.

The transport costs for a pound of steel for the different DC-construction site combinations are given in the table below.

<img src = "w1pp1.png">

Suppose SteelCo wants to meet the customer's demand for steel at minimum cost. How many pounds of steel should be shipped from the DC in Allentown, PA, to the four construction projects, in total?

In [4]:
xaB = p.LpVariable('allentown_to_buffalo', lowBound = 0, cat = 'Integer')
xaA = p.LpVariable('allentown_to_albany', lowBound = 0, cat = 'Integer')
xaP = p.LpVariable('allentown_to_philadelphia', lowBound = 0, cat = 'Integer')
xaC = p.LpVariable('allentown_to_cleveland', lowBound = 0, cat = 'Integer')
xtB = p.LpVariable('toledo_to_buffalo', lowBound = 0, cat = 'Integer')
xtA = p.LpVariable('toledo_to_albany', lowBound = 0, cat = 'Integer')
xtP = p.LpVariable('toledo_to_philadelphia', lowBound = 0, cat = 'Integer')
xtC = p.LpVariable('toledo_to_cleveland', lowBound = 0, cat = 'Integer')

prob = p.LpProblem('Steel_Transportation', p.LpMinimize)

prob += 52*xaB + 32*xaA + 11*xaP + 69*xaC + 45*xtB + 84*xtA + 76*xtP + 15*xtC
prob += xaB + xtB >= 40500
prob += xaA + xtA >= 22230
prob += xaP + xtP >= 85200
prob += xaC + xtC >= 47500
prob += xaB + xaA + xaP + xaC <= 100000
prob += xtB + xtA + xtP + xtC <= 100000

prob.solve()

print('Optimal Solution: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ', item.varValue)


Optimal Solution:  4569920.0
Solution Status: Optimal
allentown_to_albany  =  14800.0
allentown_to_buffalo  =  0.0
allentown_to_cleveland  =  0.0
allentown_to_philadelphia  =  85200.0
toledo_to_albany  =  7430.0
toledo_to_buffalo  =  40500.0
toledo_to_cleveland  =  47500.0
toledo_to_philadelphia  =  0.0


<b>Practice Problem 3: Ralph Calvin Hilfiger (RCH) Industries</b>

Clothing manufacturer and retailer Ralph Calvin Hilfiger (RCH) Industries operates five (5) factories. They distribute their products through five (5) regional distribution centers (DCs) each of which supports a specific market. 

To reduce the number of direct shipments between factories and regional DCs, as well as improve the load factor of shipments, RCH has utilized a crossdocking (XD) facility for some time.  Owing to the success of their existing crossdocking facility (XD1), RCH has now decided to open an additional facility (XD2).

The clothes are transported through the network in small plastic totes via truck. The transportation costs for a tote between the five factories and the two crossdocking facilities are given in the table below.

<img src="w1pp31.png">

The transportation costs for a tote between the crossdocking facilities and the regional DCs are given in the table below.

<img src="w1pp32.png">

The five factories have the following production capacities per week (number of totes):

Factory 1: 200

Factory 2: 300

Factory 3: 100

Factory 4: 150

Factory 5: 220

The five regional DCs face the following demand per week (number of totes):

DC 1: 150

DC 2: 100

DC 3: 110

DC 4: 200

DC 5: 180

The crossdocking facilities have much spare capacity.

[Hint:  You need to build a transshipment model in your spreadsheet.  To check your spreadsheet, try setting each decision variable (the flow on each of the 20 arcs) = 1.  Your total cost should be $712 with 5 units going through each crossdock.  If this is not the case, check your model!]

Suppose RCH wishes to plan the flow of product from Factories to Crossdocks to Distribution Centers in order to minimize total costs.

How much should be shipped through the new facility, crossdocking facility two?


In [20]:
factories = ['Factory1', 'Factory2', 'Factory3', 'Factory4', 'Factory5']
factoriescap = [200,300,100,150,220]
dcs = ['DC1', 'DC2', 'DC3', 'DC4', 'DC5']
dcscap = [150,100,110,200,180]

crossdocks = ['Crossdock1','Crossdock2']
                   #crossdock1 #crossdock2
cost_fac_cross = [[30,          50], #factory1
                  [23,          66], #factory2
                  [35,          14], #factory3
                  [70,          12], #factory4
                  [65,          70]] #factory5

                #DC1 DC2 DC3 DC4 DC5
cost_cross_dc = [[12,25,22,40,41], #crossdock1
                 [65,22,23,12,15]] #crossdock2

factoriessupply = dict(zip(factories,factoriescap))
dcsdemand = dict(zip(dcs,dcscap))

fac_cross_dict = p.makeDict([factories, crossdocks], cost_fac_cross)
cross_dc_dict = p.makeDict([crossdocks, dcs], cost_cross_dc)

clothes = p.LpProblem('Clothes', p.LpMinimize)

fac_to_cross_route = [(f,c) for f in factories for c in crossdocks]

fac_to_cross_vars = p.LpVariable.dicts("route", (factories,crossdocks), lowBound = 0, cat = 'Integer')

cross_to_dc_route = [(c,d) for c in crossdocks for d in dcs]

cross_to_dc_vars = p.LpVariable.dicts("route", (crossdocks,dcs), lowBound = 0, cat = 'Integer')

clothes += p.lpSum([fac_to_cross_vars[f][c]*fac_cross_dict[f][c] for (f,c) in fac_to_cross_route]) \
            + p.lpSum([cross_to_dc_vars[c][d]*cross_dc_dict[c][d] for (c,d) in cross_to_dc_route])
                   
for f in factories:
    clothes += p.lpSum([fac_to_cross_vars[f][c] for c in crossdocks]) <= factoriessupply[f]
    
for d in dcs:
    clothes += p.lpSum([cross_to_dc_vars[c][d] for c in crossdocks]) >= dcsdemand[d]

for c in crossdocks:
    clothes += p.lpSum([fac_to_cross_vars[f][c] for f in factories]) == p.lpSum([cross_to_dc_vars[c][d] for d in dcs])
    
clothes.solve()

print('minimum cost: ', p.value(clothes.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in clothes.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  30220.0
Solution Status: Optimal
route_Crossdock1_DC1  =  150.0
route_Crossdock1_DC2  =  100.0
route_Crossdock1_DC3  =  110.0
route_Crossdock1_DC4  =  0.0
route_Crossdock1_DC5  =  0.0
route_Crossdock2_DC1  =  0.0
route_Crossdock2_DC2  =  0.0
route_Crossdock2_DC3  =  0.0
route_Crossdock2_DC4  =  200.0
route_Crossdock2_DC5  =  180.0
route_Factory1_Crossdock1  =  60.0
route_Factory1_Crossdock2  =  130.0
route_Factory2_Crossdock1  =  300.0
route_Factory2_Crossdock2  =  0.0
route_Factory3_Crossdock1  =  0.0
route_Factory3_Crossdock2  =  100.0
route_Factory4_Crossdock1  =  0.0
route_Factory4_Crossdock2  =  150.0
route_Factory5_Crossdock1  =  0.0
route_Factory5_Crossdock2  =  0.0


<b>Practice Problem 4: SandyCo - Multiple Optima</b>
    
For this problem, we will use the Transhipment Model for SandyCo (SandyCo Part 2). To recap, SandyCo has two (2) facilities that mine, clean, and sort sand for use in cement, children’s playboxes, and small beaches. They distribute the sand from their two (2) plants, through two (2) packaging centers (DCs), to three (3) different customer regions where it is packaged and sold.

Each plant has a specific maximum weekly supply of available sand and each region has an expected minimum weekly required demand. The cost to distribute a ton of sand differs between each plant and region pairing due to distance and other factors.

The maximum weekly supply from Plant 1 is 100 tons, from Plant 2 it is 125 tons. The expected minimum weekly required demand (in tons) is:

Region 1: 25
    
Region 2: 95
    
Region 3: 80
    
The transportation costs in dollars per ton of sand are given in the table below.

<img src = "w1pp41.png">

In the lesson we learned that in the optimal solution to the problem, the total cost is $69,200 per week. The solution presented in the lecture had the following weekly flows:

Plant 1 to DC B: 75

Plant 2 to DC A: 25

Plant 2 to DC B: 100

DC A to Region 1: 25

DC B to Region 2: 95

DC B to Region 3: 80

Input the flows above as decision-variables in your spreadsheet model. Don't run the solver. The spreadsheet will tell you that the total weekly cost is indeed $69,200 per week.

You hear from a colleague using another solver that there is another good solution to the problem. He tells you to consider following weekly flows:

Plant 1 to DC B: 75

Plant 2 to DC A: 105

Plant 2 to DC B: 20

DC A to Region 1: 25

DC A to Region 3: 80

DC B to Region 2: 95

Input the suggested flows instead of the flows from the lesson. What is the minimum weekly cost using the suggested flow volumes?

Round your answer to the nearest integer.

In [7]:
75*210 + 105*185 + 20*205 + 25*175 + 80*165 + 95*130

69200

Part2

After discussions with a number of carriers, SandyCo realizes that transportation costs to DC A need to be updated. The new costs are $180 per ton 

from Plant 1 to DC A and $190 per ton from Plant 2 to DC A.

How many tons of sand should be delivered from Plant 2 to DC B with the new costs?

Part3

After hearing that a rival sand distributor is facing economic difficulties, SandyCo wants to reconsider its demand estimates. Were the rival to default, SandyCo believes that their demand would increase by roughly 10% across the board, leading to the following minimum expected demands: Region 1 - 28, Region 2 - 105, and Region 3 - 87 tons respectively.

How many tons of sand should be delivered from DC A to Region 3 if the expected minimum demand was to increase by the amount specified? Note that the costs from Part 2 still apply.


In [2]:
supply = ['Supply1', 'Supply2']
supplycap = [100, 125]
demand = ['Demand1', 'Demand2', 'Demand3']
demandcap = [25,95,80]

intermediate = ['Intermediate1','Intermediate2']
                   #int1      #int2
cost_sup_int   = np.array([[180,       210], #sup1
                  [190,       205]]) #sup2
                  
                #Dem1 Dem2 Dem3
cost_int_dem = np.array([[175, 180, 165], #int1
                [235, 130, 145]]) #int2

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int)
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem)

prob = p.LpProblem('Sand', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route]) \
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s]
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) >= demandconstraint[d]

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand])
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  68325.0
Solution Status: Optimal
route_Intermediate1_Demand1  =  25.0
route_Intermediate1_Demand2  =  0.0
route_Intermediate1_Demand3  =  75.0
route_Intermediate2_Demand1  =  0.0
route_Intermediate2_Demand2  =  95.0
route_Intermediate2_Demand3  =  5.0
route_Supply1_Intermediate1  =  100.0
route_Supply1_Intermediate2  =  0.0
route_Supply2_Intermediate1  =  0.0
route_Supply2_Intermediate2  =  100.0


Part 4

The rival firm did not default - so your demand is back to the original values (Region 1: 25, Region 2: 95, and Region 3: 80).

The management of Plant 2 has suggested they may use some of the available space in the plant for an on-site packaging machine. With such a machine in place, Plant 2 would be able to bypass the DCs and ship directly to Region 1. Direct transport from Plant 2 to Region 1 costs $275 per ton. 

All other costs remain the same as in Part 2.

If Plant 2 invests in a on-site sand packaging machine, how many tons of sand should be delivered directly from Plant 2 to Region 1?


In [22]:
supply = ['Supply1', 'Supply2']
supplycap = [100, 125]
demand = ['Demand1', 'Demand2', 'Demand3']
demandcap = [25,95,80]

intermediate = ['Intermediate1','Intermediate2']
                   #int1      #int2
cost_sup_int   = np.array([[180,       210], #sup1
                  [190,       205]]) #sup2
                  
                #Dem1 Dem2 Dem3
cost_int_dem = np.array([[175, 180, 165], #int1
                [235, 130, 145]]) #int2

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int)
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem)

prob = p.LpProblem('Sand', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

sup2todem1 = p.LpVariable('supply2_to_demand1', lowBound = 0, cat = 'Integer') #direct shipment from Plant2 (Supply2) to Region1(Demand1)

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route]) \
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + 275*sup2todem1 #added transportation cost from Plant2 to Region1
                   
for s in supply:
    if s == "Supply2": #add amount of sand transported from Plant2 to Region1 for Supply2
        prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) + sup2todem1 <= supplyconstraint[s]
    else:
        prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s]

for d in demand:
    if d == "Demand1": #add amount of sand transported from Plant2 to Region1 for Demand1
        prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) + sup2todem1 >= demandconstraint[d]
    else:
        prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) >= demandconstraint[d]

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand])
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  66300.0
Solution Status: Optimal
route_Intermediate1_Demand1  =  0.0
route_Intermediate1_Demand2  =  0.0
route_Intermediate1_Demand3  =  80.0
route_Intermediate2_Demand1  =  0.0
route_Intermediate2_Demand2  =  95.0
route_Intermediate2_Demand3  =  0.0
route_Supply1_Intermediate1  =  80.0
route_Supply1_Intermediate2  =  0.0
route_Supply2_Intermediate1  =  0.0
route_Supply2_Intermediate2  =  95.0
supply2_to_demand1  =  25.0


Part 5

After investing in the new machine, a breakdown in DC B reduces the weekly capacity of DC B to 80 tons. All other costs and demand remain the same as in Part 2. (Note that the bypass option from Part 4 remains.)

How many tons of sand should Plant 1 produce if the capacity of DC B drops to 80 tons? There is no capacity constraint for DC A.

In [24]:
supply = ['Supply1', 'Supply2']
supplycap = [100, 125]
demand = ['Demand1', 'Demand2', 'Demand3']
demandcap = [25,95,80]

intermediate = ['Intermediate1','Intermediate2']
intermediatecap = [10000000, 80] #capacity of DCB (Intermediate B) is 80 tons, while for DCA (Intermediate A), \
                                 #it is considered unlimited, hence the big number used

                   #int1      #int2
cost_sup_int   = np.array([[180,       210], #sup1
                  [190,       205]]) #sup2
                  
                #Dem1 Dem2 Dem3
cost_int_dem = np.array([[175, 180, 165], #int1
                [235, 130, 145]]) #int2

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))
intermediateconstraint = dict(zip(intermediate,intermediatecap)) #added the constraint for the DCs (Intermediate)

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int)
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem)

prob = p.LpProblem('Sand', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

sup2todem1 = p.LpVariable('supply2_to_demand1', lowBound = 0, cat = 'Integer') #direct shipment from Plant2 (Supply2) to Region1(Demand1)

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route]) \
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + 275*sup2todem1 #added transportation cost from Plant2 to Region1
                   
for s in supply:
    if s == "Supply2": #add amount of sand transported from Plant2 to Region1 for Supply2
        prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) + sup2todem1 <= supplyconstraint[s]
    else:
        prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s]

for d in demand:
    if d == "Demand1": #add amount of sand transported from Plant2 to Region1 for Demand1
        prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) + sup2todem1 >= demandconstraint[d]
    else:
        prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) >= demandconstraint[d]

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand])
    
for i in intermediate: #constraint for DCs (Intermediate) capacity
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) <= intermediateconstraint[i]
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  66675.0
Solution Status: Optimal
route_Intermediate1_Demand1  =  0.0
route_Intermediate1_Demand2  =  15.0
route_Intermediate1_Demand3  =  80.0
route_Intermediate2_Demand1  =  0.0
route_Intermediate2_Demand2  =  80.0
route_Intermediate2_Demand3  =  0.0
route_Supply1_Intermediate1  =  95.0
route_Supply1_Intermediate2  =  0.0
route_Supply2_Intermediate1  =  0.0
route_Supply2_Intermediate2  =  80.0
supply2_to_demand1  =  25.0


<b>Practice Problem 5: Food Delivery Startup</b>
    
You and a friend just started a food delivery startup. The idea is to deliver freshly made, organic, lunch-boxes to be sold in university cafeterias. You have secured deals with cafeterias at three universities and are now looking for a good location for the business.

To avoid spending too much time in traffic, you think it may be a good idea to find a location close to all three universities.

The different parameters (weights and coordinates for the locations) are summed up in the following table.

<img src="w1pp51.png">

What is the weighted X-coordinate?

What is the weighted Y-coordinate?

What is the average weighted distance from the business location to the 3 universities using the center of gravity technique?

What is the average weighted distance from the business location to the 3 universities using the Weber technique?

In [216]:
food = weber_problem(['MIT','HAR','YALE'],[250,150,100],[20,20,100],[30,40,150])

food.find_cog()
food.find_weber()


#def weighteddist(x,y, locations):
    #dist = 0
    #for i in range(len(locations)):
     #   dist = dist + locations.wgt.iloc[i]*math.sqrt((x-locations.x.iloc[i])**2+(y-locations.y.iloc[i])**2)
    #return dist
    #return sum([locations.wgt.iloc[i]*math.sqrt((x-locations.x.iloc[i])**2+\
     #                                           (y-locations.y.iloc[i])**2) for i in range(len(locations))])


Center of Gravity x coordinate:  36.0
Center of Gravity y coordinate:  57.0
Center of Gravity avg. weighted dist:  45.274676417067106
weber x coordinate:  20
weber y coordinate:  30
weber avg. weighted dist:  31.844410203711917


(20, 30, 31.844410203711917)

<b>Practice Problem 6: Temporary Networks</b>

When a natural disaster strikes, normal supply chains are disrupted and many vital supplies cannot reach those affected through the normal routes. To handle the first couple of days after the disaster, many specialized Non Governmental Organizations (NGOs) work hard to secure vital supplies such as food, blankets, and medicines, and deliver these to people in the affect areas. To do so, they set up temporary supply networks, which only operate as long as needed.

Suppose you are working with one such NGO to set up a temporary network to distribute disaster-kits in the aftermath of a hurricane. You have secured kits from your central supply facility that will be flown in regularly. The kits need to be delivered once per week as long as needed to eight (8) temporary shelters, which are located a few hours drive from the airport.

To simplify the operations, you aim to set up a logistics and distribution center (DC) that controls all distribution. All incoming shipments will be transported from the airport directly to the center by the military, and from the DC you will plan how the kits are delivered to the shelters.

You are choosing between five (5) locations for the DC. The distances between the five (5) potential locations and the eight (8) shelters are shown in the table below.

<img src="w1pp61.png">

The weekly demand at each shelter has been estimated as follows:

<img src="w1pp62.png">

The capacity of the DC will be limited to 150,000 kits per week. It costs $100,000 per week to set up and run a DC.

[Hint:  You need to build a facility location model in your spreadsheet, following the steps of the lessons.  To check your spreadsheet, try setting each decision variable (the flow on each of the 40 arcs and the five binary open-or-not variables) = 1.  Your total cost should be 500,199.  If this is not the case, check your model!]

Part1

Trucks that can operate on the roads after the hurricane are in short supply, so you want to choose the location of the DC to minimize the transport work (tonkm, or the total kit-miles travelled) in the temporary network. Note that transport work per kit is proportional to distance.

Which of the five locations should the NGO choose to minimize the transport work in the temporary network?

In [131]:
supply = ['Supply1', 'Supply2', 'Supply3', 'Supply4' , 'Supply5']
supplycap = [150000, 150000, 150000, 150000, 150000 ]
demand = ['Demand1', 'Demand2', 'Demand3', 'Demand4', 'Demand5', 'Demand6', 'Demand7', 'Demand8']
demandcap = [14000,8000,25000,22000,20000,17000,18500,23000]

                   #dem1 #dem2 #dem3 #dem4 #dem5 #dem6 #dem7 #dem8
cost_sup_dem   = np.array([[0.61, 3.32, 9.5, 6.52, 7.77, 1.92, 8.52, 9.75], #sup1
                  [6.04, 0.51, 1.34, 6.06, 0.22, 6.33, 4.61, 3.28], #sup2
                  [4.99, 2.41, 2.33, 3.95, 8.84, 7.94, 7.87, 0.94], #sup3
                  [5.58, 8.8, 6.32, 8.54, 5.15, 6.06, 9.42, 2.16], #sup4
                  [7.87, 9.64, 0.7, 5.92, 2.7, 0.26, 0.5, 3.4]])#sup5
                  

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

sup_dem_dict = p.makeDict([supply, demand], cost_sup_dem)

prob = p.LpProblem('Kits', p.LpMinimize)

sup_to_dem_route = [(s,d) for s in supply for d in demand]

sup_to_dem_vars = p.LpVariable.dicts("route", (supply,demand), lowBound = 0, cat = 'Continuous') #amount of kits travelling between DC and shelter
                  
sup_open = p.LpVariable.dicts("open", supply, cat = 'Binary') #binary variables DC open or close

prob += p.lpSum([sup_to_dem_vars[s][d]*sup_dem_dict[s][d] for (s,d) in sup_to_dem_route]) \
        + p.lpSum([sup_open[s]*100000 for s in supply]) #fixed cost of opening DC
                  
prob += p.lpSum([sup_open[s] for s in supply]) <= 1 #number of DC minimum must be open
                  
prob += p.lpSum([sup_open[s] for s in supply]) >= 1 #number of DC maximum must be open
                   
for s in supply:
    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) <= supplyconstraint[s] #constraint must be less than DC capacity
                  
for d in demand:
    prob += p.lpSum([sup_to_dem_vars[s][d] for s in supply]) >= demandconstraint[d] #constraint must be equal or more than shelter demand

for s in supply:
    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) - 10000000000*sup_open[s] <= 0 #linking constraint if any channel from DC is flowing, the DC must be open
    
prob.solve()

print('minimum cost:', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost: 580910.0
Solution Status: Optimal
open_Supply1  =  0.0
open_Supply2  =  0.0
open_Supply3  =  0.0
open_Supply4  =  0.0
open_Supply5  =  1.0
route_Supply1_Demand1  =  0.0
route_Supply1_Demand2  =  0.0
route_Supply1_Demand3  =  0.0
route_Supply1_Demand4  =  0.0
route_Supply1_Demand5  =  0.0
route_Supply1_Demand6  =  0.0
route_Supply1_Demand7  =  0.0
route_Supply1_Demand8  =  0.0
route_Supply2_Demand1  =  0.0
route_Supply2_Demand2  =  0.0
route_Supply2_Demand3  =  0.0
route_Supply2_Demand4  =  0.0
route_Supply2_Demand5  =  0.0
route_Supply2_Demand6  =  0.0
route_Supply2_Demand7  =  0.0
route_Supply2_Demand8  =  0.0
route_Supply3_Demand1  =  0.0
route_Supply3_Demand2  =  0.0
route_Supply3_Demand3  =  0.0
route_Supply3_Demand4  =  0.0
route_Supply3_Demand5  =  0.0
route_Supply3_Demand6  =  0.0
route_Supply3_Demand7  =  0.0
route_Supply3_Demand8  =  0.0
route_Supply4_Demand1  =  0.0
route_Supply4_Demand2  =  0.0
route_Supply4_Demand3  =  0.0
route_Supply4_Demand4  =  0.0
route_S

Part 2

Due to road conditions, the shelters furthest away from the center may suffer from late and unreliable deliveries. As a result you are thinking about opening more than one DC.

To trade-off the fixed costs with the variable costs, you estimate that each transported mile costs $2 per kit.

Assuming the military is ok with delivering your supplies to more than one DC, what is the optimal number of DCs?

Part 3

You realize that the actual cost of setting up and operating a DC is not perfectly known. What is the optimal number of DCs if the fixed costs are instead $300,000 per week per DC? Assume the same transportation costs as in Part 2.

In [154]:
#Part 3

supply = ['Supply1', 'Supply2', 'Supply3', 'Supply4' , 'Supply5']
supplycap = [150000, 150000, 150000, 150000, 150000 ]
demand = ['Demand1', 'Demand2', 'Demand3', 'Demand4', 'Demand5', 'Demand6', 'Demand7', 'Demand8']
demandcap = [14000,8000,25000,22000,20000,17000,18500,23000]

                   #dem1 #dem2 #dem3 #dem4 #dem5 #dem6 #dem7 #dem8
cost_sup_dem   = np.array([[0.61, 3.32, 9.5, 6.52, 7.77, 1.92, 8.52, 9.75], #sup1
                  [6.04, 0.51, 1.34, 6.06, 0.22, 6.33, 4.61, 3.28], #sup2
                  [4.99, 2.41, 2.33, 3.95, 8.84, 7.94, 7.87, 0.94], #sup3
                  [5.58, 8.8, 6.32, 8.54, 5.15, 6.06, 9.42, 2.16], #sup4
                  [7.87, 9.64, 0.7, 5.92, 2.7, 0.26, 0.5, 3.4]]) #sup5
                  

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

sup_dem_dict = p.makeDict([supply, demand], cost_sup_dem)

prob = p.LpProblem('Kits', p.LpMinimize)

sup_to_dem_route = [(s,d) for s in supply for d in demand]

sup_to_dem_vars = p.LpVariable.dicts("route", (supply,demand), lowBound = 0, cat = 'Continuous')
                  
sup_open = p.LpVariable.dicts("open", supply, cat = 'Binary')

prob += p.lpSum([sup_to_dem_vars[s][d]*sup_dem_dict[s][d]*2 for (s,d) in sup_to_dem_route]) \ #2$ per tonkm
        + p.lpSum([sup_open[s]*300000 for s in supply])
                  
prob += p.lpSum([sup_open[s] for s in supply]) <= 5
                  
prob += p.lpSum([sup_open[s] for s in supply]) >= 1
                   
for s in supply:
    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) <= supplyconstraint[s]
                  
for d in demand:
    prob += p.lpSum([sup_to_dem_vars[s][d] for s in supply]) >= demandconstraint[d]

for s in supply:
    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) - 10000000*sup_open[s] <= 0
    
prob.solve()

print('minimum cost:', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost: 1165660.0
Solution Status: Optimal
open_Supply1  =  0.0
open_Supply2  =  0.0
open_Supply3  =  1.0
open_Supply4  =  0.0
open_Supply5  =  1.0
route_Supply1_Demand1  =  0.0
route_Supply1_Demand2  =  0.0
route_Supply1_Demand3  =  0.0
route_Supply1_Demand4  =  0.0
route_Supply1_Demand5  =  0.0
route_Supply1_Demand6  =  0.0
route_Supply1_Demand7  =  0.0
route_Supply1_Demand8  =  0.0
route_Supply2_Demand1  =  0.0
route_Supply2_Demand2  =  0.0
route_Supply2_Demand3  =  0.0
route_Supply2_Demand4  =  0.0
route_Supply2_Demand5  =  0.0
route_Supply2_Demand6  =  0.0
route_Supply2_Demand7  =  0.0
route_Supply2_Demand8  =  0.0
route_Supply3_Demand1  =  14000.0
route_Supply3_Demand2  =  8000.0
route_Supply3_Demand3  =  0.0
route_Supply3_Demand4  =  22000.0
route_Supply3_Demand5  =  0.0
route_Supply3_Demand6  =  0.0
route_Supply3_Demand7  =  0.0
route_Supply3_Demand8  =  23000.0
route_Supply4_Demand1  =  0.0
route_Supply4_Demand2  =  0.0
route_Supply4_Demand3  =  0.0
route_Supply4_Demand4

Part4

When you see the optimal solution from Part 3, you realize that under that solution, much of the demand is more than 2 miles away from a DC. How does the optimal solution change if we require that at least 60% of demand should be less than 2 miles from a DC?

In [73]:
supply = ['Supply1', 'Supply2', 'Supply3', 'Supply4' , 'Supply5']
supplycap = [150000, 150000, 150000, 150000, 150000 ]
demand = ['Demand1', 'Demand2', 'Demand3', 'Demand4', 'Demand5', 'Demand6', 'Demand7', 'Demand8']
demandcap = [14000,8000,25000,22000,20000,17000,18500,23000]

                   #dem1 #dem2 #dem3 #dem4 #dem5 #dem6 #dem7 #dem8
cost_sup_dem   = np.array([[0.61, 3.32, 9.5, 6.52, 7.77, 1.92, 8.52, 9.75], #sup1
                  [6.04, 0.51, 1.34, 6.06, 0.22, 6.33, 4.61, 3.28], #sup2
                  [4.99, 2.41, 2.33, 3.95, 8.84, 7.94, 7.87, 0.94], #sup3
                  [5.58, 8.8, 6.32, 8.54, 5.15, 6.06, 9.42, 2.16], #sup4
                  [7.87, 9.64, 0.7, 5.92, 2.7, 0.26, 0.5, 3.4]]) #sup5

dist_less_than_2 = cost_sup_dem <= 2 #remember to use less than equal to sign

dist_less_than_2 = dist_less_than_2.astype(int) / sum(demandcap) #create matrix whose value is 1 if dist less than 2, 0 otherwise

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

sup_dem_dict = p.makeDict([supply, demand], cost_sup_dem)

dist_less_than_2_dict = p.makeDict([supply, demand], dist_less_than_2)

prob = p.LpProblem('Kits', p.LpMinimize)

sup_to_dem_route = [(s,d) for s in supply for d in demand]

sup_to_dem_vars = p.LpVariable.dicts("route", (supply,demand), lowBound = 0, cat = 'Continuous')
                  
sup_open = p.LpVariable.dicts("open", supply, cat = 'Binary')

prob += p.lpSum([sup_to_dem_vars[s][d]*sup_dem_dict[s][d]*2 for (s,d) in sup_to_dem_route]) \
        + p.lpSum([sup_open[s]*300000 for s in supply])
                  
prob += p.lpSum([sup_open[s] for s in supply]) <= 5
                  
prob += p.lpSum([sup_open[s] for s in supply]) >= 1

prob += p.lpSum([sup_to_dem_vars[s][d]*dist_less_than_2_dict[s][d] for (s,d) in sup_to_dem_route]) >= 0.6 #constraint 
                                                #that the % of demand within less than 2 miles is equal or more than 60%
                   
for s in supply:
    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) <= supplyconstraint[s]

for d in demand:
    prob += p.lpSum([sup_to_dem_vars[s][d] for s in supply]) == demandconstraint[d] #change to == equal sign as to enforce hard
                                    #constraint (i.e.: otherwise the solution will just add kits to DCs just to increase 
                                    #the percentage of demand served less than 2 miles to be 60%

for s in supply:
    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) - 100000000*sup_open[s] <= 0
    
    
prob.solve()

print('minimum cost:', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost: 1259780.0
Solution Status: Optimal
open_Supply1  =  0.0
open_Supply2  =  1.0
open_Supply3  =  0.0
open_Supply4  =  0.0
open_Supply5  =  1.0
route_Supply1_Demand1  =  0.0
route_Supply1_Demand2  =  0.0
route_Supply1_Demand3  =  0.0
route_Supply1_Demand4  =  0.0
route_Supply1_Demand5  =  0.0
route_Supply1_Demand6  =  0.0
route_Supply1_Demand7  =  0.0
route_Supply1_Demand8  =  0.0
route_Supply2_Demand1  =  14000.0
route_Supply2_Demand2  =  8000.0
route_Supply2_Demand3  =  0.0
route_Supply2_Demand4  =  0.0
route_Supply2_Demand5  =  20000.0
route_Supply2_Demand6  =  0.0
route_Supply2_Demand7  =  0.0
route_Supply2_Demand8  =  23000.0
route_Supply3_Demand1  =  0.0
route_Supply3_Demand2  =  0.0
route_Supply3_Demand3  =  0.0
route_Supply3_Demand4  =  0.0
route_Supply3_Demand5  =  0.0
route_Supply3_Demand6  =  0.0
route_Supply3_Demand7  =  0.0
route_Supply3_Demand8  =  0.0
route_Supply4_Demand1  =  0.0
route_Supply4_Demand2  =  0.0
route_Supply4_Demand3  =  0.0
route_Supply4_Demand4

<h3>Graded Assignment 1:  Locky Locke Inc.</h3>

You have finally been promoted to Senior Manager for the nuts and bolts category at Locky Locke Inc., a manufacturer with a large demand for bolts at its three (3) factories. You have surveyed the market and found four (4) suppliers for a certain steady selling SKU, ZA27R46.  Your factories' weekly demand for bolts are:

Factory 1: 9,500 cases/week

Factory 2: 6,500 cases/week

Factory 3: 17,000 cases/week

Each of the four suppliers can provide the ZA27R46, but they differ in unit price, weekly capacity, and distance to Locky Locke Inc. three factories. The unit prices and the weekly capacity for each of the four suppliers are shown below: 

Supplier 1:  U.S. Ainbolt  - Unit cost:  2.50 $/case  Capacity  9,500 cases/week

Supplier 2:  Der Bolt, Thun  - Unit cost:  3.20  $/case  Capacity  10,000 cases/week

Supplier 3: Li Tningbolt -  Unit cost:  1.90  $/case  Capacity  7,000 cases/week

Supplier 4:  M. Bolton -   Unit cost:  2.80  $/case  Capacity  9,000 cases/week

You are trying to plan the weekly flow of bolts from these suppliers to your factories. Based on the current contract with your transportation carrier, the unit transportation cost for a case of bolts is $0.05 per mile 

from any of the suppliers to any of the factories.  The distances between the four suppliers and the three factories are given in the table below.

<img src="w1ga1.png">

Part 1

Currently, the factories operate independently. Disregarding any of the other factories, which supplier should Factory 2 select in order to have the lowest total (purchase and transport) cost for them?

Part 2

You want to take a holistic view of bolt sourcing instead of having each factory select its supplier on its own. You decide to create a model to help you select the optimal sourcing assignment for all of the factories for the ZA27R46 bolts. Based on this optimization model, what is the total weekly cost for supplying bolts for Locky Locke Inc.? Be sure to include the purchase price as well as the transportation costs.

Part 3

Based on your optimization model, which supplier should Factory 2 select in order to have the lowest total (purchase and transport) cost for Locky Locke Inc.?

Part 4

You want to discuss the situation with the general manager of Factory 2. How much is the difference between Factory 2's total landed cost per week in Part 1 compared to the total landed cost per week from Part 3?

In [77]:
supply = ['Supply1', 'Supply2', 'Supply3', 'Supply4']
supplycap = np.array([9500, 10000, 7000, 9000])
supplyprice = np.array([2.5,3.2,1.9,2.8])

demand = ['Demand1', 'Demand2', 'Demand3',]
demandcap = np.array([9500, 6500, 17000])

                   #dem1 #dem2 #dem3
dist_sup_dem   = np.array([[120, 380, 806], #sup1
                  [465, 717, 853], #sup2
                  [553, 572, 830], #sup3
                  [915, 406, 564]]) #sup4

cost_sup_dem = dist_sup_dem*0.05
                  
#======================================================

print('PART1')

for i in range(len(supplycap)):
    transportcost = demandcap[1]*cost_sup_dem[i][1]
    purchasecost = demandcap[1]*supplyprice[i]
    totalcost = transportcost+purchasecost
    print("cost factory"+str(i+1)+": ",totalcost)
    
#======================================================
    

supplyconstraint = dict(zip(supply,supplycap))
supplypricedict = dict(zip(supply,supplyprice))

demandconstraint = dict(zip(demand,demandcap))

sup_dem_dict = p.makeDict([supply, demand], cost_sup_dem)

prob = p.LpProblem('Bolts', p.LpMinimize)

sup_to_dem_route = [(s,d) for s in supply for d in demand]

sup_to_dem_vars = p.LpVariable.dicts("route", (supply,demand), lowBound = 0, cat = 'Continuous')
                  
prob += p.lpSum([sup_to_dem_vars[s][d]*sup_dem_dict[s][d] for (s,d) in sup_to_dem_route]) \
        + p.lpSum([sup_to_dem_vars[s][d]*supplypricedict[s] for (s,d) in sup_to_dem_route]) #transport cost + purchase cost as obj. function

                  
#prob += p.lpSum([sup_open[s] for s in supply]) <= 5
                  
#prob += p.lpSum([sup_open[s] for s in supply]) >= 1
                   
for s in supply:
    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) <= supplyconstraint[s]
                  
for d in demand:
    prob += p.lpSum([sup_to_dem_vars[s][d] for s in supply]) >= demandconstraint[d]
    
prob.solve()

print('\nPART2')

print('minimum cost:', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

#===================================================================

print("\nPART4")

TLC1 = 139750
TLC2 = 198250
print('Difference in cost for Factory2 in Part1 vs Part2: ',TLC1-TLC2)



PART1
cost factory1:  139750.0
cost factory2:  253825.0
cost factory3:  198250.0
cost factory4:  150150.0

PART2
minimum cost: 923575.0
Solution Status: Optimal
route_Supply1_Demand1  =  9500.0
route_Supply1_Demand2  =  0.0
route_Supply1_Demand3  =  0.0
route_Supply2_Demand1  =  0.0
route_Supply2_Demand2  =  0.0
route_Supply2_Demand3  =  7500.0
route_Supply3_Demand1  =  0.0
route_Supply3_Demand2  =  6500.0
route_Supply3_Demand3  =  500.0
route_Supply4_Demand1  =  0.0
route_Supply4_Demand2  =  0.0
route_Supply4_Demand3  =  9000.0

PART4
Difference in cost for Factory2 in Part1 vs Part2:  -58500


Part 5

You decide to re-negotiate with Supplier 1 (U.S. Ainbolt) to increase their capacity. They agree to increase it to 14,000 cases per week. How many cases per week does Supplier 1 (U.S. Ainbolt) send to Factory 2 in the new optimal solution?
Input your answer in integers.

Based on this optimization model, what is the total weekly cost for supplying bolts for Locky Locke Inc.? Be sure to include the purchase price as well as the transportation costs.

In [80]:
supply = ['Supply1', 'Supply2', 'Supply3', 'Supply4']
supplycap = np.array([14000, 10000, 7000, 9000])
supplyprice = np.array([2.5,3.2,1.9,2.8])

demand = ['Demand1', 'Demand2', 'Demand3',]
demandcap = np.array([9500, 6500, 17000])

                   #dem1 #dem2 #dem3
dist_sup_dem   = np.array([[120, 380, 806], #sup1
                  [465, 717, 853], #sup2
                  [553, 572, 830], #sup3
                  [915, 406, 564]]) #sup4

cost_sup_dem = dist_sup_dem*0.05
            
supplyconstraint = dict(zip(supply,supplycap))
supplypricedict = dict(zip(supply,supplyprice))

demandconstraint = dict(zip(demand,demandcap))

sup_dem_dict = p.makeDict([supply, demand], cost_sup_dem)

prob = p.LpProblem('Bolts', p.LpMinimize)

sup_to_dem_route = [(s,d) for s in supply for d in demand]

sup_to_dem_vars = p.LpVariable.dicts("route", (supply,demand), lowBound = 0, cat = 'Continuous')
                  
prob += p.lpSum([sup_to_dem_vars[s][d]*sup_dem_dict[s][d] for (s,d) in sup_to_dem_route])\
        + p.lpSum([sup_to_dem_vars[s][d]*supplypricedict[s] for (s,d) in sup_to_dem_route])

                  
#prob += p.lpSum([sup_open[s] for s in supply]) <= 5
                  
#prob += p.lpSum([sup_open[s] for s in supply]) >= 1
                   
for s in supply:
    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) <= supplyconstraint[s]
                  
for d in demand:
    prob += p.lpSum([sup_to_dem_vars[s][d] for s in supply]) >= demandconstraint[d]
    
prob.solve()

print('\nPART5')

print('minimum cost:', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


PART5
minimum cost: 872050.0
Solution Status: Optimal
route_Supply1_Demand1  =  9500.0
route_Supply1_Demand2  =  4500.0
route_Supply1_Demand3  =  0.0
route_Supply2_Demand1  =  0.0
route_Supply2_Demand2  =  0.0
route_Supply2_Demand3  =  3000.0
route_Supply3_Demand1  =  0.0
route_Supply3_Demand2  =  2000.0
route_Supply3_Demand3  =  5000.0
route_Supply4_Demand1  =  0.0
route_Supply4_Demand2  =  0.0
route_Supply4_Demand3  =  9000.0


<h3>Graded Assignment 2: GoClean&Co</h3>

GoClean&Co (GCC) hired you as its Logistics Manager in charge of transshipments. GoClean&Co is a joint public-private energy company dedicated to generating clean and safe nuclear energy. GCC uses a new technology based on radioactive pellets. These pellets can generate electricity for years in GCC's nuclear plants. Once the radioactive pellets are depleted, they are removed from active use and collected at the plants for removal at the end of the month. Once a month, the depleted pellets are removed from the plants, transported to an inspection site for examination, and then moved again to a dedicated storage site for safekeeping.

You have been informed that two of GCC's nuclear plants, called Plant One and Plant Two, need to move out depleted pellets in the following amounts:

Plant One wants to ship out up to 280 pellets per month

Plant Two wants to ship out up to 360 pellets per month

This is how many pellets these two plants want to ship out per month. However, the exact number of pellets that will actually be moved out from each plant depends on the space that is available at the dedicated storage sites that would ultimately receive them. GCC is mandated by law to move as many pellets out of the plants as the storage sites are able to receive. You have been informed that two of GCC's dedicated storage sites, called Storage X and Storage Y, are ready to receive depleted pellets from GCC's plants, in the following amounts:

Storage X is ready to receive a total of 305 pellets per month

Storage Y is ready to receive a total of 325 pellets per month

Since the two plants want to ship out more pellets per month than the two storage sites can receive, you know that some pellets will have to stay in their plant of origin until a later time. This is not a problem, however, since every one of GCC's plants has the capacity to temporarily store whatever pellets are not shipped out, and the law allows this.

At this time, only one inspection site is available to receive the pellets from the two plants, inspect them, and send them out to the two storage sites. It is called Inspection Site A. Notice that inspection sites do not have storage capacity: the pellets come in, are inspected, and continue their trip to the storage sites.

Moving radioactive material is not easy and therefore it is not cheap either. There is a lot of planning and security involved. Each shipment of pellets has to be closely monitored and protected to prevent the loss of radioactive material. Also, the vehicles transporting the material have to move relatively slowly, to avoid accidents. Therefore, it is very expensive to transport the radioactive pellets from the plant to the inspection site, and from there to the storage site. Based on estimates and historical data, GCC has calculated the following transportation costs, in dollars per pellet moved:

It costs 13.58 dollars to move one pellet from Plant One to Inspection Site A

It costs 16.54 dollars to move one pellet from Plant Two to Inspection Site A

It costs 7.57 dollars to move one pellet from Inspection Site A to Storage X

It costs 16.46 dollars to move one pellet from Inspection Site A to Storage Y

You decide to optimize the transhipment of depleted pellets from the plants to the storage sites (stopping on the way at Inspection Site A, of course) in order to minimize the overall monthly transportation cost. In this optimized solution, how many pellets will Plant Two be able to ship out to the Inspection Site A every month? (This question is not graded).

In [82]:

q1X = p.LpVariable('plant1_to_storageX', lowBound = 0, cat = 'Integer')
q1Y = p.LpVariable('plant1_to_storageY', lowBound = 0, cat = 'Integer')
q2X = p.LpVariable('plant2_to_storageX', lowBound = 0, cat = 'Integer')
q2Y = p.LpVariable('plant2_to_storageY', lowBound = 0, cat = 'Integer')

prob = p.LpProblem('Radioactive_Transportation', p.LpMinimize)

prob += (13.58+7.57)*q1X + (13.58+16.46)*q1Y + (16.54+7.57)*q2X + (16.54+16.46)*q2Y
prob += q1X + q2X == 305
prob += q1Y + q2Y == 325
prob += q1X + q1Y <= 280
prob += q2X + q2Y <= 360

prob.solve()

print('Optimal Solution: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ', item.varValue)

Optimal Solution:  17249.75
Solution Status: Optimal
plant1_to_storageX  =  0.0
plant1_to_storageY  =  280.0
plant2_to_storageX  =  305.0
plant2_to_storageY  =  45.0


Part 2

GCC has calculated the following transportation costs for Inspection Site B, in dollars per pellet moved:

It costs 12.6 dollars to move one pellet from Plant One to Inspection Site B

It costs 16.2 dollars to move one pellet from Plant Two to Inspection Site B

It costs 9.45 dollars to move one pellet from Inspection Site B to Storage X

It costs 15.64 dollars to move one pellet from Inspection Site B to Storage Y

As before, you will optimize the transhipment of depleted pellets from the plants to the storage sites, stopping first at the inspection sites, so as to minimize the overall monthly transportation cost. How much will this cost GCC every month? .

In [85]:
supply = ['Supply1', 'Supply2']
supplycap = [280, 360]
demand = ['Demand1', 'Demand2']
demandcap = [305, 325]

intermediate = ['Intermediate1','Intermediate2']
                   #int1      #int2
cost_sup_int   = np.array([[13.58,       12.6], #sup1
                  [16.54,       16.2]]) #sup2
                  
                #Dem1 Dem2
cost_int_dem = np.array([[7.57, 16.46], #int1
                [9.45, 15.64]]) #int2

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int)
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem)

prob = p.LpProblem('Radioactive_Waste', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route]) \
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s]
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d]

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand])
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  16693.550000000003
Solution Status: Optimal
route_Intermediate1_Demand1  =  305.0
route_Intermediate1_Demand2  =  0.0
route_Intermediate2_Demand1  =  0.0
route_Intermediate2_Demand2  =  325.0
route_Supply1_Intermediate1  =  0.0
route_Supply1_Intermediate2  =  280.0
route_Supply2_Intermediate1  =  305.0
route_Supply2_Intermediate2  =  45.0


New Plant and Storage Site

You have just received two e-mails from your boss. The first informs you that a third nuclear plant, called Plant Three, is ready to move out up to 340 depleted pellets per month. The second tells you that a third dedicated storage site, called Storage Z, is ready to receive a total of 320 depleted pellets per month from GCC's plants. GCC has calculated the following transportation costs for the new plant and storage site, in dollars per pellet moved:

It costs 16.37 dollars to move one pellet from Plant Three to Inspection Site A

It costs 10.81 dollars to move one pellet from Plant Three to Inspection Site B

It costs 15.69 dollars to move one pellet from Inspection Site A to Storage Z

It costs 15.41 dollars to move one pellet from Inspection Site B to Storage Z

Again you will optimize the transhipment of depleted pellets from the three plants to the three storage sites, stopping first at the inspection site, so as to minimize the overall monthly transportation cost. In this optimal solution, how many pellets is Plant One sending to Inspection Site B?

In this optimized solution, how much is the overall monthly transportation cost for GCC? (This question is graded.) Hint: The answer is more than 21000 but less than 26000.

In [86]:
supply = ['Supply1', 'Supply2', 'Supply3']
supplycap = [280, 360, 340]
demand = ['Demand1', 'Demand2', 'Demand3']
demandcap = [305, 325, 320]

intermediate = ['Intermediate1','Intermediate2']
                   #int1      #int2
cost_sup_int   = np.array([[13.58,       12.6], #sup1
                  [16.54,       16.2], #sup2
                  [16.37,       10.81]]) #sup3
                  
                #Dem1 #Dem2   #Dem3
cost_int_dem = np.array([[7.57, 16.46, 15.69], #int1
                [9.45, 15.64, 15.41]]) #int2

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int)
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem)

prob = p.LpProblem('Radioactive_Waste', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route]) \
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s]
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d]

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand])
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  24976.15
Solution Status: Optimal
route_Intermediate1_Demand1  =  305.0
route_Intermediate1_Demand2  =  0.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate2_Demand1  =  0.0
route_Intermediate2_Demand2  =  325.0
route_Intermediate2_Demand3  =  320.0
route_Supply1_Intermediate1  =  0.0
route_Supply1_Intermediate2  =  280.0
route_Supply2_Intermediate1  =  305.0
route_Supply2_Intermediate2  =  25.0
route_Supply3_Intermediate1  =  0.0
route_Supply3_Intermediate2  =  340.0


Part 4

GCC has learned that a new regulation, to be enacted soon, will require that no depleted pellets be stored at the plants past the monthly removal date. To acomodate this, the storage sites have increased the total number of pellets they are willing to receive every month, as follows:

Storage X is now ready to receive a total of 315 pellets per month

Storage Y is now ready to receive a total of 335 pellets per month

Storage Z is now ready to receive a total of 330 pellets per month

The new regulations also make it easier for pellets to be moved, which reduces all the transportation costs in the network by 3.13 percent. (This means you have to multiply all the previous transportation costs by 0.9687 in order to get the new costs.) One last time, you will optimize the transhipment of depleted pellets from the three plants to the three storage sites, stopping first at the inspection sites, so as to minimize the overall monthly transportation cost. (Hint: In this optimized solution, Plant Two is shipping 315 pellets per month to Inspection Site A.)

In this optimized solution, how much is the overall monthly transportation cost for GCC?

In [88]:
supply = ['Supply1', 'Supply2', 'Supply3']
supplycap = [280, 360, 340]
demand = ['Demand1', 'Demand2', 'Demand3']
demandcap = [315, 335, 330]

intermediate = ['Intermediate1','Intermediate2']
                   #int1      #int2
cost_sup_int   = np.array([[13.58,       12.6], #sup1
                  [16.54,       16.2], #sup2
                  [16.37,       10.81]]) #sup3
                  
                #Dem1 #Dem2   #Dem3
cost_int_dem = np.array([[7.57, 16.46, 15.69], #int1
                [9.45, 15.64, 15.41]]) #int2

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*0.9687)
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*0.9687)

prob = p.LpProblem('Radioactive_Waste', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route]) \
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s]
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d]

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand])
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  25042.590225
Solution Status: Optimal
route_Intermediate1_Demand1  =  315.0
route_Intermediate1_Demand2  =  0.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate2_Demand1  =  0.0
route_Intermediate2_Demand2  =  335.0
route_Intermediate2_Demand3  =  330.0
route_Supply1_Intermediate1  =  0.0
route_Supply1_Intermediate2  =  280.0
route_Supply2_Intermediate1  =  315.0
route_Supply2_Intermediate2  =  45.0
route_Supply3_Intermediate1  =  0.0
route_Supply3_Intermediate2  =  340.0


<h3>Graded Assignment 3: Notebooks for the world</h3>
    
Since you are attending the SC2x course, your best friend Sarah knows that you learned how to calculate, for a given group of cities with their Cartesian coordinates, a location for a distribution center that would minimize the distance. Thus, she asks you to calculate the coordinates for a distribution center, so that she can minimize its distance to ten cities in Mexico."

You found a map of Mexico online, and after asking Sarah for a list of the ten major cities in Mexico, you decide to use an improvised grid to assign horizontal and vertical coordinates to the cities. These coordinates, shown in the table below, use as origin a point in the top left corner of the map. You tried to make these units as close to kilometers as possible. However, you prefer to think of them as relative coordinates on an arbitrary Euclidean grid.

<img src="w1ga31.png">

"Do you have weights for these cities?", you ask your friend. 

"No", she says, "but let's assume they all have equal weight, a weight of 1 for example".

Part 0

Giving each city equal weight, calculate the location for the distribution center that would minimize the distance to the ten cities provided. Solve this as a continuous, distance-minimizing, location (or Weber) problem with equal weights for all cities.

Which of the following coordinates is nearest to the location found? This question is not graded, it just serves to help you check whether you are on the right track!



In [208]:
mexico = weber_problem(['Chihuahua','Ciudad Victoria','Durango','Mexico City',\
                                'Monterrey','Morelia','Oaxaca','Puebla','San Luis Potosi','Villahermosa'],\
                      [1,1,1,1,1,1,1,1,1,1],[110,159,120,160,148,145,179,170,153,203],[120,80,81,42,92,42,26,40,61,35])

mexico.find_weber()

weber x coordinate:  153
weber y coordinate:  61
weber avg. weighted dist:  33.07588744975341


(153, 61, 33.07588744975341)

Part 2

Curious about Sarah's request, you ask her what she is working on. She shares with you that together with her co-workers she is volunteering for an organization that will soon distribute notebooks to every school-aged child throughout Mexico. You suggest that solving this problem with equal weights is not good enough, since some cities surely have more schoolchildren than others. A quick online search provides you with approximate values for the populations of the different areas where the cities are located. Using these population values, you propose to your friend using weight values, as listed in the table below.

City	Weights

Chihuahua	926926

Ciudad Victoria	322322

Durango	519519

Mexico City	20439419

Monterrey	4524520

Morelia	785785

Oaxaca	650650

Puebla	2931929

San Luis Potosi	1042041

Villahermosa	640640

Using these weights, solve this as a continuous, demand-weighted distance-minimizing, location (or Weber) problem.

What is the horizontal coordinate?

What is the vertical coordinate?
 

In [209]:
locations = pd.DataFrame(index=['Chihuahua','Ciudad Victoria','Durango','Mexico City',\
                                'Monterrey','Morelia','Oaxaca','Puebla','San Luis Potosi','Villahermosa'],\
                         data={'wgt':[926926,322322,519519,20439419,4524520,785785,650650,2931929,1042041,640640],\
                               'x':[110,159,120,160,148,145,179,170,153,203],\
                               'y':[120,80,81,42,92,42,26,40,61,35]})


mexico = weber_problem(['Chihuahua','Ciudad Victoria','Durango','Mexico City',\
                                'Monterrey','Morelia','Oaxaca','Puebla','San Luis Potosi','Villahermosa'],\
                      [926926,322322,519519,20439419,4524520,785785,650650,2931929,1042041,640640],[110,159,120,160,148,145,179,170,153,203],[120,80,81,42,92,42,26,40,61,35])

mexico.find_weber()

weber x coordinate:  160
weber y coordinate:  42
weber avg. weighted dist:  14.234610918746556


(160, 42, 14.234610918746556)

Part 3

You decide to estimate the distance of distributing the notebooks from a DC in one of a set of selected cities to the school children in another city. Given that you only need a quick and dirty estimate, you assume an Euclidean space and a circuity factor of 1.25, which gives you the distances in the table below (where cities are identified using the first three letters in their name). Regardless of which city the DC is located in, there will also be transportation from the DC to the school children within that same city. For distances inside each city, you assume 5 distance units, except for Mexico City, for which you use 10 distance units since it is a larger city.

The distances between the cities are given in your arbitrary grid units, which may or may not be kilometers.

<img src="w1ga32.png">

After thinking about it for a while, your friend tells you that they may want to open two DCs. She wants these DCs to be located in two of the five candidate cities (Chihuahua, Mexico City, Monterrey, Puebla, San Luis Potosi), but they should not be located in the same city. The plan would be to distribute all the hundreds of thousands of notebooks through these two DCs, using trucks. Let's assume that the cost of opening and operating such a DC in any of these cities would be the about the same, and that the cost of outbound transportation from the DC to the cities is about the same on a per unit per mile basis.

In which two cities would you recommend that they open these two DCs, in order to minimize the outbound transportation costs? Select the answer options that apply. Use the same demand weights as in Part 2. (Hint: If you are puzzled by the fact that we are asking for this without providing any costs for DCs or trucks, you should start playing with different values and see how the model behaves.)

In [145]:
supply = ['Supply1', 'Supply2', 'Supply3', 'Supply4' , 'Supply5'] #no cap on supply

demand = ['Demand1', 'Demand2', 'Demand3', 'Demand4', 'Demand5', 'Demand6', 'Demand7', 'Demand8', 'Demand9', 'Demand10']

demandcap = [926926,322322,519519,20439419,4524520,785785,650650,2931929,1042041,640640]

                   #dem1 #dem2 #dem3 #dem4 #dem5 #dem6 #dem7 #dem8
cost_sup_dem   = np.array([[5,719,503,1158,590,1069,1458,1250,913, 1575], #sup1
                  [1158,475,698,10,643,188,310,127,253,545], #sup2
                  [590,203,376,643,5,626,911,706,393,990], #sup3
                  [1250,519,808,127,706,313,208,5,338,417], #sup4
                  [913,249,482,253,393,258,545,338,5,704]]) #sup5
                  

demandconstraint = dict(zip(demand,demandcap))

sup_dem_dict = p.makeDict([supply, demand], cost_sup_dem)

prob = p.LpProblem('Books', p.LpMinimize)

sup_to_dem_route = [(s,d) for s in supply for d in demand]

sup_to_dem_vars = p.LpVariable.dicts("route", (supply,demand), lowBound = 0, cat = 'Continuous') #amount of items travelling between DC and shelter
                  
sup_open = p.LpVariable.dicts("open", supply, cat = 'Binary') #binary variables DC open or close

prob += p.lpSum([sup_to_dem_vars[s][d]*sup_dem_dict[s][d] for (s,d) in sup_to_dem_route])\
        + p.lpSum([sup_open[s]*10000000 for s in supply]) #fixed cost of opening DC
                  
prob += p.lpSum([sup_open[s] for s in supply]) <= 2 #number of DC minimum must be open
                  
prob += p.lpSum([sup_open[s] for s in supply]) >= 2 #number of DC maximum must be open
                   
#for s in supply:
#    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) <= supplyconstraint[s] #constraint must be less than DC capacity
                  
for d in demand:
    prob += p.lpSum([sup_to_dem_vars[s][d] for s in supply]) == demandconstraint[d] #constraint must be equal to cities demand
    #using '<=' will result in a higher optimal cost compared to using '==' because in this problem there is no constraint on the supply

for s in supply:
    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) - 100000000000000000*sup_open[s] <= 0 #linking constraint if any channel from DC is flowing, the DC must be open
    
prob.solve()

print('minimum cost:', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost: 2389242876.0
Solution Status: Optimal
open_Supply1  =  0.0
open_Supply2  =  1.0
open_Supply3  =  1.0
open_Supply4  =  0.0
open_Supply5  =  0.0
route_Supply1_Demand1  =  0.0
route_Supply1_Demand10  =  0.0
route_Supply1_Demand2  =  0.0
route_Supply1_Demand3  =  0.0
route_Supply1_Demand4  =  0.0
route_Supply1_Demand5  =  0.0
route_Supply1_Demand6  =  0.0
route_Supply1_Demand7  =  0.0
route_Supply1_Demand8  =  0.0
route_Supply1_Demand9  =  0.0
route_Supply2_Demand1  =  0.0
route_Supply2_Demand10  =  640640.0
route_Supply2_Demand2  =  0.0
route_Supply2_Demand3  =  0.0
route_Supply2_Demand4  =  20439419.0
route_Supply2_Demand5  =  0.0
route_Supply2_Demand6  =  785785.0
route_Supply2_Demand7  =  650650.0
route_Supply2_Demand8  =  2931929.0
route_Supply2_Demand9  =  1042041.0
route_Supply3_Demand1  =  926926.0
route_Supply3_Demand10  =  0.0
route_Supply3_Demand2  =  322322.0
route_Supply3_Demand3  =  519519.0
route_Supply3_Demand4  =  0.0
route_Supply3_Demand5  =  4524520.0
route

Part 4

After some market research on carriers in Mexico, you calculate an average transportation cost of $1 per distance unit per 1001 notebooks 

(e.g. you would pay $100

to move 1001 notebooks 100 distance units). You also estimate that the fixed cost of operating each distribution center in one of these cities is $464,000 dollars per year. 

The yearly demands in the cities are approximated by the weights used in the previous parts.

Assuming that these values are correct and applicable throughout the country, how many distribution centers would you recommend opening?

Input your answer in integers.

Where would you recommend that they open these DCs, in order to minimize the total cost (fixed + transportation)?

In [147]:
supply = ['Supply1', 'Supply2', 'Supply3', 'Supply4' , 'Supply5'] #no cap on supply

demand = ['Demand1', 'Demand2', 'Demand3', 'Demand4', 'Demand5', 'Demand6', 'Demand7', 'Demand8', 'Demand9', 'Demand10']

demandcap = [926926,322322,519519,20439419,4524520,785785,650650,2931929,1042041,640640]

                   #dem1 #dem2 #dem3 #dem4 #dem5 #dem6 #dem7 #dem8
cost_sup_dem   = np.array([[5,719,503,1158,590,1069,1458,1250,913, 1575], #sup1
                  [1158,475,698,10,643,188,310,127,253,545], #sup2
                  [590,203,376,643,5,626,911,706,393,990], #sup3
                  [1250,519,808,127,706,313,208,5,338,417], #sup4
                  [913,249,482,253,393,258,545,338,5,704]]) #sup5


demandconstraint = dict(zip(demand,demandcap))

sup_dem_dict = p.makeDict([supply, demand], cost_sup_dem*1/1001) #transport cost of 1dollar per distance unit per 1001 books

prob = p.LpProblem('Books', p.LpMinimize)

sup_to_dem_route = [(s,d) for s in supply for d in demand]

sup_to_dem_vars = p.LpVariable.dicts("route", (supply,demand), lowBound = 0, cat = 'Continuous') #amount of items travelling between DC and shelter
                  
sup_open = p.LpVariable.dicts("open", supply, cat = 'Binary') #binary variables DC open or close

prob += p.lpSum([sup_to_dem_vars[s][d]*sup_dem_dict[s][d] for (s,d) in sup_to_dem_route])\
        + p.lpSum([sup_open[s]*464000 for s in supply]) #fixed cost of opening DC of 464000 dollars each
                  
prob += p.lpSum([sup_open[s] for s in supply]) <= 5 #number of DC minimum must be open
                  
prob += p.lpSum([sup_open[s] for s in supply]) >= 1 #number of DC maximum must be open
                   
#for s in supply:
#    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) <= supplyconstraint[s] #constraint must be less than DC capacity
                  
for d in demand:
    prob += p.lpSum([sup_to_dem_vars[s][d] for s in supply]) == demandconstraint[d] #constraint must be equal to cities demand
    #using '<=' will result in a higher optimal cost compared to using '==' because in this problem there is no constraint on the supply

for s in supply:
    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) - 100000000000000000*sup_open[s] <= 0 #linking constraint if any channel from DC is flowing, the DC must be open
    
prob.solve()

print('minimum cost:', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost: 3175608.0
Solution Status: Optimal
open_Supply1  =  1.0
open_Supply2  =  1.0
open_Supply3  =  1.0
open_Supply4  =  1.0
open_Supply5  =  0.0
route_Supply1_Demand1  =  926926.0
route_Supply1_Demand10  =  0.0
route_Supply1_Demand2  =  0.0
route_Supply1_Demand3  =  0.0
route_Supply1_Demand4  =  0.0
route_Supply1_Demand5  =  0.0
route_Supply1_Demand6  =  0.0
route_Supply1_Demand7  =  0.0
route_Supply1_Demand8  =  0.0
route_Supply1_Demand9  =  0.0
route_Supply2_Demand1  =  0.0
route_Supply2_Demand10  =  0.0
route_Supply2_Demand2  =  0.0
route_Supply2_Demand3  =  0.0
route_Supply2_Demand4  =  20439419.0
route_Supply2_Demand5  =  0.0
route_Supply2_Demand6  =  785785.0
route_Supply2_Demand7  =  0.0
route_Supply2_Demand8  =  0.0
route_Supply2_Demand9  =  1042041.0
route_Supply3_Demand1  =  0.0
route_Supply3_Demand10  =  0.0
route_Supply3_Demand2  =  322322.0
route_Supply3_Demand3  =  519519.0
route_Supply3_Demand4  =  0.0
route_Supply3_Demand5  =  4524520.0
route_Supply3_Demand6  =

<h1>Week 2</h1>

<h3>Practice Problem 2: Papper</h3>

Papper is a paper goods manufacturer. One of their main paper products is a paper towel, which is produced in two (2) plants in the contiguous U.S. From the plants, the paper towels are sent to nine (9) regional warehouses (RW), from where they are then shipped to customers across the U.S.

The distribution manager at Papper wants to reduce the number of direct transports in the network, reduce the inventory levels at the plants, and move inventory closer to the regional warehouses, by introducing a central warehouse (CW) where all finished paper towels are stored before further shipment. That is, the idea is to have the towels flowing from the plants to the CW and then to the RWs. The CW will have a small safety stock, but everything that comes into the CW during a week should also leave the CW during that week.

The distribution manager has found five (5) candidate locations for the CW that he is now choosing between. Capacity is not an issue at any of them.

All data, including weekly demand data (in number of boxes), is found here (Excel) or here (Libre Office). Assume transportation cost is $1 per mile per box of paper towels.

<img src="w2pp21.png">

Out of the candidate locations, which central warehouse location minimizes total costs (while meeting the constraints)?

In [15]:
supply = ['Supply1', 'Supply2']
supplycap = np.array([1500, 500])
supply_variable_cost = np.array([8,13])

demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5', 'Demand6','Demand7', 'Demand8', 'Demand9']
demandcap = np.array([140, 180, 240,210,175,130,320,280,160])

intermediate = ['Intermediate1','Intermediate2','Intermediate3','Intermediate4','Intermediate5']
intermediate_fixed_cost = np.array([100000,90000,80000,70000,60000])
intermediate_variable_cost = np.array([8,10,25,30,50])

                          #int1 #int2 #int3 #int4 #int5
cost_sup_int   = np.array([[77,   48,   41,   57,   94], #sup1
                           [54,    33,   21,   94,   44]]) #sup2
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7 #Dem8 #Dem9
cost_int_dem = np.array([[89,78,69,95,85,73,88,73,86], #int1
                         [90,91,63,61,94,58,89,90,55], #int2
                         [81,82,71,98,62,71,85,75,72], #int3
                         [73,97,55,75,86,54,62,99,59], #int4
                        [85,98,93,82,87,58,98,72,50]]) #int5

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

supplyvariablecost = dict(zip(supply,supply_variable_cost))

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost))

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per box
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*1) #cost of 1 dollar per mile per box

prob = p.LpProblem('Paper_Production', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([supplyvariablecost[s]*sup_to_int_vars[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) >= demandconstraint[d] #RW demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
for i in intermediate:
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 10000000000*int_open[i] <= 0 #linking constraint between flow and CW open
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 1 #number of DC minimum must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  349490.0
Solution Status: Optimal
open_Intermediate1  =  0.0
open_Intermediate2  =  1.0
open_Intermediate3  =  0.0
open_Intermediate4  =  0.0
open_Intermediate5  =  0.0
route_Intermediate1_Demand1  =  0.0
route_Intermediate1_Demand2  =  0.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate1_Demand4  =  0.0
route_Intermediate1_Demand5  =  0.0
route_Intermediate1_Demand6  =  0.0
route_Intermediate1_Demand7  =  0.0
route_Intermediate1_Demand8  =  0.0
route_Intermediate1_Demand9  =  0.0
route_Intermediate2_Demand1  =  140.0
route_Intermediate2_Demand2  =  180.0
route_Intermediate2_Demand3  =  240.0
route_Intermediate2_Demand4  =  210.0
route_Intermediate2_Demand5  =  175.0
route_Intermediate2_Demand6  =  130.0
route_Intermediate2_Demand7  =  320.0
route_Intermediate2_Demand8  =  280.0
route_Intermediate2_Demand9  =  160.0
route_Intermediate3_Demand1  =  0.0
route_Intermediate3_Demand2  =  0.0
route_Intermediate3_Demand3  =  0.0
route_Intermediate3_Demand4  =  0.0
route_

Part 2

Suppose the distribution manager wants at least 60% of regional demand to be within 70 miles of the central warehouse(s).

How many central warehouses are used in the optimal solution, if we introduce the constraint that 60% of regional demand must be within 70 miles of a central warehouse?


In [60]:
supply = ['Supply1', 'Supply2']
supplycap = np.array([1500, 500])
supply_variable_cost = np.array([8,13])

demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5', 'Demand6','Demand7', 'Demand8', 'Demand9']
demandcap = np.array([140, 180, 240,210,175,130,320,280,160])

intermediate = ['Intermediate1','Intermediate2','Intermediate3','Intermediate4','Intermediate5']
intermediate_fixed_cost = np.array([100000,90000,80000,70000,60000])
intermediate_variable_cost = np.array([8,10,25,30,50])

                          #int1 #int2 #int3 #int4 #int5
cost_sup_int   = np.array([[77,   48,   41,   57,   94], #sup1
                           [54,    33,   21,   94,   44]]) #sup2
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7 #Dem8 #Dem9
cost_int_dem = np.array([[89,78,69,95,85,73,88,73,86], #int1
                         [90,91,63,61,94,58,89,90,55], #int2
                         [81,82,71,98,62,71,85,75,72], #int3
                         [73,97,55,75,86,54,62,99,59], #int4
                        [85,98,93,82,87,58,98,72,50]]) #int5

dist_within_limit = cost_int_dem <= 70 #remember to use less than or equal to sign

dist_within_limit = dist_within_limit.astype(int) / sum(demandcap)

dist_within_limit_dict = p.makeDict([intermediate, demand], dist_within_limit)

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

supplyvariablecost = dict(zip(supply,supply_variable_cost))

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost))

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per box
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*1) #cost of 1 dollar per mile per box

prob = p.LpProblem('Paper_Production', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([supplyvariablecost[s]*sup_to_int_vars[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d] #RW demand constraint, changed to hard constraint in this case

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
for i in intermediate:
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 100000000000000*int_open[i] <= 0 #linking constraint between flow and CW open
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 5 #number of DC max must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1 #number of DC minimal must be open

prob += p.lpSum([int_to_dem_vars[i][d]*dist_within_limit_dict[i][d] for (i,d) in int_to_dem_route]) >= 0.6 #60% of demand must be within less than 70 miles of CW
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  490882.0
Solution Status: Optimal
open_Intermediate1  =  0.0
open_Intermediate2  =  1.0
open_Intermediate3  =  1.0
open_Intermediate4  =  1.0
open_Intermediate5  =  0.0
route_Intermediate1_Demand1  =  0.0
route_Intermediate1_Demand2  =  0.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate1_Demand4  =  0.0
route_Intermediate1_Demand5  =  0.0
route_Intermediate1_Demand6  =  0.0
route_Intermediate1_Demand7  =  0.0
route_Intermediate1_Demand8  =  0.0
route_Intermediate1_Demand9  =  0.0
route_Intermediate2_Demand1  =  0.0
route_Intermediate2_Demand2  =  0.0
route_Intermediate2_Demand3  =  240.0
route_Intermediate2_Demand4  =  210.0
route_Intermediate2_Demand5  =  0.0
route_Intermediate2_Demand6  =  130.0
route_Intermediate2_Demand7  =  134.0
route_Intermediate2_Demand8  =  0.0
route_Intermediate2_Demand9  =  160.0
route_Intermediate3_Demand1  =  140.0
route_Intermediate3_Demand2  =  180.0
route_Intermediate3_Demand3  =  0.0
route_Intermediate3_Demand4  =  0.0
route_Inte

Part 3

Suppose the same service constraints as in Part 2 applies.

After investigating the transport market, the manager realizes that he has completely overestimated the inbound transportation costs. In fact, inbound transportation costs can be reduced to $0.3 per mile and box.

How does this change the optimal network design (compared to Part 2)?

In [74]:
supply = ['Supply1', 'Supply2']
supplycap = np.array([1500, 500])
supply_variable_cost = np.array([8,13])

demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5', 'Demand6','Demand7', 'Demand8', 'Demand9']
demandcap = np.array([140, 180, 240,210,175,130,320,280,160])

intermediate = ['Intermediate1','Intermediate2','Intermediate3','Intermediate4','Intermediate5']
intermediate_fixed_cost = np.array([100000,90000,80000,70000,60000])
intermediate_variable_cost = np.array([8,10,25,30,50])

                          #int1 #int2 #int3 #int4 #int5
cost_sup_int   = np.array([[77,   48,   41,   57,   94], #sup1
                           [54,    33,   21,   94,   44]]) #sup2
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7 #Dem8 #Dem9
cost_int_dem = np.array([[89,78,69,95,85,73,88,73,86], #int1
                         [90,91,63,61,94,58,89,90,55], #int2
                         [81,82,71,98,62,71,85,75,72], #int3
                         [73,97,55,75,86,54,62,99,59], #int4
                        [85,98,93,82,87,58,98,72,50]]) #int5

dist_within_limit = cost_int_dem <= 70 #remember to use less than or equal to sign

dist_within_limit = dist_within_limit.astype(int) / sum(demandcap)

dist_within_limit_dict = p.makeDict([intermediate, demand], dist_within_limit)

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

supplyvariablecost = dict(zip(supply,supply_variable_cost))

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost))

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*0.3) #cost for inbound transport changed to 0.3 
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*1) #cost of 1 dollar per mile per box

prob = p.LpProblem('Paper_Production', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([supplyvariablecost[s]*sup_to_int_vars[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d] #RW demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
for i in intermediate:
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 100000000000000*int_open[i] <= 0 #linking constraint between flow and CW open
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 5 #number of DC max must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1 #number of DC minimal must be open

prob += p.lpSum([int_to_dem_vars[i][d]*dist_within_limit_dict[i][d] for (i,d) in int_to_dem_route]) >= 0.6
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  437072.5
Solution Status: Optimal
open_Intermediate1  =  0.0
open_Intermediate2  =  1.0
open_Intermediate3  =  1.0
open_Intermediate4  =  1.0
open_Intermediate5  =  0.0
route_Intermediate1_Demand1  =  0.0
route_Intermediate1_Demand2  =  0.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate1_Demand4  =  0.0
route_Intermediate1_Demand5  =  0.0
route_Intermediate1_Demand6  =  0.0
route_Intermediate1_Demand7  =  0.0
route_Intermediate1_Demand8  =  0.0
route_Intermediate1_Demand9  =  0.0
route_Intermediate2_Demand1  =  140.0
route_Intermediate2_Demand2  =  180.0
route_Intermediate2_Demand3  =  240.0
route_Intermediate2_Demand4  =  210.0
route_Intermediate2_Demand5  =  0.0
route_Intermediate2_Demand6  =  130.0
route_Intermediate2_Demand7  =  0.0
route_Intermediate2_Demand8  =  0.0
route_Intermediate2_Demand9  =  160.0
route_Intermediate3_Demand1  =  0.0
route_Intermediate3_Demand2  =  0.0
route_Intermediate3_Demand3  =  0.0
route_Intermediate3_Demand4  =  0.0
route_Interm

<h3>Practice Problem 3: Cocup</h3>
    
Cocup is a paper cup manufacturer making generic coffee cups used for take away coffee around the world. They have one (1) manufacturing plant and two (2) Distribution Centers (DCs), as well as eight (8) regional sales offices (SOs) with warehouses on site. Paper cups are made at the plant and shipped by external carriers through the DCs to the sales offices.

The weekly demands at the eight SOs are shown below.

<img src ="w2pp31.png">

Last week the Director of Marketing at Cocup signed a contract with a growing coffee retail chain, making Cocup the retailer's sole supplier of coffee cups. The retailer will be supplied through sales offices SO1, SO2 and SO3, increasing the demand at these offices to:

SO1 = 121 thousand cups

SO2 = 96 thousand cups

SO3 = 166 thousand cups

Since the manufacturing plant is operating close to its capacity of 1,250 thousand cups per week, Cocup has started to investigate whether they should use a contract manufacturer, now that the demand for their products is increasing. They have found one contract manufacturer that meets their requirements. This contractor can provide the cups at 50dollars per thousand cups and has the capacity to provide 200 thousand cups per week. This is more expensive than the 10dollars per thousand cups of producing in-house, but because the contractor is located closer to the two DCs than Cocup's plant is, it will lead to more production capacity closer to the DCs.

Cocup has also started to investigate whether they should rent space from a third party's distribution center to support the sales offices. They are considering three (3) more DCs. The fixed and variables costs of these Potential DCs are found in the table below together with the fixed and variable costs of the current DCs (DC 1 and DC 2).

<img src = "w2pp32.png">

The distances from the DCs to the sales offices are

<img src = "w2pp33.png">

and the distances from the plants to the DCs are

<img src = "w2pp34.png">

There are no capacity limits at the DCs at this point. Transportation costs are 1$ per thousand cups and mile, for both inbound and outbound transportation.

The logistics manager of Cocup has asked you to build an optimization model to support their decision-making. The objective of the model is to minimize costs.

Based on the results of your model, would you recommend the logistics manager to use the contract manufacturer with the current distribution network when demand increases? Note that the current network has two (2) DCs, and eight (8) SOs.

In [49]:
supply = ['Supply1', 'Supply2']
supplycap = np.array([1250, 250])
supply_variable_cost = np.array([10,50])

demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5', 'Demand6','Demand7', 'Demand8']
demandcap = np.array([121,96,166,175,210,130,180,160])

intermediate = ['Intermediate1','Intermediate2','Intermediate3','Intermediate4','Intermediate5']
intermediate_fixed_cost = np.array([100,100,800000000,700000000,600000000]) #set DC3 - 5's fixed cost to be very high which is the same as saying only DC1 and DC2 will be used
intermediate_variable_cost = np.array([10,10,25,30,50])

                          #int1 #int2 #int3 #int4 #int5
cost_sup_int   = np.array([[77,   48,   41,   57,   94], #sup1
                           [54,    33,   21,   94,   44]]) #sup2
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7 #Dem8
cost_int_dem = np.array([[62,55,114,106,51,76,122,80], #int1
                         [63,92,90,50,88,47,136,47], #int2
                         [76,85,56,65,129,40,89,100], #int3
                         [42,128,74,70,45,56,81,131], #int4
                        [77,135,118,59,124,93,41,94]]) #int5

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

supplyvariablecost = dict(zip(supply,supply_variable_cost))

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost))

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per box
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*1) #cost of 1 dollar per mile per box

prob = p.LpProblem('Cup_Production', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([supplyvariablecost[s]*sup_to_int_vars[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d] #RW demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
for i in intermediate:
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 10000000000*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 2 #number of DC minimum must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 2
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  178671.0
Solution Status: Optimal
open_Intermediate1  =  1.0
open_Intermediate2  =  1.0
open_Intermediate3  =  0.0
open_Intermediate4  =  0.0
open_Intermediate5  =  0.0
route_Intermediate1_Demand1  =  0.0
route_Intermediate1_Demand2  =  96.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate1_Demand4  =  0.0
route_Intermediate1_Demand5  =  210.0
route_Intermediate1_Demand6  =  0.0
route_Intermediate1_Demand7  =  0.0
route_Intermediate1_Demand8  =  0.0
route_Intermediate2_Demand1  =  121.0
route_Intermediate2_Demand2  =  0.0
route_Intermediate2_Demand3  =  166.0
route_Intermediate2_Demand4  =  175.0
route_Intermediate2_Demand5  =  0.0
route_Intermediate2_Demand6  =  130.0
route_Intermediate2_Demand7  =  180.0
route_Intermediate2_Demand8  =  160.0
route_Intermediate3_Demand1  =  0.0
route_Intermediate3_Demand2  =  0.0
route_Intermediate3_Demand3  =  0.0
route_Intermediate3_Demand4  =  0.0
route_Intermediate3_Demand5  =  0.0
route_Intermediate3_Demand6  =  0.0
route_Int

Part 2

The manager is uncertain about how much demand will actually increase at the three sales offices. Consequently, the manager is interested in understanding how sensitive your conclusions are to the assumptions about the increase in demand.

Suppose demand increases by 20% at each of the three sales offices instead, so that the demands are SO1: 132, SO2: 96, and SO3: 186. Under this scenario, using your model, would you recommend the logistics manager to use the contract manufacturer with the current distribution network?

In [51]:
supply = ['Supply1', 'Supply2']
supplycap = np.array([1250, 250])
supply_variable_cost = np.array([10,50])

demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5', 'Demand6','Demand7', 'Demand8']
demandcap = np.array([132,96,186,175,210,130,180,160])

intermediate = ['Intermediate1','Intermediate2','Intermediate3','Intermediate4','Intermediate5']
intermediate_fixed_cost = np.array([100,100,800000000,700000000,600000000]) #set DC3 - 5's fixed cost to be very high which is the same as saying only DC1 and DC2 will be used
intermediate_variable_cost = np.array([10,10,25,30,50])

                          #int1 #int2 #int3 #int4 #int5
cost_sup_int   = np.array([[77,   48,   41,   57,   94], #sup1
                           [54,    33,   21,   94,   44]]) #sup2
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7 #Dem8
cost_int_dem = np.array([[62,55,114,106,51,76,122,80], #int1
                         [63,92,90,50,88,47,136,47], #int2
                         [76,85,56,65,129,40,89,100], #int3
                         [42,128,74,70,45,56,81,131], #int4
                        [77,135,118,59,124,93,41,94]]) #int5

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

supplyvariablecost = dict(zip(supply,supply_variable_cost))

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost))

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per box
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*1) #cost of 1 dollar per mile per box

prob = p.LpProblem('Cup_Production', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([supplyvariablecost[s]*sup_to_int_vars[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d] #RW demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
for i in intermediate:
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 10000000000*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 2 #number of DC minimum must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 2
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  183595.0
Solution Status: Optimal
open_Intermediate1  =  1.0
open_Intermediate2  =  1.0
open_Intermediate3  =  0.0
open_Intermediate4  =  0.0
open_Intermediate5  =  0.0
route_Intermediate1_Demand1  =  0.0
route_Intermediate1_Demand2  =  96.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate1_Demand4  =  0.0
route_Intermediate1_Demand5  =  210.0
route_Intermediate1_Demand6  =  0.0
route_Intermediate1_Demand7  =  0.0
route_Intermediate1_Demand8  =  0.0
route_Intermediate2_Demand1  =  132.0
route_Intermediate2_Demand2  =  0.0
route_Intermediate2_Demand3  =  186.0
route_Intermediate2_Demand4  =  175.0
route_Intermediate2_Demand5  =  0.0
route_Intermediate2_Demand6  =  130.0
route_Intermediate2_Demand7  =  180.0
route_Intermediate2_Demand8  =  160.0
route_Intermediate3_Demand1  =  0.0
route_Intermediate3_Demand2  =  0.0
route_Intermediate3_Demand3  =  0.0
route_Intermediate3_Demand4  =  0.0
route_Intermediate3_Demand5  =  0.0
route_Intermediate3_Demand6  =  0.0
route_Int

Part 3

After receiving additional information from Marketing, the logistics manager is fairly confident that the original demand estimates from (Part 1) are correct (SO1: 121, SO2: 96, and SO3: 166 thousand cups). For the remainder of the analyses, she wants you to use these values.

The logistics manager is also interested in exploring adding more DCs to the network, if it reduces total costs. If Cocup does not use the contractor, what is the optimal number of DCs?

Part 4

When you show your analysis to the manager she comments that the solution seems to be largely driven by the transportation costs. She tells you that based on the contracts they have with their carriers, there is much uncertainty about the real transportation costs. In fact, fluctuations in oil prices have lead to very volatile prices over the last years and she anticipates this may continue.

Since transportation costs are historically low at this point in time, she wants you to investigate what will happen if costs were to increase in the close future. She therefore asks you to redo the analysis from Part 3 with higher transportation costs.

Consider the same situation as in Part 3. If outbound transportation costs increase by 50%, which of the following statements is/are correct?

In [72]:
supply = ['Supply1', 'Supply2']
supplycap = np.array([1250, 250])
supply_variable_cost = np.array([10,50])

demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5', 'Demand6','Demand7', 'Demand8']
demandcap = np.array([121,96,166,175,210,130,180,160])

intermediate = ['Intermediate1','Intermediate2','Intermediate3','Intermediate4','Intermediate5']
intermediate_fixed_cost = np.array([100,100,80,70,60]) #all DC has potential to be used
intermediate_variable_cost = np.array([10,10,25,30,50])

                          #int1 #int2 #int3 #int4 #int5
cost_sup_int   = np.array([[77,   48,   41,   57,   94], #sup1
                           [54,    33,   21,   94,   44]]) #sup2
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7 #Dem8
cost_int_dem = np.array([[62,55,114,106,51,76,122,80], #int1
                         [63,92,90,50,88,47,136,47], #int2
                         [76,85,56,65,129,40,89,100], #int3
                         [42,128,74,70,45,56,81,131], #int4
                        [77,135,118,59,124,93,41,94]]) #int5

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

supplyvariablecost = dict(zip(supply,supply_variable_cost))

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost)) 

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per box
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*1.5) #outbound transport cost increased by 50%

prob = p.LpProblem('Cup_Production', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([supplyvariablecost[s]*sup_to_int_vars[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) >= demandconstraint[d] #RW demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
for i in intermediate:
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 10000000000*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 5 #number of DC maximum must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1 #number of DC minimum must be open
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  199962.0
Solution Status: Optimal
open_Intermediate1  =  1.0
open_Intermediate2  =  1.0
open_Intermediate3  =  1.0
open_Intermediate4  =  1.0
open_Intermediate5  =  1.0
route_Intermediate1_Demand1  =  0.0
route_Intermediate1_Demand2  =  96.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate1_Demand4  =  0.0
route_Intermediate1_Demand5  =  0.0
route_Intermediate1_Demand6  =  0.0
route_Intermediate1_Demand7  =  0.0
route_Intermediate1_Demand8  =  0.0
route_Intermediate2_Demand1  =  0.0
route_Intermediate2_Demand2  =  0.0
route_Intermediate2_Demand3  =  0.0
route_Intermediate2_Demand4  =  175.0
route_Intermediate2_Demand5  =  0.0
route_Intermediate2_Demand6  =  0.0
route_Intermediate2_Demand7  =  0.0
route_Intermediate2_Demand8  =  160.0
route_Intermediate3_Demand1  =  0.0
route_Intermediate3_Demand2  =  0.0
route_Intermediate3_Demand3  =  166.0
route_Intermediate3_Demand4  =  0.0
route_Intermediate3_Demand5  =  0.0
route_Intermediate3_Demand6  =  130.0
route_Intermedi

Part 5

The manager had another comment on you analysis in Part 3. Suppose outbound transportation costs remain at 1dollar per 1,000 cups and mile. Even with several DCs, this leads to a large portion of demand at the SOs being more than 50 miles from a DC.

Consider the same situation as in Part 3. If 75% of demand should be within 50 miles of a DC, what implications does it have on the volumes at DC 2 compared to a solution where there is no such constraint?

In [77]:
supply = ['Supply1', 'Supply2']
supplycap = np.array([1250, 250])
supply_variable_cost = np.array([10,50])

demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5', 'Demand6','Demand7', 'Demand8']
demandcap = np.array([121,96,166,175,210,130,180,160])

intermediate = ['Intermediate1','Intermediate2','Intermediate3','Intermediate4','Intermediate5']
intermediate_fixed_cost = np.array([100,100,80,70,60]) #all DC has potential to be used
intermediate_variable_cost = np.array([10,10,25,30,50])

                          #int1 #int2 #int3 #int4 #int5
cost_sup_int   = np.array([[77,   48,   41,   57,   94], #sup1
                           [54,    33,   21,   94,   44]]) #sup2
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7 #Dem8
cost_int_dem = np.array([[62,55,114,106,51,76,122,80], #int1
                         [63,92,90,50,88,47,136,47], #int2
                         [76,85,56,65,129,40,89,100], #int3
                         [42,128,74,70,45,56,81,131], #int4
                        [77,135,118,59,124,93,41,94]]) #int5

dist_within_limit = cost_int_dem <= 50 #remember to use less than or equal to sign

dist_within_limit = dist_within_limit.astype(int) / sum(demandcap)

dist_within_limit_dict = p.makeDict([intermediate, demand], dist_within_limit)


supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

supplyvariablecost = dict(zip(supply,supply_variable_cost))

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost)) 

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per box
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*1) #outbound transport cost increased by 50%

prob = p.LpProblem('Cup_Production', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([supplyvariablecost[s]*sup_to_int_vars[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d] #RW demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
for i in intermediate:
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 1000000000000*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 5 #number of DC maximum must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1 #number of DC minimum must be open

prob += p.lpSum([int_to_dem_vars[i][d]*dist_within_limit_dict[i][d] for (i,d) in int_to_dem_route]) >= 0.75 # 75% of demand must be within limit
    

prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

    
#Explanation

#DC 2 is cheap to operate in scale but is relatively far away from several of the SOs. 
#Increasing the LOS means that we should use one more DC (DC 5) and reduce the volume at DC 2. 
#This is once again a trade-off between service and cost.

#To see this, include the LOS-constraint and put it on a low level (say 50%). We see that the flow 
#through DC2 is 586 thousand cups. Then increase the constraint to 75%. The flow through DC2 is 
#now reduced to 465 thousand cups.



minimum cost:  169913.0
Solution Status: Optimal
open_Intermediate1  =  1.0
open_Intermediate2  =  1.0
open_Intermediate3  =  1.0
open_Intermediate4  =  1.0
open_Intermediate5  =  1.0
route_Intermediate1_Demand1  =  0.0
route_Intermediate1_Demand2  =  96.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate1_Demand4  =  0.0
route_Intermediate1_Demand5  =  0.0
route_Intermediate1_Demand6  =  0.0
route_Intermediate1_Demand7  =  0.0
route_Intermediate1_Demand8  =  0.0
route_Intermediate2_Demand1  =  0.0
route_Intermediate2_Demand2  =  0.0
route_Intermediate2_Demand3  =  0.0
route_Intermediate2_Demand4  =  175.0
route_Intermediate2_Demand5  =  0.0
route_Intermediate2_Demand6  =  130.0
route_Intermediate2_Demand7  =  0.0
route_Intermediate2_Demand8  =  160.0
route_Intermediate3_Demand1  =  0.0
route_Intermediate3_Demand2  =  0.0
route_Intermediate3_Demand3  =  166.0
route_Intermediate3_Demand4  =  0.0
route_Intermediate3_Demand5  =  0.0
route_Intermediate3_Demand6  =  0.0
route_Intermedi

Part 6

The last comment the manager had about your analysis in Part 3 has to do with prioritization. Even though it may be optimal to use several third party DCs, Cocup may want to start with one to ensure that all processes are working before ramping up.

Consider the same situation as in Part 3. If Cocup was to use only one of the third party DCs, which one should it use?

In [82]:
supply = ['Supply1', 'Supply2']
supplycap = np.array([1250, 250])
supply_variable_cost = np.array([10,50])

demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5', 'Demand6','Demand7', 'Demand8']
demandcap = np.array([121,96,166,175,210,130,180,160])

intermediate = ['Intermediate1','Intermediate2','Intermediate3','Intermediate4','Intermediate5']
intermediate_fixed_cost = np.array([100,100,80,70,60]) #all DC has potential to be used
intermediate_variable_cost = np.array([10,10,25,30,50])

                          #int1 #int2 #int3 #int4 #int5
cost_sup_int   = np.array([[77,   48,   41,   57,   94], #sup1
                           [54,    33,   21,   94,   44]]) #sup2
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7 #Dem8
cost_int_dem = np.array([[62,55,114,106,51,76,122,80], #int1
                         [63,92,90,50,88,47,136,47], #int2
                         [76,85,56,65,129,40,89,100], #int3
                         [42,128,74,70,45,56,81,131], #int4
                        [77,135,118,59,124,93,41,94]]) #int5

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

supplyvariablecost = dict(zip(supply,supply_variable_cost))

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost)) 

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per box
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*1) 

prob = p.LpProblem('Cup_Production', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([supplyvariablecost[s]*sup_to_int_vars[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) >= demandconstraint[d] #RW demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
for i in intermediate:
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 10000000000*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 3 #number of DC maximum must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1 #number of DC minimum must be open

prob += int_open['Intermediate1'] == 1 #force constraint Intermediate1 must be open

prob += int_open['Intermediate2'] == 1 #force constraint Intermediate2 must be open
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  167415.0
Solution Status: Optimal
open_Intermediate1  =  1.0
open_Intermediate2  =  1.0
open_Intermediate3  =  1.0
open_Intermediate4  =  0.0
open_Intermediate5  =  0.0
route_Intermediate1_Demand1  =  0.0
route_Intermediate1_Demand2  =  96.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate1_Demand4  =  0.0
route_Intermediate1_Demand5  =  210.0
route_Intermediate1_Demand6  =  0.0
route_Intermediate1_Demand7  =  0.0
route_Intermediate1_Demand8  =  0.0
route_Intermediate2_Demand1  =  121.0
route_Intermediate2_Demand2  =  0.0
route_Intermediate2_Demand3  =  0.0
route_Intermediate2_Demand4  =  175.0
route_Intermediate2_Demand5  =  0.0
route_Intermediate2_Demand6  =  130.0
route_Intermediate2_Demand7  =  0.0
route_Intermediate2_Demand8  =  160.0
route_Intermediate3_Demand1  =  0.0
route_Intermediate3_Demand2  =  0.0
route_Intermediate3_Demand3  =  166.0
route_Intermediate3_Demand4  =  0.0
route_Intermediate3_Demand5  =  0.0
route_Intermediate3_Demand6  =  0.0
route_Inter

<h3>Practice Problem 4: Uncertainty at Cocup</h3>

Consider again Cocup, the paper cup manufacturer. Cocup has one (1) manufacturing plant and two (2) Distribution Centers (DCs), as well as eight (8) regional sales offices (SOs) with warehouses on site. Paper cups are made at the plant and shipped by external carriers through the DCs to the sales offices.

As you may remember, the manufacturing plant is operating close to its capacity of 1,250 thousand cups per week. Because of this, Cocup has started to investigate if they should use a contract manufacturer. They have found one contract manufacturer that met their requirements. This contractor can provide the cups at 50dollar per thousand cups and has the capacity to provide 200 thousand cups per week. This is more expensive than the 10dollar per thousand cups of producing in-house, but because the contractor is located closer to the two DCs than Cocup's plant is, it will lead to more production capacity closer to the DCs.

Cocup has also started to investigate whether they should rent space from a third party's distribution center to support the sales offices. They are considering three (3) more DCs, to add to the two (2) DCs they already have in their network. Transportation costs are 1$ per thousand cups and mile, for both inbound and outbound transportation.

As you may remember, the logistics manager of Cocup was skeptical about the demand estimates. The report from Marketing calmed him down momentarily, but still he cannot let go of the idea of the demand estimates being off.

He asks you to perform a few more analyses, based on the model you created last week.

To address the uncertainty in demand, he wants you to assume demands at the Sales Regions are normally distributed with a CV of 0.1. In particular, he wants you to run a number of scenarios based on realizations from these normal distributions.

Run your model over 10 generated demand realizations, where demands are assumed to follow normal distributions with a CV (Coef. of Variation; CV = stddev/mean) of 0.1.

Based on your FIRST RUN, should Cocup use the contractor?

In [3]:
supply = ['Supply1', 'Supply2']
supplycap = np.array([1250, 250])
supply_variable_cost = np.array([10,50])

demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5', 'Demand6','Demand7', 'Demand8']
demandcap = np.array([121,96,166,175,210,130,180,160])
demandcap = [int(stats.norm(demandcap[i], 0.1*demandcap[i]).ppf(random.random())) for i in range(len(demandcap))] 
#add randomness to the demand which is modeled as a normal distribution with a CV of 0.1


intermediate = ['Intermediate1','Intermediate2','Intermediate3','Intermediate4','Intermediate5']
intermediate_fixed_cost = np.array([100,100,80,70,60]) #all DC has potential to be used
intermediate_variable_cost = np.array([10,10,25,30,50])

                          #int1 #int2 #int3 #int4 #int5
cost_sup_int   = np.array([[77,   48,   41,   57,   94], #sup1
                           [54,    33,   21,   94,   44]]) #sup2
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7 #Dem8
cost_int_dem = np.array([[62,55,114,106,51,76,122,80], #int1
                         [63,92,90,50,88,47,136,47], #int2
                         [76,85,56,65,129,40,89,100], #int3
                         [42,128,74,70,45,56,81,131], #int4
                        [77,135,118,59,124,93,41,94]]) #int5

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

supplyvariablecost = dict(zip(supply,supply_variable_cost))

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost)) 

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per box
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*1) 

prob = p.LpProblem('Cup_Production', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([supplyvariablecost[s]*sup_to_int_vars[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d] #RW demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
for i in intermediate:
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 10000000000*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 3 #number of DC maximum must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1 #number of DC minimum must be open

prob += int_open['Intermediate1'] == 1 #force constraint Intermediate1 must be open

prob += int_open['Intermediate2'] == 1 #force constraint Intermediate2 must be open
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  168325.0
Solution Status: Optimal
open_Intermediate1  =  1.0
open_Intermediate2  =  1.0
open_Intermediate3  =  1.0
open_Intermediate4  =  0.0
open_Intermediate5  =  0.0
route_Intermediate1_Demand1  =  0.0
route_Intermediate1_Demand2  =  86.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate1_Demand4  =  0.0
route_Intermediate1_Demand5  =  201.0
route_Intermediate1_Demand6  =  0.0
route_Intermediate1_Demand7  =  0.0
route_Intermediate1_Demand8  =  0.0
route_Intermediate2_Demand1  =  138.0
route_Intermediate2_Demand2  =  0.0
route_Intermediate2_Demand3  =  0.0
route_Intermediate2_Demand4  =  176.0
route_Intermediate2_Demand5  =  0.0
route_Intermediate2_Demand6  =  132.0
route_Intermediate2_Demand7  =  0.0
route_Intermediate2_Demand8  =  162.0
route_Intermediate3_Demand1  =  0.0
route_Intermediate3_Demand2  =  0.0
route_Intermediate3_Demand3  =  177.0
route_Intermediate3_Demand4  =  0.0
route_Intermediate3_Demand5  =  0.0
route_Intermediate3_Demand6  =  0.0
route_Inter

<h3>Practice Problem 5: RCH Industries' Differentiation</h3>
    
Ralph Calvin Hilfiger (RCH) Industries - the clothing manufacturer from Week 1 - has found that different clothing styles have very different demands in the five (5) markets. For instance, the more stylish items sell relatively more in Europe, while the more leisurely items seem to do much better in the U.S. To accommodate this, they want to update the network design, and rethink the product mix produced in each of the five factories.

As before, RCH distributes their products from their five (5) factories through (2) cross docking facilities (XD) on to five (5) regional distribution centers (DCs), each of which supports a specific market. The clothes are transported through the network in small plastic totes via truck. There are no capacity constraints at the cross docks at this point.

RCH has grouped their clothing items into two major style categories: Stylish and Leisure. The demand for the categories in the five markets, as well as production capacities and transportation costs are found below

<img src="w2pp51.png">

Suppose RCH wishes to plan the flow of products from Factories to Crossdocks to Distribution Centers in order to minimize total costs, subject to the constraints. Build a spreadsheet model of the network.

In the optimal solution, how many factories are dedicated to producing only one style group?

In [16]:
types = ['Type1', 'Type2']
supply = ['Supply1', 'Supply2','Supply3','Supply4','Supply5']
                        #sup1 #sup2 #sup3 #sup4 #sup5
supplycap = np.array([[150,   300,  90,   140,  220],#type1
                      [200,   300,  70,    30,  220]])#type2


demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5']
                      #dem1 #dem2 #dem3 #dem4 #dem5
demandcap = np.array([[130,  45,   70,   100,  5], #type1
                      [15,   45,   40,   100,175]])#type2

intermediate = ['Intermediate1','Intermediate2']

                            #sup1       #sup2       #sup3       #sup4      #sup5
                            #int1#int2 #int1#int2  #int1#int2  #int1#int2 #int1#int2
cost_sup_int   = np.array([[[30,   50],[23,    66],[35,    14],[70,   12],[65,   70]], #type1
                           [[33,   55],[25,    73],[39,    15],[77,   13],[12,   14]]]) #type2
                  
                            #int1                     #int2
                         #Dem1#Dem2#Dem3#Dem4#Dem5   #Dem1#Dem2#Dem3#Dem4#Dem5
cost_int_dem = np.array([[[12, 25,  22,  40,    41], [65,  22,  23,  12,   15]],#type1
                         [[13, 28,  24,  44,    45], [72,  24,  25,  13,   17]]]) #type2

supplyconstraint = p.makeDict([types, supply], supplycap)
demandconstraint = p.makeDict([types, demand], demandcap)

sup_int_dict = p.makeDict([types, supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per unit
int_dem_dict = p.makeDict([types, intermediate, demand], cost_int_dem*1) 

prob = p.LpProblem('Clothes_Production', p.LpMinimize)

sup_to_int_route = [(t,s,i) for t in types for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (types, supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(t,i,d) for t in types for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (types, intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[t][s][i]*sup_int_dict[t][s][i] for (t,s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[t][i][d]*int_dem_dict[t][i][d] for (t,i,d) in int_to_dem_route])

for t in types: #note the posisitoning of the for loops and how it corresponds to the constraints we are trying to model
    for s in supply:
        prob += p.lpSum([sup_to_int_vars[t][s][i] for i in intermediate]) <= supplyconstraint[t][s] #plant cap constraint

    for d in demand:
        prob += p.lpSum([int_to_dem_vars[t][i][d] for i in intermediate]) >= demandconstraint[t][d] #RW demand constraint

    for i in intermediate:
        prob += p.lpSum([sup_to_int_vars[t][s][i] for s in supply]) == p.lpSum([int_to_dem_vars[t][i][d] for d in demand]) #conservation of flow constraint

prob += p.lpSum([sup_to_int_vars[t]['Supply1'][i] for t in types for i in intermediate]) <= 200 #constraint of total production in Sup1 must not exceed a certain number
prob += p.lpSum([sup_to_int_vars[t]['Supply2'][i] for t in types for i in intermediate]) <= 300 #constraint of total production in Sup2 must not exceed a certain number
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  23630.0
Solution Status: Optimal
route_Type1_Intermediate1_Demand1  =  130.0
route_Type1_Intermediate1_Demand2  =  0.0
route_Type1_Intermediate1_Demand3  =  0.0
route_Type1_Intermediate1_Demand4  =  0.0
route_Type1_Intermediate1_Demand5  =  0.0
route_Type1_Intermediate2_Demand1  =  0.0
route_Type1_Intermediate2_Demand2  =  45.0
route_Type1_Intermediate2_Demand3  =  70.0
route_Type1_Intermediate2_Demand4  =  100.0
route_Type1_Intermediate2_Demand5  =  5.0
route_Type1_Supply1_Intermediate1  =  0.0
route_Type1_Supply1_Intermediate2  =  0.0
route_Type1_Supply2_Intermediate1  =  130.0
route_Type1_Supply2_Intermediate2  =  0.0
route_Type1_Supply3_Intermediate1  =  0.0
route_Type1_Supply3_Intermediate2  =  80.0
route_Type1_Supply4_Intermediate1  =  0.0
route_Type1_Supply4_Intermediate2  =  140.0
route_Type1_Supply5_Intermediate1  =  0.0
route_Type1_Supply5_Intermediate2  =  0.0
route_Type2_Intermediate1_Demand1  =  15.0
route_Type2_Intermediate1_Demand2  =  0.0
route_Type2_Inte

Part 2

Suppose the production equipment is fully flexible, so that any factory can produce any mix of the two styles, as long as the following capacity constraints are not violated for the combined total production of both types: Factory 1 - 200; Factory 2 - 300; Factory 3 - 100; Factory 4 - 150, Factory 5 - 220 (note that these are the same capacity limits as in the Practice Problem in Week 1). Suppose RCH wants to minimize costs.

In the optimal solution, how many factories are now dedicated to producing only one style group?

In [28]:
types = ['Type1', 'Type2']
supply = ['Supply1', 'Supply2','Supply3','Supply4','Supply5']
                      #sup1 #sup2 #sup3 #sup4 #sup5
supplycap = np.array([200,   300,  100,   150,  220]) #constraint is changed to total production capacity of any product type


demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5']
                      #dem1 #dem2 #dem3 #dem4 #dem5
demandcap = np.array([[130,  45,   70,   100,  5], #type1
                      [15,   45,   40,   100,175]])#type2

intermediate = ['Intermediate1','Intermediate2']

                            #sup1       #sup2       #sup3       #sup4      #sup5
                            #int1#int2 #int1#int2  #int1#int2  #int1#int2 #int1#int2
cost_sup_int   = np.array([[[30,   50],[23,    66],[35,    14],[70,   12],[65,   70]], #type1
                           [[33,   55],[25,    73],[39,    15],[77,   13],[12,   14]]]) #type2
                  
                            #int1                     #int2
                         #Dem1#Dem2#Dem3#Dem4#Dem5   #Dem1#Dem2#Dem3#Dem4#Dem5
cost_int_dem = np.array([[[12, 25,  22,  40,    41], [65,  22,  23,  12,   15]],#type1
                         [[13, 28,  24,  44,    45], [72,  24,  25,  13,   17]]]) #type2

supplyconstraint = p.makeDict([supply], supplycap)
demandconstraint = p.makeDict([types, demand], demandcap)

sup_int_dict = p.makeDict([types, supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per unit
int_dem_dict = p.makeDict([types, intermediate, demand], cost_int_dem*1) 

prob = p.LpProblem('Clothes_Production', p.LpMinimize)

sup_to_int_route = [(t,s,i) for t in types for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (types, supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(t,i,d) for t in types for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (types, intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[t][s][i]*sup_int_dict[t][s][i] for (t,s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[t][i][d]*int_dem_dict[t][i][d] for (t,i,d) in int_to_dem_route])

for t in types:
    for d in demand:
        prob += p.lpSum([int_to_dem_vars[t][i][d] for i in intermediate]) >= demandconstraint[t][d] #demand constraint

    for i in intermediate:
        prob += p.lpSum([sup_to_int_vars[t][s][i] for s in supply]) == p.lpSum([int_to_dem_vars[t][i][d] for d in demand]) #conservation of flow constraint

for s in supply: #note the for loops inside the lp.sum, compare with when the constraint is per type per factory
        prob += p.lpSum([sup_to_int_vars[t][s][i] for t in types for i in intermediate]) <= supplyconstraint[s] #plant cap constraint is changed to be capacity for both types combined         

prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  24230.0
Solution Status: Optimal
route_Type1_Intermediate1_Demand1  =  130.0
route_Type1_Intermediate1_Demand2  =  45.0
route_Type1_Intermediate1_Demand3  =  70.0
route_Type1_Intermediate1_Demand4  =  0.0
route_Type1_Intermediate1_Demand5  =  0.0
route_Type1_Intermediate2_Demand1  =  0.0
route_Type1_Intermediate2_Demand2  =  0.0
route_Type1_Intermediate2_Demand3  =  0.0
route_Type1_Intermediate2_Demand4  =  100.0
route_Type1_Intermediate2_Demand5  =  5.0
route_Type1_Supply1_Intermediate1  =  0.0
route_Type1_Supply1_Intermediate2  =  0.0
route_Type1_Supply2_Intermediate1  =  245.0
route_Type1_Supply2_Intermediate2  =  0.0
route_Type1_Supply3_Intermediate1  =  0.0
route_Type1_Supply3_Intermediate2  =  100.0
route_Type1_Supply4_Intermediate1  =  0.0
route_Type1_Supply4_Intermediate2  =  5.0
route_Type1_Supply5_Intermediate1  =  0.0
route_Type1_Supply5_Intermediate2  =  0.0
route_Type2_Intermediate1_Demand1  =  15.0
route_Type2_Intermediate1_Demand2  =  0.0
route_Type2_Inter

Part 3

RCH does not feel comfortable with your solution from Part 2 - too few factories are producing both styles, they claim.

If at least 10 units of each style needs to be produced in every factory, how much will the optimal cost increase compared to the solution in Part 2? Answer in dollars without the dollar sign, e.g., if the solution is $45.45, write 45.

In [30]:
types = ['Type1', 'Type2']
supply = ['Supply1', 'Supply2','Supply3','Supply4','Supply5']
                      #sup1 #sup2 #sup3 #sup4 #sup5
supplycap = np.array([200,   300,  100,   150,  220]) #constraint is changed to total production capacity of any product type


demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5']
                      #dem1 #dem2 #dem3 #dem4 #dem5
demandcap = np.array([[130,  45,   70,   100,  5], #type1
                      [15,   45,   40,   100,175]])#type2

intermediate = ['Intermediate1','Intermediate2']

                            #sup1       #sup2       #sup3       #sup4      #sup5
                            #int1#int2 #int1#int2  #int1#int2  #int1#int2 #int1#int2
cost_sup_int   = np.array([[[30,   50],[23,    66],[35,    14],[70,   12],[65,   70]], #type1
                           [[33,   55],[25,    73],[39,    15],[77,   13],[12,   14]]]) #type2
                  
                            #int1                     #int2
                         #Dem1#Dem2#Dem3#Dem4#Dem5   #Dem1#Dem2#Dem3#Dem4#Dem5
cost_int_dem = np.array([[[12, 25,  22,  40,    41], [65,  22,  23,  12,   15]],#type1
                         [[13, 28,  24,  44,    45], [72,  24,  25,  13,   17]]]) #type2

supplyconstraint = p.makeDict([supply], supplycap)
demandconstraint = p.makeDict([types, demand], demandcap)

sup_int_dict = p.makeDict([types, supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per unit
int_dem_dict = p.makeDict([types, intermediate, demand], cost_int_dem*1) 

prob = p.LpProblem('Clothes_Production', p.LpMinimize)

sup_to_int_route = [(t,s,i) for t in types for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (types, supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(t,i,d) for t in types for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (types, intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[t][s][i]*sup_int_dict[t][s][i] for (t,s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[t][i][d]*int_dem_dict[t][i][d] for (t,i,d) in int_to_dem_route])

for t in types:
    for s in supply:
        prob += p.lpSum([sup_to_int_vars[t][s][i] for i in intermediate]) >= 10 #enforce the constraint of minimum number of product type must be produced each
    
    for d in demand:
        prob += p.lpSum([int_to_dem_vars[t][i][d] for i in intermediate]) >= demandconstraint[t][d] #demand constraint

    for i in intermediate:
        prob += p.lpSum([sup_to_int_vars[t][s][i] for s in supply]) == p.lpSum([int_to_dem_vars[t][i][d] for d in demand]) #conservation of flow constraint

for s in supply:
        prob += p.lpSum([sup_to_int_vars[t][s][i] for t in types for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
        
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  24930.0
Solution Status: Optimal
route_Type1_Intermediate1_Demand1  =  130.0
route_Type1_Intermediate1_Demand2  =  45.0
route_Type1_Intermediate1_Demand3  =  70.0
route_Type1_Intermediate1_Demand4  =  0.0
route_Type1_Intermediate1_Demand5  =  0.0
route_Type1_Intermediate2_Demand1  =  0.0
route_Type1_Intermediate2_Demand2  =  0.0
route_Type1_Intermediate2_Demand3  =  0.0
route_Type1_Intermediate2_Demand4  =  100.0
route_Type1_Intermediate2_Demand5  =  5.0
route_Type1_Supply1_Intermediate1  =  10.0
route_Type1_Supply1_Intermediate2  =  0.0
route_Type1_Supply2_Intermediate1  =  225.0
route_Type1_Supply2_Intermediate2  =  0.0
route_Type1_Supply3_Intermediate1  =  0.0
route_Type1_Supply3_Intermediate2  =  90.0
route_Type1_Supply4_Intermediate1  =  0.0
route_Type1_Supply4_Intermediate2  =  15.0
route_Type1_Supply5_Intermediate1  =  10.0
route_Type1_Supply5_Intermediate2  =  0.0
route_Type2_Intermediate1_Demand1  =  15.0
route_Type2_Intermediate1_Demand2  =  0.0
route_Type2_Int

<h3>Practice Problem 8:  NERD Revisited</h3>

You just cannot get away from the Root Beer distributors! The New England Root Beer Distributors (NERD) have been using the model developed earlier to determine the optimal number of DCs to open for their New England distribution network. For the remainder of this Practice Problem - use the attached Excel or LibreOffice spreadsheet.

The settings for this run are similar to Scenario 6 in Week 2 Lesson 1 (slide 33).  Specifically:

<img src="w2pp61.png">
<img src="w2pp62.png">
<img src="w2pp63.png">

Capacity at both the Bellows Fall and Scranton plants is unlimited (=9999)  (cells G4 & G5)

Minimum number of DC's is set to 1 and Maximum number of DCs is set to 5 (cells C33 and D33)

The Level of Service of Maximum Average Distance is set to 100 miles (cell D34)

The Level of Service of Minimum Percentage of Demand within 50 miles is set to 50% (cell C35)

The DC capacities are 500 barrels/week for SP and NA, 1000 barrels/week for BO and PR, and 2000 barrels/week for WO

<b>Run the model. You should get an optimal total cost of $68,265 per week. The solution should be:</b>

Producing 500 barrels at the BF plant and 1500 barrels at the SC Plant

Opening three DCs:  NA (500 barrels/week), SP (500 barrels/week), and WO (1000 barrels per week)

Maximum average distance to a DC is 33 miles

Percent of customers within 50 miles of a DC is 86.5%. 

If you do not get this solution - make sure you did not change any of the input values or reload the spreadsheet from above. We will use this model for the rest of the questions. 

In [4]:
supply = ['Supply1', 'Supply2'] #sup1 is BF plant, sup2 is scranton plant
supplycap = np.array([9999, 9999])
supply_variable_cost = np.array([2,0.75])

demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5', 'Demand6','Demand7', 'Demand8','Demand9', 'Demand10','Demand11', 'Demand12']
demandcap = np.array([450,60,80,130,110,140,140,70,120,310,200,190])

intermediate = ['Intermediate1','Intermediate2','Intermediate3','Intermediate4','Intermediate5']
intermediate_fixed_cost = np.array([11000,5000,9000,8000,7000])
intermediate_variable_cost = np.array([1.50,0.95,1.05,1.10,1.12])
intermediatecap = np.array([1000,500,1000,500,2000])


                          #int1 #int2 #int3 #int4 #int5
cost_sup_int   = np.array([[3.40,3.00,4.40,3.04,3.36], #sup1
                           [4.80,5.25,5.12,4.00,4.20]]) #sup2
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7 #Dem8 #Dem9 #Dem10#Dem11 #Dem12
cost_int_dem = np.array([[8,93,69,98,55,37,128,95,62,42,82,34], #int1
                         [37,65,33,103,20,12,137,113,48,72,79,41], #int2
                         [42,106,105,73,92,72,94,57,104,17,68,38], #int3
                         [82,59,101,27,93,79,63,57,127,68,12,47], #int4
                         [34,68,72,66,60,41,98,71,85,38,47,18]]) #int5

dist_within_limit = cost_int_dem <= 50 #remember to use less than or equal to sign

dist_within_limit = dist_within_limit.astype(int) / sum(demandcap)

dist_within_limit_dict = p.makeDict([intermediate, demand], dist_within_limit)

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))
intermediateconstraint = dict(zip(intermediate,intermediatecap))

supplyvariablecost = dict(zip(supply,supply_variable_cost))

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost))

int_dem_dist = p.makeDict([intermediate, demand], cost_int_dem/sum(demandcap)) #maximum avg. distance constraint
sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int) 
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*0.55) #cost of 0.55 dollar per mile per unit 


prob = p.LpProblem('Root_Beer_Production', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([supplyvariablecost[s]*sup_to_int_vars[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) >= demandconstraint[d] #RW demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 10000000000*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) <= intermediateconstraint[i]
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 5 #number of DC maximum must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1 #number of DC minimum must be open

prob += p.lpSum([int_to_dem_vars[i][d]*dist_within_limit_dict[i][d] for (i,d) in int_to_dem_route]) >= 0.5 #pct of demand within certain limit constraint
 
prob += p.lpSum([int_to_dem_vars[i][d]*int_dem_dist[i][d] for (i,d) in int_to_dem_route]) <= 100  
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  68264.5
Solution Status: Optimal
open_Intermediate1  =  0.0
open_Intermediate2  =  1.0
open_Intermediate3  =  0.0
open_Intermediate4  =  1.0
open_Intermediate5  =  1.0
route_Intermediate1_Demand1  =  0.0
route_Intermediate1_Demand10  =  0.0
route_Intermediate1_Demand11  =  0.0
route_Intermediate1_Demand12  =  0.0
route_Intermediate1_Demand2  =  0.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate1_Demand4  =  0.0
route_Intermediate1_Demand5  =  0.0
route_Intermediate1_Demand6  =  0.0
route_Intermediate1_Demand7  =  0.0
route_Intermediate1_Demand8  =  0.0
route_Intermediate1_Demand9  =  0.0
route_Intermediate2_Demand1  =  0.0
route_Intermediate2_Demand10  =  0.0
route_Intermediate2_Demand11  =  0.0
route_Intermediate2_Demand12  =  0.0
route_Intermediate2_Demand2  =  50.0
route_Intermediate2_Demand3  =  80.0
route_Intermediate2_Demand4  =  0.0
route_Intermediate2_Demand5  =  110.0
route_Intermediate2_Demand6  =  140.0
route_Intermediate2_Demand7  =  0.0
route_Interme

Part 1

You want to understand the inventory impact of the different designs for the NERD network. You have collected information on the average inventory (in dollars) and throughput (in barrels) at a number of NERD DCs. Plotting this you notice it is a non-linear relationship of the form: Ii=αTβi where the average inventory cost (in dollars) at location i in dollars, Ii, is a function of the throughput (in barrels) at that facility, Ti. You used regression to find the values of alpha and beta to be 41.00 and 0.62, respectively.

Using this relationship, what is the expected weekly inventory cost (safety stock and cycle stock) for the optimal solution described above? Only include the inventory at the DCs, not the regional DCs located at each market city. Ignore pipeline inventory. Enter your answer in dollars with no commas or currency symbols.

In [26]:
def inv_cost(alpha, beta, throughput):
    return alpha*(throughput)**beta

alpha = 41
beta = 0.62

totalinvcost = sum([inv_cost(alpha, beta, sup_to_int_vars[s][i].varValue) for (s,i) in sup_to_int_route])

print("Total Inventory Cost at DCs: ", totalinvcost)

Total Inventory Cost at DCs:  6835.416665682128


Part 2

Using the same relationship as in Part 1, what is the expected weekly inventory costs in dollars (safety stock and cycle stock) when forcing a single DC solution? Only include the inventory at the DCs, not the regional DCs located at each market city. Ignore pipeline inventory. Enter your answer in dollars with no commas or currency symbols.

In [28]:
supply = ['Supply1', 'Supply2'] #sup1 is BF plant, sup2 is scranton plant
supplycap = np.array([9999, 9999])
supply_variable_cost = np.array([2,0.75])

demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5', 'Demand6','Demand7', 'Demand8','Demand9', 'Demand10','Demand11', 'Demand12']
demandcap = np.array([450,60,80,130,110,140,140,70,120,310,200,190])

intermediate = ['Intermediate1','Intermediate2','Intermediate3','Intermediate4','Intermediate5']
intermediate_fixed_cost = np.array([11000,5000,9000,8000,7000])
intermediate_variable_cost = np.array([1.50,0.95,1.05,1.10,1.12])
intermediatecap = np.array([1000,500,1000,500,2000])


                          #int1 #int2 #int3 #int4 #int5
cost_sup_int   = np.array([[3.40,3.00,4.40,3.04,3.36], #sup1
                           [4.80,5.25,5.12,4.00,4.20]]) #sup2
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7 #Dem8 #Dem9 #Dem10#Dem11 #Dem12
cost_int_dem = np.array([[8,93,69,98,55,37,128,95,62,42,82,34], #int1
                         [37,65,33,103,20,12,137,113,48,72,79,41], #int2
                         [42,106,105,73,92,72,94,57,104,17,68,38], #int3
                         [82,59,101,27,93,79,63,57,127,68,12,47], #int4
                         [34,68,72,66,60,41,98,71,85,38,47,18]]) #int5

dist_within_limit = cost_int_dem <= 50 #remember to use less than or equal to sign

dist_within_limit = dist_within_limit.astype(int) / sum(demandcap)

dist_within_limit_dict = p.makeDict([intermediate, demand], dist_within_limit)

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))
intermediateconstraint = dict(zip(intermediate,intermediatecap))

supplyvariablecost = dict(zip(supply,supply_variable_cost))

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost))

int_dem_dist = p.makeDict([intermediate, demand], cost_int_dem/sum(demandcap))
sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int) 
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*0.55) #cost of 0.55 dollar per mile per unit 


prob = p.LpProblem('Root_Beer_Production', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([supplyvariablecost[s]*sup_to_int_vars[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) >= demandconstraint[d] #RW demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 10000000000*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) <= intermediateconstraint[i]
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 1 #number of DC maximum must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1 #number of DC minimum must be open

prob += p.lpSum([int_to_dem_vars[i][d]*dist_within_limit_dict[i][d] for (i,d) in int_to_dem_route]) >= 0.5 #pct of demand within certain limit constraint
 
prob += p.lpSum([int_to_dem_vars[i][d]*int_dem_dist[i][d] for (i,d) in int_to_dem_route]) <= 100  
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

def inv_cost(alpha, beta, throughput):
    return alpha*(throughput)**beta

alpha = 41
beta = 0.62

totalinvcost = sum([inv_cost(alpha, beta, sup_to_int_vars[s][i].varValue) for (s,i) in sup_to_int_route])

print("Total Inventory Cost at DCs: ", totalinvcost)

minimum cost:  73892.50000000001
Solution Status: Optimal
open_Intermediate1  =  0.0
open_Intermediate2  =  0.0
open_Intermediate3  =  0.0
open_Intermediate4  =  0.0
open_Intermediate5  =  1.0
route_Intermediate1_Demand1  =  0.0
route_Intermediate1_Demand10  =  0.0
route_Intermediate1_Demand11  =  0.0
route_Intermediate1_Demand12  =  0.0
route_Intermediate1_Demand2  =  0.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate1_Demand4  =  0.0
route_Intermediate1_Demand5  =  0.0
route_Intermediate1_Demand6  =  0.0
route_Intermediate1_Demand7  =  0.0
route_Intermediate1_Demand8  =  0.0
route_Intermediate1_Demand9  =  0.0
route_Intermediate2_Demand1  =  0.0
route_Intermediate2_Demand10  =  0.0
route_Intermediate2_Demand11  =  0.0
route_Intermediate2_Demand12  =  0.0
route_Intermediate2_Demand2  =  0.0
route_Intermediate2_Demand3  =  0.0
route_Intermediate2_Demand4  =  0.0
route_Intermediate2_Demand5  =  0.0
route_Intermediate2_Demand6  =  0.0
route_Intermediate2_Demand7  =  0.0
route_Int

Part 3

You want to test some things with the model. Specifically, you want to drive the level of service up. Set the Minimum percent of customers within 50 miles parameter (cell C35) to 99% and re-solve the model (max number of DCs = 5). Your total cost should be $72,175 with four DCs open (BO, NA, SP, and WO).

Look very closely at your solution.

Which of the following statements are true with this run?

This solution is invalid and should not be considered --> <i><b>correct, status of solution is infeasible</i></b>

The network cannot actually meet this level of service requirement (Min Percent within 50 Miles ≥ 99%) --> <i><b>correct, status of solution is infeasible</i></b>

The Bellows Falls plant will not be used

The solution is fine and has no issues

None of these statements are true

<b>Explanation</b>

The first two statements are true. By increasing the required level of service, we are forcing the model to solve the impossible. Notice the number of barrels that are being delivered to Worcester (390 per week when only 190 are demanded) and Nashua (190 per week when only 140 are demanded). Why is this happening? Well, recall that we set the demand constraints to be greater than or equal to the minimum demand. In most cases, we will not exceed the minimum since we are charged for this in the objective function. In this case, though, the only way the model can satisfy the LOS constraint is to "over-serve" Worcester and Nashua in order to increase the percentage of customers within 50 miles of a DC! Suppose you changed the constraint to be equal to instead of greater than or equal to. Go ahead, try it. I'll wait.

See - the model will not solve since it cannot find a feasible solution! This is another illustration of the model doing ANYTHING to meet its constraints and minimize costs.



In [43]:
supply = ['Supply1', 'Supply2'] #sup1 is BF plant, sup2 is scranton plant
supplycap = np.array([9999, 9999])
supply_variable_cost = np.array([2,0.75])

demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5', 'Demand6','Demand7', 'Demand8','Demand9', 'Demand10','Demand11', 'Demand12']
demandcap = np.array([450,60,80,130,110,140,140,70,120,310,200,190])

intermediate = ['Intermediate1','Intermediate2','Intermediate3','Intermediate4','Intermediate5']
intermediate_fixed_cost = np.array([11000,5000,9000,8000,7000])
intermediate_variable_cost = np.array([1.50,0.95,1.05,1.10,1.12])
intermediatecap = np.array([1000,500,1000,500,2000])


                          #int1 #int2 #int3 #int4 #int5
cost_sup_int   = np.array([[3.40,3.00,4.40,3.04,3.36], #sup1
                           [4.80,5.25,5.12,4.00,4.20]]) #sup2
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7 #Dem8 #Dem9 #Dem10#Dem11 #Dem12
cost_int_dem = np.array([[8,93,69,98,55,37,128,95,62,42,82,34], #int1
                         [37,65,33,103,20,12,137,113,48,72,79,41], #int2
                         [42,106,105,73,92,72,94,57,104,17,68,38], #int3
                         [82,59,101,27,93,79,63,57,127,68,12,47], #int4
                         [34,68,72,66,60,41,98,71,85,38,47,18]]) #int5

dist_within_limit = cost_int_dem <= 50 #remember to use less than or equal to sign

dist_within_limit = dist_within_limit.astype(int) / sum(demandcap)

dist_within_limit_dict = p.makeDict([intermediate, demand], dist_within_limit)

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))
intermediateconstraint = dict(zip(intermediate,intermediatecap))

supplyvariablecost = dict(zip(supply,supply_variable_cost))

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost))

int_dem_dist = p.makeDict([intermediate, demand], cost_int_dem/sum(demandcap))
sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int) 
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*0.55) #cost of 0.55 dollar per mile per unit 


prob = p.LpProblem('Root_Beer_Production', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([supplyvariablecost[s]*sup_to_int_vars[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d] #RW demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 10000000000*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) <= intermediateconstraint[i]
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 5 #number of DC maximum must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1 #number of DC minimum must be open

prob += p.lpSum([int_to_dem_vars[i][d]*dist_within_limit_dict[i][d] for (i,d) in int_to_dem_route]) >= 0.99 #pct of demand within certain limit constraint
 
prob += p.lpSum([int_to_dem_vars[i][d]*int_dem_dist[i][d] for (i,d) in int_to_dem_route]) <= 100  
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  54605.000761
Solution Status: Infeasible
open_Intermediate1  =  6.5e-08
open_Intermediate2  =  0.99999983
open_Intermediate3  =  3.8e-08
open_Intermediate4  =  5e-08
open_Intermediate5  =  2.2e-08
route_Intermediate1_Demand1  =  450.0
route_Intermediate1_Demand10  =  0.0
route_Intermediate1_Demand11  =  0.0
route_Intermediate1_Demand12  =  0.0
route_Intermediate1_Demand2  =  0.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate1_Demand4  =  0.0
route_Intermediate1_Demand5  =  0.0
route_Intermediate1_Demand6  =  200.0
route_Intermediate1_Demand7  =  0.0
route_Intermediate1_Demand8  =  0.0
route_Intermediate1_Demand9  =  0.0
route_Intermediate2_Demand1  =  0.0
route_Intermediate2_Demand10  =  0.0
route_Intermediate2_Demand11  =  0.0
route_Intermediate2_Demand12  =  0.0
route_Intermediate2_Demand2  =  0.0
route_Intermediate2_Demand3  =  80.0
route_Intermediate2_Demand4  =  0.0
route_Intermediate2_Demand5  =  110.0
route_Intermediate2_Demand6  =  -60.0
route_Intermediat

Part 4

NERD management has suggested several different initiatives to lower the total costs to serve its customers in New England. Reset the Minimum Percent within 50 miles parameter (cell C35) back to 50% for this part.

Which of the following potential changes would you recommend pursuing further, based on the total expected costs? Assume that these are independent changes to be explored separately.

Select the best answer

Increase the capacity of the Worcester (WO) DC to 2500 barrels per week

Increase the capacity of the Nashua (NA) DC to 1000 barrels per week

Negotiating the cost per barrel per mile charged by the local courier to 0.50dollars (down from 0.55) --> <b><i>tried, will lead to lowest cost compated to others </i></b>

Negotiating the cost per barrel from the Bellows Falls plant to 1.75dollars (down from 2.00)

Negotiating the cost per barrel from the Scranton plant to 0.50dollars (down from 0.75)

<b>Explanation</b>

Increasing the capacity of the WO DC does nothing. You are not using current capacity, adding more does not shift volume from the other DCs.

Increasing the capacity of the NA DC does a little - diverting just 10 barrels per week through the NA DC. Savings is about $18 per week. Not really a game changer, eh?

Reducing the Bellows Falls costs from $2 to $1.75 a barrel will decrease costs by about $125 per week - not a real game changer either.

Reducing the Scranton Plant costs from $0.75 to $0.50 a barrel will decrease costs by about $375 per week - not really worth it.

But, negotiating the courier transport costs for the outbound delivery from $0.55 per barrel per mile to $0.50 will lower costs by $3,300 per week. Not a trivial amount.

You should have had an idea that this is the area to focus on by looking at the cost buckets in cells B4:B8. Notice that the largest costs are outbound transportation - this is the local delivery by the courier.

This is how a supply chain network design model can be used to test out different potential initiatives.

<b>Part 5</b>

The NERD CEO lives in Boston. She really, really wants to locate the DC there. But, she understands that it has to make economic sense and wants to understand how much more expensive Boston is than the other locations for a DC. Currently, Boston is not selected to open in the optimal solution (when allowing any number of DCs). Specifically, she is targeting the weekly fixed costs (cell C12). These are the labor, electricity, and other weekly costs that are pretty much independent of the flow through the facility.

What does the Fixed Cost (cell C12) for the Boston DC need to be so that it becomes economically justifiable to open? That is, at what cost will the Boston DC enter the optimal solution? Enter your answer in dollars with no commas or currency symbols.

<b>Explanation</b>

The fixed cost needs to be lowered to about 9,660dollars in order for the Boston DC to enter the solution. How did we find that? Trial and error. It was originally 11k (Boston out of solution) so I tried 7k (Boston in!). Then I kept cutting the difference in half, e.g. 9k (in), 10k (out), 9.5k (in), etc. until I found the breakpoint of 9.66k. I allowed a pretty wide range here - I did not want you to calculate it to the penny.

Why is this valuable? Well, we just used the model to estimate the value of a move that the model would not have considered. Lowering the fixed cost from 11,000dollars to 9,660dollars per week might be an easy thing to do or something really hard. We don't know - but the executives at NERD probably do! The model has given them a target cost to consider. Management might be willing to pay a little more for this for some reasons that the model does not consider as well. That is why the humans need to make the final decision - and not let the model do it!

In [66]:
supply = ['Supply1', 'Supply2'] #sup1 is BF plant, sup2 is scranton plant
supplycap = np.array([9999, 9999])
supply_variable_cost = np.array([2,0.75])

demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5', 'Demand6','Demand7', 'Demand8','Demand9', 'Demand10','Demand11', 'Demand12']
demandcap = np.array([450,60,80,130,110,140,140,70,120,310,200,190])

intermediate = ['Intermediate1','Intermediate2','Intermediate3','Intermediate4','Intermediate5']
intermediate_fixed_cost = np.array([9662,5000,9000,8000,7000])
intermediate_variable_cost = np.array([1.50,0.95,1.05,1.10,1.12])
intermediatecap = np.array([1000,500,1000,500,2000])


                          #int1 #int2 #int3 #int4 #int5
cost_sup_int   = np.array([[3.40,3.00,4.40,3.04,3.36], #sup1
                           [4.80,5.25,5.12,4.00,4.20]]) #sup2
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7 #Dem8 #Dem9 #Dem10#Dem11 #Dem12
cost_int_dem = np.array([[8,93,69,98,55,37,128,95,62,42,82,34], #int1
                         [37,65,33,103,20,12,137,113,48,72,79,41], #int2
                         [42,106,105,73,92,72,94,57,104,17,68,38], #int3
                         [82,59,101,27,93,79,63,57,127,68,12,47], #int4
                         [34,68,72,66,60,41,98,71,85,38,47,18]]) #int5

dist_within_limit = cost_int_dem <= 50 #remember to use less than or equal to sign

dist_within_limit = dist_within_limit.astype(int) / sum(demandcap)

dist_within_limit_dict = p.makeDict([intermediate, demand], dist_within_limit)

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))
intermediateconstraint = dict(zip(intermediate,intermediatecap))

supplyvariablecost = dict(zip(supply,supply_variable_cost))

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost))

int_dem_dist = p.makeDict([intermediate, demand], cost_int_dem/sum(demandcap))
sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int) 
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*0.55) #cost of 0.55 dollar per mile per unit 


prob = p.LpProblem('Root_Beer_Production', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([supplyvariablecost[s]*sup_to_int_vars[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d] #RW demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 10000000000*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) <= intermediateconstraint[i]
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 5 #number of DC maximum must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1 #number of DC minimum must be open

prob += p.lpSum([int_to_dem_vars[i][d]*dist_within_limit_dict[i][d] for (i,d) in int_to_dem_route]) >= 0.50 #pct of demand within certain limit constraint
 
prob += p.lpSum([int_to_dem_vars[i][d]*int_dem_dist[i][d] for (i,d) in int_to_dem_route]) <= 100  
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  68264.0
Solution Status: Optimal
open_Intermediate1  =  1.0
open_Intermediate2  =  1.0
open_Intermediate3  =  0.0
open_Intermediate4  =  1.0
open_Intermediate5  =  0.0
route_Intermediate1_Demand1  =  450.0
route_Intermediate1_Demand10  =  310.0
route_Intermediate1_Demand11  =  0.0
route_Intermediate1_Demand12  =  190.0
route_Intermediate1_Demand2  =  0.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate1_Demand4  =  0.0
route_Intermediate1_Demand5  =  0.0
route_Intermediate1_Demand6  =  0.0
route_Intermediate1_Demand7  =  0.0
route_Intermediate1_Demand8  =  40.0
route_Intermediate1_Demand9  =  10.0
route_Intermediate2_Demand1  =  0.0
route_Intermediate2_Demand10  =  0.0
route_Intermediate2_Demand11  =  0.0
route_Intermediate2_Demand12  =  0.0
route_Intermediate2_Demand2  =  60.0
route_Intermediate2_Demand3  =  80.0
route_Intermediate2_Demand4  =  0.0
route_Intermediate2_Demand5  =  110.0
route_Intermediate2_Demand6  =  140.0
route_Intermediate2_Demand7  =  0.0
route

<h3>Graded Assignment 1: Extra Scenarios</h3>
    
You may recall that, at the end of the 'Advanced Scenarios' video, Dr. Caplice asked you to create new scenarios and play with the spreadsheet. I have no doubts that you have done this on your own ;-) but just in case, this graded assignment will focus precisely on doing that. We will create a series of six new scenarios. Each one of the new scenarios will build upon the previous one, so make sure to get each one right before you move on to the next. We have added a few sanity checks along the way -- these are not graded, they are only there to help you stay on track!

Now that you have nailed Scenario 10, let's create the first new scenario in our series: we will call it Scenario 11. The parameters in this scenario are identical to those in Scenario 10, with one important exception: we will assign a capacity of 2000 units to the BF Plant. Considering this new capacity, proceed to solve this scenario optimally.

Let's now create a new scenario, called Scenario 12. Its parameters are identical to those in Scenario 11, with one very important change: for the first time, we will assign - in addition to their variable cost - a fixed cost to operating each one of the plants. Consider a fixed cost for using the plants as follows:

15100dollars for plant BFP

15100dollars for plant SCP

The fixed cost for each plant is paid only if that plant is being used by us. If we are not using units from that plant, we do not pay the fixed cost. Now, solve the problem optimally.

What is the optimal cost for Scenario 12? Please give your answer in dollars, but without a dollar sign or comma.

In [7]:
supply = ['Supply1', 'Supply2'] #sup1 is BF plant, sup2 is scranton plant
supplycap = np.array([2000, 9999])
supply_variable_cost = np.array([2,0.75])
supply_fixed_cost = np.array([15100,15100])

demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5', 'Demand6','Demand7', 'Demand8','Demand9', 'Demand10','Demand11', 'Demand12']
demandcap = np.array([450,60,80,130,110,140,140,70,120,310,200,190])

intermediate = ['Intermediate1','Intermediate2','Intermediate3','Intermediate4','Intermediate5']
intermediate_fixed_cost = np.array([11000,5000,9000,8000,7000])
intermediate_variable_cost = np.array([1.50,0.95,1.05,1.10,1.12])
intermediatecap = np.array([1000,500,1000,500,2000])


                          #int1 #int2 #int3 #int4 #int5
cost_sup_int   = np.array([[3.40,3.00,4.40,3.04,3.36], #sup1
                           [4.80,5.25,5.12,4.00,4.20]]) #sup2
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7 #Dem8 #Dem9 #Dem10 #Dem11 #Dem12
cost_int_dem = np.array([[8,93,69,98,55,37,128,95,62,42,82,34], #int1
                         [37,65,33,103,20,12,137,113,48,72,79,41], #int2
                         [42,106,105,73,92,72,94,57,104,17,68,38], #int3
                         [82,59,101,27,93,79,63,57,127,68,12,47], #int4
                         [34,68,72,66,60,41,98,71,85,38,47,18]]) #int5

dist_within_limit = cost_int_dem <= 50 #remember to use less than or equal to sign

dist_within_limit = dist_within_limit.astype(int) / sum(demandcap)

dist_within_limit_dict = p.makeDict([intermediate, demand], dist_within_limit)

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))
intermediateconstraint = dict(zip(intermediate,intermediatecap))

supplyvariablecost = dict(zip(supply,supply_variable_cost))
supplyfixedcost = dict(zip(supply,supply_fixed_cost))

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost))

int_dem_dist = p.makeDict([intermediate, demand], cost_int_dem/sum(demandcap))
sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int) 
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*0.55) #cost of 0.55 dollar per mile per unit 


prob = p.LpProblem('Root_Beer_Production', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

sup_open = p.LpVariable.dicts("open", supply, cat = 'Binary')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([supplyvariablecost[s]*sup_to_int_vars[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])\
            + p.lpSum([supplyfixedcost[s]*sup_open[s] for s in supply])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) - 10000000000*sup_open[s] <= 0 #linking constraint between flow and supply open
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) >= demandconstraint[d] #demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 10000000000*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) <= intermediateconstraint[i]
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 5 #number of DC maximum must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1 #number of DC minimum must be open

prob += p.lpSum([int_to_dem_vars[i][d]*dist_within_limit_dict[i][d] for (i,d) in int_to_dem_route]) >= 0.8 #pct of demand within certain limit constraint
 
prob += p.lpSum([int_to_dem_vars[i][d]*int_dem_dist[i][d] for (i,d) in int_to_dem_route]) <= 25  
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  94048.70000000001
Solution Status: Optimal
open_Intermediate1  =  1.0
open_Intermediate2  =  1.0
open_Intermediate3  =  1.0
open_Intermediate4  =  1.0
open_Intermediate5  =  1.0
open_Supply1  =  1.0
open_Supply2  =  0.0
route_Intermediate1_Demand1  =  450.0
route_Intermediate1_Demand10  =  0.0
route_Intermediate1_Demand11  =  0.0
route_Intermediate1_Demand12  =  0.0
route_Intermediate1_Demand2  =  0.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate1_Demand4  =  0.0
route_Intermediate1_Demand5  =  0.0
route_Intermediate1_Demand6  =  0.0
route_Intermediate1_Demand7  =  0.0
route_Intermediate1_Demand8  =  0.0
route_Intermediate1_Demand9  =  0.0
route_Intermediate2_Demand1  =  0.0
route_Intermediate2_Demand10  =  0.0
route_Intermediate2_Demand11  =  0.0
route_Intermediate2_Demand12  =  0.0
route_Intermediate2_Demand2  =  30.0
route_Intermediate2_Demand3  =  80.0
route_Intermediate2_Demand4  =  0.0
route_Intermediate2_Demand5  =  110.0
route_Intermediate2_Demand6  =  1

Part 4

Let's create now Scenario 13, using parameters identical to Scenario 12, except that now the fixed cost of the SC Plant has been reduced greatly, and is now 11100dollars. (Notice that the fixed cost for BFP remains unchanged at $15100.) Solve the problem optimally.

What is the optimal cost for Scenario 13? Please give your answer in dollars, but without a dollar sign or comma.

Part 5

Now let's create Scenario 14. All our parameters will be identical to those in Scenario 13, except that now we will allow a more relaxed maximum weighted average distance of customers to DCs of 35 miles instead of the previous 25 miles, and a more lenient minimum percentage of customers within 50 miles of DCs of 60% instead of the previous 80%. Solve the problem optimally.

Which DCs did we stop using, compared to the previous scenario? This question is for your benefit and has no points.

Part 7

For Scenario 15, all the parameters will be identical to those in Scenario 14, with one exception: our friends at the BO DC have decided to become more efficient, and have a new variable cost of 1.00dollars per unit, and a new fixed cost of 5,000dollars. Solve the problem optimally.

What is the optimal cost for Scenario 15? Please give your answer in dollars, but without a dollar sign or comma.

Part 8

Let's now create our last scenario, Scenario 16. Its parameters are identical to those in Scenario 15, except that there is a new consumer: Ferrisbourgh. Its demand is 120 units, and its distance to the DCs (in miles) is as follows: 70 to BO, 65 to NA, 85 to PR, 50 to SP and 70 to WO. Solve the problem optimally.

Which of the following is closer to the new cost? This question is for your benefit and has no points.

Part 9

What is the optimal cost for Scenario 16? Please give your answer in dollars, but without a dollar sign or comma.

In [11]:
supply = ['Supply1', 'Supply2'] #sup1 is BF plant, sup2 is scranton plant
supplycap = np.array([2000, 9999])
supply_variable_cost = np.array([2,0.75])
supply_fixed_cost = np.array([15100,11100])

demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5', 'Demand6','Demand7', 'Demand8','Demand9', 'Demand10','Demand11', 'Demand12', 'Demand13']
demandcap = np.array([450,60,80,130,110,140,140,70,120,310,200,190,120])

intermediate = ['Intermediate1','Intermediate2','Intermediate3','Intermediate4','Intermediate5']
intermediate_fixed_cost = np.array([5000,5000,9000,8000,7000])
intermediate_variable_cost = np.array([1.00,0.95,1.05,1.10,1.12])
intermediatecap = np.array([1000,500,1000,500,2000])


                          #int1 #int2 #int3 #int4 #int5
cost_sup_int   = np.array([[3.40,3.00,4.40,3.04,3.36], #sup1
                           [4.80,5.25,5.12,4.00,4.20]]) #sup2
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7 #Dem8 #Dem9 #Dem10 #Dem11 #Dem12#Dem13
cost_int_dem = np.array([[8,93,69,98,55,37,128,95,62,42,82,34,70], #int1
                         [37,65,33,103,20,12,137,113,48,72,79,41,65], #int2
                         [42,106,105,73,92,72,94,57,104,17,68,38,85], #int3
                         [82,59,101,27,93,79,63,57,127,68,12,47,50], #int4
                         [34,68,72,66,60,41,98,71,85,38,47,18,70]]) #int5

dist_within_limit = cost_int_dem <= 50 #remember to use less than or equal to sign

dist_within_limit = dist_within_limit.astype(int) / sum(demandcap)

dist_within_limit_dict = p.makeDict([intermediate, demand], dist_within_limit)

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))
intermediateconstraint = dict(zip(intermediate,intermediatecap))

supplyvariablecost = dict(zip(supply,supply_variable_cost))
supplyfixedcost = dict(zip(supply,supply_fixed_cost))

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost))

int_dem_dist = p.makeDict([intermediate, demand], cost_int_dem/sum(demandcap))
sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int) 
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*0.55) #cost of 0.55 dollar per mile per unit 


prob = p.LpProblem('Root_Beer_Production', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

sup_open = p.LpVariable.dicts("open", supply, cat = 'Binary')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([supplyvariablecost[s]*sup_to_int_vars[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])\
            + p.lpSum([supplyfixedcost[s]*sup_open[s] for s in supply])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) - 10000000000*sup_open[s] <= 0 #linking constraint between flow and supply open
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) >= demandconstraint[d] #demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 10000000000*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) <= intermediateconstraint[i]
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 5 #number of DC maximum must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1 #number of DC minimum must be open

prob += p.lpSum([int_to_dem_vars[i][d]*dist_within_limit_dict[i][d] for (i,d) in int_to_dem_route]) >= 0.6 #pct of demand within certain limit constraint
 
prob += p.lpSum([int_to_dem_vars[i][d]*int_dem_dist[i][d] for (i,d) in int_to_dem_route]) <= 35  
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  83839.90000000001
Solution Status: Optimal
open_Intermediate1  =  1.0
open_Intermediate2  =  1.0
open_Intermediate3  =  0.0
open_Intermediate4  =  1.0
open_Intermediate5  =  1.0
open_Supply1  =  0.0
open_Supply2  =  1.0
route_Intermediate1_Demand1  =  450.0
route_Intermediate1_Demand10  =  0.0
route_Intermediate1_Demand11  =  0.0
route_Intermediate1_Demand12  =  0.0
route_Intermediate1_Demand13  =  0.0
route_Intermediate1_Demand2  =  0.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate1_Demand4  =  0.0
route_Intermediate1_Demand5  =  0.0
route_Intermediate1_Demand6  =  0.0
route_Intermediate1_Demand7  =  0.0
route_Intermediate1_Demand8  =  0.0
route_Intermediate1_Demand9  =  0.0
route_Intermediate2_Demand1  =  0.0
route_Intermediate2_Demand10  =  0.0
route_Intermediate2_Demand11  =  0.0
route_Intermediate2_Demand12  =  0.0
route_Intermediate2_Demand13  =  50.0
route_Intermediate2_Demand2  =  0.0
route_Intermediate2_Demand3  =  80.0
route_Intermediate2_Demand4  =  0

<b>Graded Assignment 2:  Colombian Premium Coffee</b>
    
A traditional Colombian Coffee manufacturer called CPC has hired your services as a supply chain consultor to analyze the distribution of its brand-new premium product, named Coffee Indeed ®. This product is 1 package containing 1-kilogram of ground and roasted Arabica coffee to be commercialized and distributed in three cities: Bogotá (DC 1), Medellín (DC 2), and Cali (DC 3). Due to governmental tax benefits, the company roasts and packs the product in two factories: Barranquilla (Plant 1) and Santander (Plant 2). CPC hires carriers to transport the products from the plants to each of the three distribution centers (DCs). Each DC has a different demand, and each plant has a different production capacity.

Demand:

Bogotá: 345,000 units per week

Medellín: 263,000 units per week

Cali: 172,000 units per week

Plant Capacities:

Barranquilla: 610,000 units per week

Santander: 350,000 units per week

Frank Evans, the network design manager, asked you to create a network design model to optimize the transportation flow of Coffee Indeed ® from the plants to the DCs.

CPC organizes the units of product in pallets, each pallet holds 1,000 units (one thousand units). The price table of the carrier to move 1 pallet from each plant to each DC ($/pallet) is shown below:

 $/pallet	            Bogotá (DC 1)	Medellín (DC 2)	  Cali (DC 3)
 
Barranquilla (Plant 1)	130.00	           90.60	       142.00

Santander (Plant 2)	     53.60	           41.60	       88.80

What is the minimum total weekly cost for distributing the products from the plants to the three DCs?

In [26]:
types = ['Type1']
supply = ['Supply1', 'Supply2']
                        #sup1 #sup2
supplycap = np.array([[610,   350]])#type1
                      #[200,   300]])#type2


demand = ['Demand1', 'Demand2', 'Demand3']
                      #dem1 #dem2 #dem3
demandcap = np.array([[345,263,172]]) #type1
                      #[15,   45,   40]])#type2

                            #sup1            #sup2   
                            #dem1#dem2#dem3 #dem1#dem2#dem3
cost_sup_dem   = np.array([[[130,90.60,142],[53.6, 41.6, 88.8]]]) #type1
                           #[[33,   55],[25,    73]]) #type2
                  
supplyconstraint = p.makeDict([types, supply], supplycap)
demandconstraint = p.makeDict([types, demand], demandcap)

sup_dem_dict = p.makeDict([types, supply, demand], cost_sup_dem*1) #cost of 1 dollar per mile per unit

prob = p.LpProblem('Coffee_Production', p.LpMinimize)

sup_to_dem_route = [(t,s,d) for t in types for s in supply for d in demand]

sup_to_dem_vars = p.LpVariable.dicts("route", (types, supply,demand), lowBound = 0, cat = 'Continuous')

prob += p.lpSum([sup_to_dem_vars[t][s][d]*sup_dem_dict[t][s][d] for (t,s,d) in sup_to_dem_route])\


for t in types: #note the posisitoning of the for loops and how it corresponds to the constraints we are trying to model
    for s in supply:
        prob += p.lpSum([sup_to_dem_vars[t][s][d] for d in demand]) <= supplyconstraint[t][s] #plant cap constraint

    for d in demand:
        prob += p.lpSum([sup_to_dem_vars[t][s][d] for s in supply]) >= demandconstraint[t][d] #RW demand constraint


prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  66477.8
Solution Status: Optimal
route_Type1_Supply1_Demand1  =  0.0
route_Type1_Supply1_Demand2  =  263.0
route_Type1_Supply1_Demand3  =  167.0
route_Type1_Supply2_Demand1  =  345.0
route_Type1_Supply2_Demand2  =  0.0
route_Type1_Supply2_Demand3  =  5.0


Part 2

The marketing team realizes that a 1-kilogram package might not attend all types of demand for premium coffee and decide to test a second product. This other product is 1 package containing 500 grams of the ground and roasted Arabica coffee to be commercialized in the same cities.

Both plants can produce both products: the 1-kilogram Coffee Indeed ® (product 1) and the 500-grams Coffee Indeed ® (product 2). Capacity in units applies to any product unit, independently of its weight or packaging. This new product 2 is transported in the same vehicles of product 1 using the same transportation costs; however, 1 pallet can hold 2000 units (two thousand) of product 2.

As the team foresees light cannibalization of the market share, the planning team forecasted a new demand for both products.

 Demand (units/week)	Bogotá (DC 1)	Medellín (DC 2)	Cali (DC 3)
 
1-kilogram Coffee Indeed ® (product 1)	276,000	210,400	137,600

500-grams Coffee Indeed ® (product 2)	138,000	105,200	68,800

What is the minimum weekly cost for moving both products from the plants to the three DCs?

In [32]:
types = ['Type1', 'Type2']
supply = ['Supply1', 'Supply2']
                        #sup1 #sup2
supplycap = np.array([610,   350])


demand = ['Demand1', 'Demand2', 'Demand3']
                      #dem1 #dem2 #dem3
demandcap = np.array([[276,210.4,137.6], #type1
                      [138, 105.2, 68.8]])#type2

                            #sup1            #sup2   
                            #dem1#dem2#dem3 #dem1#dem2#dem3
cost_sup_dem   = np.array([[[130,90.60,142],[53.6, 41.6, 88.8]], #type1
                           [[130/2,90.6/2,142/2],[53.6/2,41.6/2,88.8/2]]]) #type2
                  
supplyconstraint = p.makeDict([supply], supplycap)
demandconstraint = p.makeDict([types, demand], demandcap)

sup_dem_dict = p.makeDict([types, supply, demand], cost_sup_dem*1) #cost of 1 dollar per mile per unit

prob = p.LpProblem('Coffee_Production', p.LpMinimize)

sup_to_dem_route = [(t,s,d) for t in types for s in supply for d in demand]

sup_to_dem_vars = p.LpVariable.dicts("route", (types, supply,demand), lowBound = 0, cat = 'Continuous')

prob += p.lpSum([sup_to_dem_vars[t][s][d]*sup_dem_dict[t][s][d] for (t,s,d) in sup_to_dem_route])\


for t in types: #note the posisitoning of the for loops and how it corresponds to the constraints we are trying to model
    for d in demand:
        prob += p.lpSum([sup_to_dem_vars[t][s][d] for s in supply]) == demandconstraint[t][d] #RW demand constraint
        
for s in supply:
        prob += p.lpSum([sup_to_dem_vars[t][s][d] for t in types for d in demand]) <= supplyconstraint[s] #plant cap constraint


prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  68078.59999999999
Solution Status: Optimal
route_Type1_Supply1_Demand1  =  0.0
route_Type1_Supply1_Demand2  =  210.4
route_Type1_Supply1_Demand3  =  63.6
route_Type1_Supply2_Demand1  =  276.0
route_Type1_Supply2_Demand2  =  0.0
route_Type1_Supply2_Demand3  =  74.0
route_Type2_Supply1_Demand1  =  138.0
route_Type2_Supply1_Demand2  =  105.2
route_Type2_Supply1_Demand3  =  68.8
route_Type2_Supply2_Demand1  =  0.0
route_Type2_Supply2_Demand2  =  0.0
route_Type2_Supply2_Demand3  =  0.0


Part 3

You are informed that due to a severe fire in plant 2, some packaging machinery was lost. To keep both plants producing both types of products, the maintenance manager Ms. Jean Rivera has redistributed the remaining machinery in both plants. This move enabled both plants with the flexibility of producing both types of products; however, it restricted the production capacities as described below:

Demand:

Barranquilla (Plant 1) cannot produce more than 300,000 500-grams Coffee Indeed ® (product 2) per week.

Santander (Plant 2) cannot produce more than 250,000 1-kilogram Coffee Indeed ® (product 1) per week.

Create a linear program that optimizes the flow of each product from plants to DCs while adhering to these new capacity constraints.

Please, consider at all times that total capacities of both plants haven't changed as described in the problem statement.

Calculate the minimum weekly cost for satisfying the demand of the three DCs. How much more expensive is this solution than the one in Part 2?

In [35]:
types = ['Type1', 'Type2']
supply = ['Supply1', 'Supply2']
                        #sup1 #sup2
supplycap = np.array([[99999,  250], #type1
                      [300,  99999]]) #type2  

demand = ['Demand1', 'Demand2', 'Demand3']
                      #dem1 #dem2 #dem3
demandcap = np.array([[276,210.4,137.6], #type1
                      [138, 105.2, 68.8]])#type2

                            #sup1            #sup2   
                            #dem1#dem2#dem3 #dem1#dem2#dem3
cost_sup_dem   = np.array([[[130,90.60,142],[53.6, 41.6, 88.8]], #type1
                           [[130/2,90.6/2,142/2],[53.6/2,41.6/2,88.8/2]]]) #type2
                  
supplyconstraint = p.makeDict([types, supply], supplycap)
demandconstraint = p.makeDict([types, demand], demandcap)

sup_dem_dict = p.makeDict([types, supply, demand], cost_sup_dem*1) #cost of 1 dollar per mile per unit

prob = p.LpProblem('Coffee_Production', p.LpMinimize)

sup_to_dem_route = [(t,s,d) for t in types for s in supply for d in demand]

sup_to_dem_vars = p.LpVariable.dicts("route", (types, supply,demand), lowBound = 0, cat = 'Continuous')

prob += p.lpSum([sup_to_dem_vars[t][s][d]*sup_dem_dict[t][s][d] for (t,s,d) in sup_to_dem_route])\


for t in types: #note the posisitoning of the for loops and how it corresponds to the constraints we are trying to model
    for d in demand:
        prob += p.lpSum([sup_to_dem_vars[t][s][d] for s in supply]) == demandconstraint[t][d] #RW demand constraint
                      
    for s in supply:
        prob += p.lpSum([sup_to_dem_vars[t][s][d] for d in demand]) <= supplyconstraint[t][s] #plant cap constraint

prob += p.lpSum([sup_to_dem_vars[t]['Supply1'][d] for t in types for d in demand]) <= 610 #total sup constraint irrespective of product type
                      
prob += p.lpSum([sup_to_dem_vars[t]['Supply2'][d] for t in types for d in demand]) <= 350 #total sup constraint irrespective of product type


prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  70181.8
Solution Status: Optimal
route_Type1_Supply1_Demand1  =  26.0
route_Type1_Supply1_Demand2  =  210.4
route_Type1_Supply1_Demand3  =  137.6
route_Type1_Supply2_Demand1  =  250.0
route_Type1_Supply2_Demand2  =  0.0
route_Type1_Supply2_Demand3  =  0.0
route_Type2_Supply1_Demand1  =  38.0
route_Type2_Supply1_Demand2  =  105.2
route_Type2_Supply1_Demand3  =  68.8
route_Type2_Supply2_Demand1  =  100.0
route_Type2_Supply2_Demand2  =  0.0
route_Type2_Supply2_Demand3  =  0.0


Part 4

The Financial team believes that you haven’t included the fixed cost of running a production line at a plant in your model. The controller tells you that CPC spends 12,000dollars for each type of product plant 1 produces, and 8,000dollars for each type of product plant 2 produces. In other words, if Plant 2 is producing one product, it costs 8,000dollars to run it, but if that plant is producing both products, CPC spends 16,000dollars.

How should we adjust the variables and constraints in the model to implement the changes requested by the Financial team?

Part 5

Building off of the model from Part 3 and considering the changes you identified in Part 4, create a linear program that optimizes the flow of each coffee product from plants to DCs, while adhering to the capacity constraints from Part 3 as well as the fixed cost per product per plant.

Which of the following statements are true for the optimal solution to this problem?

Part 6

What is the total weekly cost for flowing both coffee products from the two plants to the three DCs for the problem as described in Part 5?

In [39]:
types = ['Type1', 'Type2']
supply = ['Supply1', 'Supply2']
                        #sup1 #sup2
supplycap = np.array([[99999,  250], #type1
                      [300,  99999]]) #type2

                             #sup1 #sup2
supplyfixedcost = np.array([[12000, 8000], #type1
                            [12000, 8000]]) #type2

demand = ['Demand1', 'Demand2', 'Demand3']
                      #dem1 #dem2 #dem3
demandcap = np.array([[276,210.4,137.6], #type1
                      [138, 105.2, 68.8]])#type2

                            #sup1            #sup2   
                            #dem1#dem2#dem3 #dem1#dem2#dem3
cost_sup_dem   = np.array([[[130,90.60,142],[53.6, 41.6, 88.8]], #type1
                           [[130/2,90.6/2,142/2],[53.6/2,41.6/2,88.8/2]]]) #type2
                  
supplyconstraint = p.makeDict([types, supply], supplycap)
demandconstraint = p.makeDict([types, demand], demandcap)

sup_dem_dict = p.makeDict([types, supply, demand], cost_sup_dem*1) #cost of 1 dollar per mile per unit

sup_fixed_cost_dict = p.makeDict([types, supply], supplyfixedcost)

prob = p.LpProblem('Coffee_Production', p.LpMinimize)

sup_to_dem_route = [(t,s,d) for t in types for s in supply for d in demand]

sup_to_dem_vars = p.LpVariable.dicts("route", (types, supply,demand), lowBound = 0, cat = 'Continuous')

sup_open = p.LpVariable.dicts("open", (types, supply), cat = 'Binary')

prob += p.lpSum([sup_to_dem_vars[t][s][d]*sup_dem_dict[t][s][d] for (t,s,d) in sup_to_dem_route]) +\
        p.lpSum([sup_open[t][s]*sup_fixed_cost_dict[t][s] for t in types for s in supply])
        


for t in types: #note the posisitoning of the for loops and how it corresponds to the constraints we are trying to model
    for d in demand:
        prob += p.lpSum([sup_to_dem_vars[t][s][d] for s in supply]) == demandconstraint[t][d] #RW demand constraint
                      
    for s in supply:
        prob += p.lpSum([sup_to_dem_vars[t][s][d] for d in demand]) <= supplyconstraint[t][s] #plant cap constraint
        
        prob += p.lpSum([sup_to_dem_vars[t][s][d] for d in demand]) - 10000000000*sup_open[t][s] <= 0 #linking constraint between flow and fixed cost
        
    

prob += p.lpSum([sup_to_dem_vars[t]['Supply1'][d] for t in types for d in demand]) <= 610 #total sup constraint irrespective of product type
                      
prob += p.lpSum([sup_to_dem_vars[t]['Supply2'][d] for t in types for d in demand]) <= 350 #total sup constraint irrespective of product type


prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  108519.52
Solution Status: Optimal
open_Type1_Supply1  =  1.0
open_Type1_Supply2  =  1.0
open_Type2_Supply1  =  0.0
open_Type2_Supply2  =  1.0
route_Type1_Supply1_Demand1  =  238.0
route_Type1_Supply1_Demand2  =  210.4
route_Type1_Supply1_Demand3  =  137.6
route_Type1_Supply2_Demand1  =  38.0
route_Type1_Supply2_Demand2  =  0.0
route_Type1_Supply2_Demand3  =  0.0
route_Type2_Supply1_Demand1  =  0.0
route_Type2_Supply1_Demand2  =  0.0
route_Type2_Supply1_Demand3  =  0.0
route_Type2_Supply2_Demand1  =  138.0
route_Type2_Supply2_Demand2  =  105.2
route_Type2_Supply2_Demand3  =  68.8


<b>Graded Assignment 3:  WUWU Expanded</b>
    
In this assignment, we revisit the WUWU Problem, which was previously discussed in this week's Lesson 2 (specifically in Video 3)

<img src = 'w2ga31.png'>

<img src = 'w2ga32.png'>

For the questions below, we will use as our starting point the WUWU 1 problem, which is found in the WUWU1 tab of the spreadsheet (not the WUWU2 tab). As it stands now, the objective function for WUWU1 problem has an optimal value of $6,525.00. Try solving for the problem in its current form: you should get the same value.

The questions below will ask you to modify the parameters of this model, and the model itself, to include new constraints and products.

Part 1

For the first variation of the problem, we will change the capacity in both our DCs. Please increase the capacity of the Atlanta DC from 200 widgets to 300 widgets, and decrease the capacity of the Boston DC from 500 widgets to 400 widgets. Solve optimally.

How many widgets #1 is the Dallas plant sending to the Atlanta DC? (Not graded)

How many widgets #1 is the Boston DC sending to the NY region? (Not graded)

Part 2

Please provide your answer in dollars, but using only numbers. For example, for a solution of $1,234 you would enter 1234 as your answer. Do this for all answers to questions about cost in this assignment.

What is the new minimum cost of the optimal solution? (Graded)


In [45]:
types = ['Type1', 'Type2', 'Type3']
supply = ['Supply1', 'Supply2','Supply3']
                      #sup1 #sup2 #sup3
supplycap = np.array([[200,   125,  50],#type1
                      [200,   200,  50],#type2
                      [50,    150, 200]])#type3


demand = ['Demand1', 'Demand2', 'Demand3']
                      #dem1 #dem2 #dem3
demandcap = np.array([[50, 100, 75], #type1
                      [100, 50, 75], #type2
                      [25, 25, 150]])#type3

intermediate = ['Intermediate1','Intermediate2']

intermediatecap = np.array([300,400])

                            #sup1       #sup2       #sup3  
                            #int1#int2 #int1#int2  #int1#int2
cost_sup_int   = np.array([[[6,     5],[4,      7],[6,      9]], #type1
                           [[6,     5],[4,      7],[6,      9]], #type2
                           [[6,     5],[4,      7],[4,      7]]]) #type3
                  
                            #int1           #int2
                         #Dem1#Dem2#Dem3  #Dem1#Dem2#Dem3
cost_int_dem = np.array([[[8,   5,    6], [9,  7,  6]],#type1
                         [[7,   8,    5], [3,  8,  6]],#type2
                         [[7,   4,    4], [4,  5,  4]]]) #type3

supplyconstraint = p.makeDict([types, supply], supplycap)
demandconstraint = p.makeDict([types, demand], demandcap)
intermediateconstraint = p.makeDict([intermediate], intermediatecap)

sup_int_dict = p.makeDict([types, supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per unit
int_dem_dict = p.makeDict([types, intermediate, demand], cost_int_dem*1) 

prob = p.LpProblem('WUWU_Production', p.LpMinimize)

sup_to_int_route = [(t,s,i) for t in types for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (types, supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(t,i,d) for t in types for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (types, intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[t][s][i]*sup_int_dict[t][s][i] for (t,s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[t][i][d]*int_dem_dict[t][i][d] for (t,i,d) in int_to_dem_route])

for t in types: #note the posisitoning of the for loops and how it corresponds to the constraints we are trying to model
    for s in supply:
        prob += p.lpSum([sup_to_int_vars[t][s][i] for i in intermediate]) <= supplyconstraint[t][s] #plant cap constraint

    for d in demand:
        prob += p.lpSum([int_to_dem_vars[t][i][d] for i in intermediate]) == demandconstraint[t][d] #RW demand constraint

    for i in intermediate:
        prob += p.lpSum([sup_to_int_vars[t][s][i] for s in supply]) == p.lpSum([int_to_dem_vars[t][i][d] for d in demand]) #conservation of flow constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[t][s][i] for t in types for s in supply]) <= intermediateconstraint[i] #intermediatecap constraint

prob += p.lpSum([sup_to_int_vars[t]['Supply2'][i] for t in types for i in intermediate]) <= 250 #constraint of total production in Sup2 must not exceed a certain number
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  6250.0
Solution Status: Optimal
route_Type1_Intermediate1_Demand1  =  0.0
route_Type1_Intermediate1_Demand2  =  100.0
route_Type1_Intermediate1_Demand3  =  0.0
route_Type1_Intermediate2_Demand1  =  50.0
route_Type1_Intermediate2_Demand2  =  0.0
route_Type1_Intermediate2_Demand3  =  75.0
route_Type1_Supply1_Intermediate1  =  0.0
route_Type1_Supply1_Intermediate2  =  125.0
route_Type1_Supply2_Intermediate1  =  100.0
route_Type1_Supply2_Intermediate2  =  0.0
route_Type1_Supply3_Intermediate1  =  0.0
route_Type1_Supply3_Intermediate2  =  0.0
route_Type2_Intermediate1_Demand1  =  0.0
route_Type2_Intermediate1_Demand2  =  0.0
route_Type2_Intermediate1_Demand3  =  50.0
route_Type2_Intermediate2_Demand1  =  100.0
route_Type2_Intermediate2_Demand2  =  50.0
route_Type2_Intermediate2_Demand3  =  25.0
route_Type2_Supply1_Intermediate1  =  0.0
route_Type2_Supply1_Intermediate2  =  175.0
route_Type2_Supply2_Intermediate1  =  50.0
route_Type2_Supply2_Intermediate2  =  0.0
route_Type2_S

Part 3

For the second variation, we will build upon Part 1 and add a new constraint to the problem: the Miami plant will now have an overall capacity constraint of 250 widgets of any type. Calculate the new minimum cost solution. Compare this new cost to the one in Part 2.

Did the new constraint that we just added to the problem make any difference in the solution? --> no, since in part2 capacity used is only 150

In [44]:
types = ['Type1', 'Type2', 'Type3']
supply = ['Supply1', 'Supply2','Supply3']
                      #sup1 #sup2 #sup3
supplycap = np.array([[200,   125,  50],#type1
                      [200,   200,  50],#type2
                      [50,    150, 200]])#type3


demand = ['Demand1', 'Demand2', 'Demand3']
                      #dem1 #dem2 #dem3
demandcap = np.array([[50, 100, 75], #type1
                      [100, 50, 75], #type2
                      [25, 25, 150]])#type3

intermediate = ['Intermediate1','Intermediate2']

intermediatecap = np.array([300,400])

                            #sup1       #sup2       #sup3  
                            #int1#int2 #int1#int2  #int1#int2
cost_sup_int   = np.array([[[6,     5],[4,      7],[6,      9]], #type1
                           [[6,     5],[4,      7],[6,      9]], #type2
                           [[6,     5],[4,      7],[4,      7]]]) #type3
                  
                            #int1           #int2
                         #Dem1#Dem2#Dem3  #Dem1#Dem2#Dem3
cost_int_dem = np.array([[[8,   5,    6], [9,  7,  6]],#type1
                         [[7,   8,    5], [3,  8,  6]],#type2
                         [[7,   4,    4], [4,  5,  4]]]) #type3

supplyconstraint = p.makeDict([types, supply], supplycap)
demandconstraint = p.makeDict([types, demand], demandcap)
intermediateconstraint = p.makeDict([intermediate], intermediatecap)

sup_int_dict = p.makeDict([types, supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per unit
int_dem_dict = p.makeDict([types, intermediate, demand], cost_int_dem*1) 

prob = p.LpProblem('WUWU_Production', p.LpMinimize)

sup_to_int_route = [(t,s,i) for t in types for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (types, supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(t,i,d) for t in types for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (types, intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[t][s][i]*sup_int_dict[t][s][i] for (t,s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[t][i][d]*int_dem_dict[t][i][d] for (t,i,d) in int_to_dem_route])

for t in types: #note the posisitoning of the for loops and how it corresponds to the constraints we are trying to model
    for s in supply:
        prob += p.lpSum([sup_to_int_vars[t][s][i] for i in intermediate]) <= supplyconstraint[t][s] #plant cap constraint

    for d in demand:
        prob += p.lpSum([int_to_dem_vars[t][i][d] for i in intermediate]) == demandconstraint[t][d] #RW demand constraint

    for i in intermediate:
        prob += p.lpSum([sup_to_int_vars[t][s][i] for s in supply]) == p.lpSum([int_to_dem_vars[t][i][d] for d in demand]) #conservation of flow constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[t][s][i] for t in types for s in supply]) <= intermediateconstraint[i] #intermediatecap constraint

prob += p.lpSum([sup_to_int_vars[t]['Supply2'][i] for t in types for i in intermediate]) <= 250 #constraint of total production in Sup2 must not exceed a certain number

prob += p.lpSum([sup_to_int_vars[t]['Supply3'][i] for t in types for i in intermediate]) <= 250 #constraint of total production in Sup2 must not exceed a certain number
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  6250.0
Solution Status: Optimal
route_Type1_Intermediate1_Demand1  =  0.0
route_Type1_Intermediate1_Demand2  =  100.0
route_Type1_Intermediate1_Demand3  =  0.0
route_Type1_Intermediate2_Demand1  =  50.0
route_Type1_Intermediate2_Demand2  =  0.0
route_Type1_Intermediate2_Demand3  =  75.0
route_Type1_Supply1_Intermediate1  =  0.0
route_Type1_Supply1_Intermediate2  =  125.0
route_Type1_Supply2_Intermediate1  =  100.0
route_Type1_Supply2_Intermediate2  =  0.0
route_Type1_Supply3_Intermediate1  =  0.0
route_Type1_Supply3_Intermediate2  =  0.0
route_Type2_Intermediate1_Demand1  =  0.0
route_Type2_Intermediate1_Demand2  =  0.0
route_Type2_Intermediate1_Demand3  =  50.0
route_Type2_Intermediate2_Demand1  =  100.0
route_Type2_Intermediate2_Demand2  =  50.0
route_Type2_Intermediate2_Demand3  =  25.0
route_Type2_Supply1_Intermediate1  =  0.0
route_Type2_Supply1_Intermediate2  =  175.0
route_Type2_Supply2_Intermediate1  =  50.0
route_Type2_Supply2_Intermediate2  =  0.0
route_Type2_S

Part 4

For the third variation, we will build upon Part 3 and add a new constraint to the problem: the Chicago plant will now have an overall capacity constraint of 250 widgets of any type. Solve optimally.

How many widgets #2 is the Chicago plant sending to the Boston DC?

How many widgets #3 is the Boston DC sending to the PA region?

Part 5

Remember to give your answer to this and all cost questions in terms of dollars, using only numbers.

What is the new minimum cost of the optimal solution? (Graded)

In [51]:
types = ['Widget1', 'Widget2', 'Widget3']
supply = ['Chicago', 'Dallas','Miami']
                      #sup1 #sup2 #sup3
supplycap = np.array([[200,   125,  50],#type1
                      [200,   200,  50],#type2
                      [50,    150, 200]])#type3


demand = ['NY', 'VA', 'PA']
                      #dem1 #dem2 #dem3
demandcap = np.array([[50, 100, 75], #type1
                      [100, 50, 75], #type2
                      [25, 25, 150]])#type3

intermediate = ['Atlanta','Boston']

intermediatecap = np.array([300,400])

                            #sup1       #sup2       #sup3  
                            #int1#int2 #int1#int2  #int1#int2
cost_sup_int   = np.array([[[6,     5],[4,      7],[6,      9]], #type1
                           [[6,     5],[4,      7],[6,      9]], #type2
                           [[6,     5],[4,      7],[4,      7]]]) #type3
                  
                            #int1           #int2
                         #Dem1#Dem2#Dem3  #Dem1#Dem2#Dem3
cost_int_dem = np.array([[[8,   5,    6], [9,  7,  6]],#type1
                         [[7,   8,    5], [3,  8,  6]],#type2
                         [[7,   4,    4], [4,  5,  4]]]) #type3

supplyconstraint = p.makeDict([types, supply], supplycap)
demandconstraint = p.makeDict([types, demand], demandcap)
intermediateconstraint = p.makeDict([intermediate], intermediatecap)

sup_int_dict = p.makeDict([types, supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per unit
int_dem_dict = p.makeDict([types, intermediate, demand], cost_int_dem*1) 

prob = p.LpProblem('WUWU_Production', p.LpMinimize)

sup_to_int_route = [(t,s,i) for t in types for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (types, supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(t,i,d) for t in types for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (types, intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[t][s][i]*sup_int_dict[t][s][i] for (t,s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[t][i][d]*int_dem_dict[t][i][d] for (t,i,d) in int_to_dem_route])

for t in types: #note the posisitoning of the for loops and how it corresponds to the constraints we are trying to model
    for s in supply:
        prob += p.lpSum([sup_to_int_vars[t][s][i] for i in intermediate]) <= supplyconstraint[t][s] #plant cap constraint

    for d in demand:
        prob += p.lpSum([int_to_dem_vars[t][i][d] for i in intermediate]) == demandconstraint[t][d] #RW demand constraint

    for i in intermediate:
        prob += p.lpSum([sup_to_int_vars[t][s][i] for s in supply]) == p.lpSum([int_to_dem_vars[t][i][d] for d in demand]) #conservation of flow constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[t][s][i] for t in types for s in supply]) <= intermediateconstraint[i] #intermediatecap constraint


prob += p.lpSum([sup_to_int_vars[t]['Chicago'][i] for t in types for i in intermediate]) <= 250 #constraint of total production in Sup2 must not exceed a certain number

prob += p.lpSum([sup_to_int_vars[t]['Dallas'][i] for t in types for i in intermediate]) <= 250 #constraint of total production in Sup2 must not exceed a certain number

prob += p.lpSum([sup_to_int_vars[t]['Miami'][i] for t in types for i in intermediate]) <= 250 #constraint of total production in Sup2 must not exceed a certain number
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  6400.0
Solution Status: Optimal
route_Widget1_Atlanta_NY  =  25.0
route_Widget1_Atlanta_PA  =  0.0
route_Widget1_Atlanta_VA  =  100.0
route_Widget1_Boston_NY  =  25.0
route_Widget1_Boston_PA  =  75.0
route_Widget1_Boston_VA  =  0.0
route_Widget1_Chicago_Atlanta  =  0.0
route_Widget1_Chicago_Boston  =  100.0
route_Widget1_Dallas_Atlanta  =  125.0
route_Widget1_Dallas_Boston  =  0.0
route_Widget1_Miami_Atlanta  =  0.0
route_Widget1_Miami_Boston  =  0.0
route_Widget2_Atlanta_NY  =  0.0
route_Widget2_Atlanta_PA  =  75.0
route_Widget2_Atlanta_VA  =  0.0
route_Widget2_Boston_NY  =  100.0
route_Widget2_Boston_PA  =  0.0
route_Widget2_Boston_VA  =  50.0
route_Widget2_Chicago_Atlanta  =  0.0
route_Widget2_Chicago_Boston  =  100.0
route_Widget2_Dallas_Atlanta  =  75.0
route_Widget2_Dallas_Boston  =  50.0
route_Widget2_Miami_Atlanta  =  0.0
route_Widget2_Miami_Boston  =  0.0
route_Widget3_Atlanta_NY  =  0.0
route_Widget3_Atlanta_PA  =  75.0
route_Widget3_Atlanta_VA  =  25.0
route_W

Part 6

The fourth variation builds upon Part 4 and adds two new constraints. First, we add a 150 widget capacity constraint at the Atlanta DC specific to widget 1. In other words, the Atlanta DC can process at most 150 units of widget 1. Second, we add a similar constraint for the the Boston DC specific to widget 2: the Boston DC can now handle at most 200 units of widget 2.

What is the new minimum cost of the optimal solution?

In [59]:
types = ['Widget1', 'Widget2', 'Widget3']
supply = ['Chicago', 'Dallas','Miami']

                      #sup1 #sup2 #sup3
supplycap = np.array([[200,   125,  50],#type1
                      [200,   200,  50],#type2
                      [50,    150, 200]])#type3

demand = ['NY', 'VA', 'PA']

                      #dem1 #dem2 #dem3
demandcap = np.array([[50, 100, 75], #type1
                      [100, 50, 75], #type2
                      [25, 25, 150]])#type3

intermediate = ['Atlanta','Boston']

                            #int1 #int2
intermediatecap = np.array([[150, 999999],#type1
                            [999999,   200],#type2
                            [999999, 999999]])#type3

intermediatetotalcap = np.array([300,400])

                            #sup1       #sup2       #sup3  
                            #int1#int2 #int1#int2  #int1#int2
cost_sup_int   = np.array([[[6,     5],[4,      7],[6,      9]], #type1
                           [[6,     5],[4,      7],[6,      9]], #type2
                           [[6,     5],[4,      7],[4,      7]]]) #type3
                  
                            #int1           #int2
                         #Dem1#Dem2#Dem3  #Dem1#Dem2#Dem3
cost_int_dem = np.array([[[8,   5,    6], [9,  7,  6]],#type1
                         [[7,   8,    5], [3,  8,  6]],#type2
                         [[7,   4,    4], [4,  5,  4]]]) #type3

supplyconstraint = p.makeDict([types, supply], supplycap)
demandconstraint = p.makeDict([types, demand], demandcap)
intermediateconstraint = p.makeDict([types,intermediate], intermediatecap)
intermediatetotalconstraint = p.makeDict([intermediate], intermediatetotalcap)

sup_int_dict = p.makeDict([types, supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per unit
int_dem_dict = p.makeDict([types, intermediate, demand], cost_int_dem*1) 

prob = p.LpProblem('WUWU_Production', p.LpMinimize)

sup_to_int_route = [(t,s,i) for t in types for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (types, supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(t,i,d) for t in types for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (types, intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[t][s][i]*sup_int_dict[t][s][i] for (t,s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[t][i][d]*int_dem_dict[t][i][d] for (t,i,d) in int_to_dem_route])

for t in types: #note the posisitoning of the for loops and how it corresponds to the constraints we are trying to model
    for s in supply:
        prob += p.lpSum([sup_to_int_vars[t][s][i] for i in intermediate]) <= supplyconstraint[t][s] #plant cap constraint

    for d in demand:
        prob += p.lpSum([int_to_dem_vars[t][i][d] for i in intermediate]) == demandconstraint[t][d] #RW demand constraint

    for i in intermediate:
        prob += p.lpSum([sup_to_int_vars[t][s][i] for s in supply]) == p.lpSum([int_to_dem_vars[t][i][d] for d in demand]) #conservation of flow constraint
        prob += p.lpSum([sup_to_int_vars[t][s][i] for s in supply]) <= intermediateconstraint[t][i]

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[t][s][i] for t in types for s in supply]) <= intermediatetotalconstraint[i] #intermediatecap constraint


prob += p.lpSum([sup_to_int_vars[t]['Chicago'][i] for t in types for i in intermediate]) <= 250 #constraint of total production in Sup2 must not exceed a certain number

prob += p.lpSum([sup_to_int_vars[t]['Dallas'][i] for t in types for i in intermediate]) <= 250 #constraint of total production in Sup2 must not exceed a certain number

prob += p.lpSum([sup_to_int_vars[t]['Miami'][i] for t in types for i in intermediate]) <= 250 #constraint of total production in Sup2 must not exceed a certain number
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  6400.0
Solution Status: Optimal
route_Widget1_Atlanta_NY  =  25.0
route_Widget1_Atlanta_PA  =  0.0
route_Widget1_Atlanta_VA  =  100.0
route_Widget1_Boston_NY  =  25.0
route_Widget1_Boston_PA  =  75.0
route_Widget1_Boston_VA  =  0.0
route_Widget1_Chicago_Atlanta  =  0.0
route_Widget1_Chicago_Boston  =  100.0
route_Widget1_Dallas_Atlanta  =  125.0
route_Widget1_Dallas_Boston  =  0.0
route_Widget1_Miami_Atlanta  =  0.0
route_Widget1_Miami_Boston  =  0.0
route_Widget2_Atlanta_NY  =  0.0
route_Widget2_Atlanta_PA  =  75.0
route_Widget2_Atlanta_VA  =  0.0
route_Widget2_Boston_NY  =  100.0
route_Widget2_Boston_PA  =  0.0
route_Widget2_Boston_VA  =  50.0
route_Widget2_Chicago_Atlanta  =  0.0
route_Widget2_Chicago_Boston  =  100.0
route_Widget2_Dallas_Atlanta  =  75.0
route_Widget2_Dallas_Boston  =  50.0
route_Widget2_Miami_Atlanta  =  0.0
route_Widget2_Miami_Boston  =  0.0
route_Widget3_Atlanta_NY  =  0.0
route_Widget3_Atlanta_PA  =  75.0
route_Widget3_Atlanta_VA  =  25.0
route_W

Part 7

The fifth and last variation builds upon Part 6 and adds a new commodity to the portfolio: Widget 4. Demand in the regions for Widget 4 are as follows:

20 units in NY

20 units in VA

10 units in PA

Production capacity for Widget 4 in the different plants is as follows:

Up to 20 in Chicago

Up to 40 in Dallas

Up to 25 in Miami

For Widget 4, the transportation costs from plants to DCs are:

6 from Chicago to Atlanta

5 from Chicago to Boston

4 from Dallas to Atlanta

7 from Dallas to Boston

5 from Miami to Atlanta

8 from Miami to Boston

For Widget 4, the transportation costs from DC to regions are:

8 from Atlanta to NY

6 from Atlanta to VA

6 from Atlanta to PA

5 from Boston to NY

6 from Boston to VA

5 from Boston to PA

What is the new minimum cost of the optimal solution? (Graded)

In [63]:
types = ['Widget1', 'Widget2', 'Widget3', 'Widget4']
supply = ['Chicago', 'Dallas','Miami']

                      #sup1 #sup2 #sup3
supplycap = np.array([[200,   125,  50],#type1
                      [200,   200,  50],#type2
                      [50,    150, 200],#type3
                      [20,    40,   25]])#type4

demand = ['NY', 'VA', 'PA']

                      #dem1 #dem2 #dem3
demandcap = np.array([[50, 100, 75], #type1
                      [100, 50, 75], #type2
                      [25, 25, 150], #type3
                      [20, 20,  10]])#type4

intermediate = ['Atlanta','Boston']

                            #int1 #int2
intermediatecap = np.array([[150, 999999],#type1
                            [999999,   200],#type2
                            [999999, 999999],#type3
                            [999999, 999999]])#type4

intermediatetotalcap = np.array([300,400])

                            #sup1       #sup2       #sup3  
                            #int1#int2 #int1#int2  #int1#int2
cost_sup_int   = np.array([[[6,     5],[4,      7],[6,      9]], #type1
                           [[6,     5],[4,      7],[6,      9]], #type2
                           [[6,     5],[4,      7],[4,      7]], #type3
                           [[6,     5],[4,      7],[5,      8]]]) #type4
                  
                            #int1           #int2
                         #Dem1#Dem2#Dem3  #Dem1#Dem2#Dem3
cost_int_dem = np.array([[[8,   5,    6], [9,  7,  6]],#type1
                         [[7,   8,    5], [3,  8,  6]],#type2
                         [[7,   4,    4], [4,  5,  4]],#type3
                         [[8,   6,    6], [5,  6,  5]]]) #type4

supplyconstraint = p.makeDict([types, supply], supplycap)
demandconstraint = p.makeDict([types, demand], demandcap)
intermediateconstraint = p.makeDict([types,intermediate], intermediatecap)
intermediatetotalconstraint = p.makeDict([intermediate], intermediatetotalcap)

sup_int_dict = p.makeDict([types, supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per unit
int_dem_dict = p.makeDict([types, intermediate, demand], cost_int_dem*1) 

prob = p.LpProblem('WUWU_Production', p.LpMinimize)

sup_to_int_route = [(t,s,i) for t in types for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (types, supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(t,i,d) for t in types for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (types, intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[t][s][i]*sup_int_dict[t][s][i] for (t,s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[t][i][d]*int_dem_dict[t][i][d] for (t,i,d) in int_to_dem_route])

for t in types: #note the posisitoning of the for loops and how it corresponds to the constraints we are trying to model
    for s in supply:
        prob += p.lpSum([sup_to_int_vars[t][s][i] for i in intermediate]) <= supplyconstraint[t][s] #plant cap constraint

    for d in demand:
        prob += p.lpSum([int_to_dem_vars[t][i][d] for i in intermediate]) == demandconstraint[t][d] #RW demand constraint

    for i in intermediate:
        prob += p.lpSum([sup_to_int_vars[t][s][i] for s in supply]) == p.lpSum([int_to_dem_vars[t][i][d] for d in demand]) #conservation of flow constraint
        prob += p.lpSum([sup_to_int_vars[t][s][i] for s in supply]) <= intermediateconstraint[t][i]

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[t][s][i] for t in types for s in supply]) <= intermediatetotalconstraint[i] #intermediatecap constraint


prob += p.lpSum([sup_to_int_vars[t]['Chicago'][i] for t in types for i in intermediate]) <= 250 #constraint of total production in Sup2 must not exceed a certain number

prob += p.lpSum([sup_to_int_vars[t]['Dallas'][i] for t in types for i in intermediate]) <= 250 #constraint of total production in Sup2 must not exceed a certain number

prob += p.lpSum([sup_to_int_vars[t]['Miami'][i] for t in types for i in intermediate]) <= 250 #constraint of total production in Sup2 must not exceed a certain number
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  7020.0
Solution Status: Optimal
route_Widget1_Atlanta_NY  =  25.0
route_Widget1_Atlanta_PA  =  0.0
route_Widget1_Atlanta_VA  =  100.0
route_Widget1_Boston_NY  =  25.0
route_Widget1_Boston_PA  =  75.0
route_Widget1_Boston_VA  =  0.0
route_Widget1_Chicago_Atlanta  =  0.0
route_Widget1_Chicago_Boston  =  100.0
route_Widget1_Dallas_Atlanta  =  125.0
route_Widget1_Dallas_Boston  =  0.0
route_Widget1_Miami_Atlanta  =  0.0
route_Widget1_Miami_Boston  =  0.0
route_Widget2_Atlanta_NY  =  0.0
route_Widget2_Atlanta_PA  =  75.0
route_Widget2_Atlanta_VA  =  0.0
route_Widget2_Boston_NY  =  100.0
route_Widget2_Boston_PA  =  0.0
route_Widget2_Boston_VA  =  50.0
route_Widget2_Chicago_Atlanta  =  0.0
route_Widget2_Chicago_Boston  =  130.0
route_Widget2_Dallas_Atlanta  =  75.0
route_Widget2_Dallas_Boston  =  20.0
route_Widget2_Miami_Atlanta  =  0.0
route_Widget2_Miami_Boston  =  0.0
route_Widget3_Atlanta_NY  =  0.0
route_Widget3_Atlanta_PA  =  55.0
route_Widget3_Atlanta_VA  =  25.0
route_W

<h3>MicroMasters Practice Problem 1: Fruity Juice</h3>
    
Fruity Juice sells freshly squeezed fruit drinks from its two (2) stores in the greater Boston area. One of the main ingredients for their specialty drinks is bananas. Each store uses 20 boxes of bananas per week for these fruit drinks.

Currently, Fruity Juice uses two (2) wholesalers to supply the bananas. Wholesaler 1 charges 25dollars per box to supply Store 1, but adds a surcharge at 1dollar per box to supply Store 2 (for a total of 26dollar per box) because Store 2 does not permit back room loading by hand lift. Wholesaler 2 charges 25.50dollar per box regardless of the store. Each wholesaler has the capacity to deliver 50 boxes per week to the stores.

Now, a manager at Fruity Juice has been given the option to source directly from a Boston-area distributor at $20 per box. However, in order to get the bananas at the lower price they have to be picked up at the warehouse by Fruity Juice. For this they can use the company van, which only fits 10 boxes. The van would be available for this trip only once a week.

Considering the cost of being stuck in the Boston traffic, bananas are not shipped between the stores.

<b>Part 1</b>

Suppose Fruity Juice wants to satisfy demand at minimal cost. What is the optimal number of boxes of bananas to source directly from the Distributor?

<b>Part 2</b>

What is the total cost of the optimal solution?

<b>Part 4</b>

Wholesaler 2 really wants your business. They are willing to offer you all the bananas you want, at $24 per box fully delivered to both stores.

What should you do?

In [5]:
supply = ['Wholesaler1', 'Wholesaler2','Direct Source']
supplycap = [50, 50, 10]
demand = ['Store1', 'Store2']
demandcap = [20,20]

                   #dem1 #dem2
cost_sup_dem   = np.array([[25, 26], #sup1
                  [24, 24], #sup2
                  [20, 20]]) #sup3
                  

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

sup_dem_dict = p.makeDict([supply, demand], cost_sup_dem)

prob = p.LpProblem('Banana_Sourcing', p.LpMinimize)

sup_to_dem_route = [(s,d) for s in supply for d in demand]

sup_to_dem_vars = p.LpVariable.dicts("route", (supply,demand), lowBound = 0, cat = 'Integer') 

prob += p.lpSum([sup_to_dem_vars[s][d]*sup_dem_dict[s][d] for (s,d) in sup_to_dem_route]) 
                  
                   
for s in supply:
    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) <= supplyconstraint[s] #constraint must be less than DC capacity
                  
for d in demand:
    prob += p.lpSum([sup_to_dem_vars[s][d] for s in supply]) >= demandconstraint[d] #constraint must be equal or more than shelter demand

prob.solve()

print('minimum cost:', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost: 920.0
Solution Status: Optimal
route_Direct_Source_Store1  =  0.0
route_Direct_Source_Store2  =  10.0
route_Wholesaler1_Store1  =  0.0
route_Wholesaler1_Store2  =  0.0
route_Wholesaler2_Store1  =  20.0
route_Wholesaler2_Store2  =  10.0


<h3>MicroMasters Practice Problem 2: Distribution of Golf Caps</h3>
    
A large Golf equipment retailer with 20 stores across Europe has decided to sell a new type of multi-purpose, one-size-fits-all golf cap. The logistics manager has approached you to help him decide how to plan the distribution.

There are five (5) wholesale locations that can supply the caps, and the retailer uses two (2) distribution centers (DCs) to break the shipments and sort them for the stores.  The Golf Caps will flow from the wholesalers to the DCs and then onto the stores. 

<b>Part 1</b>

Which wholesalers' shipments should be routed through DC 1 in order to minimize costs?

<b>Part 2</b>

How many stores are supplied by more than one DC in the cost optimal solution?

<b>Part 3</b>

What is the cost of the cost-optimal solution?

In [7]:
supply = ['Wholesaler1', 'Wholesaler2', 'Wholesaler3', 'Wholesaler4', 'Wholesaler5']
supplycap = np.array([50,50,50,50,50])

demand = ['Store1','Store2','Store3','Store4','Store5','Store6','Store7','Store8','Store9','Store10','Store11','Store12','Store13','Store14','Store15','Store16','Store17','Store18','Store19','Store20']
demandcap = np.array([12,15,9,15,13,7,6,18,11,16,17,12,5,14,11,8,13,15,15,13])

intermediate = ['DC1','DC2']

                          #int1 #int2
cost_sup_int   = np.array([[6,10], #sup1
                           [9,5], #sup2
                           [10,10], #sup3
                           [6,2], #sup4
                           [3,5]]) #sup5
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6....#Dem20
cost_int_dem = np.array([[8,5,1,10,8,2,8,1,5,7,2,1,9,9,7,1,6,8,2,8],#int1
                         [5,4,8,8,5,3,1,8,1,2,4,10,7,8,2,10,7,6,10,4]]) #int2

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))


sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per box
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*1) #cost of 1 dollar per mile per box

prob = p.LpProblem('Golf_Cap_Distribution', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) >= demandconstraint[d] #demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  2119.0
Solution Status: Optimal
route_DC1_Store1  =  0.0
route_DC1_Store10  =  0.0
route_DC1_Store11  =  17.0
route_DC1_Store12  =  12.0
route_DC1_Store13  =  0.0
route_DC1_Store14  =  0.0
route_DC1_Store15  =  0.0
route_DC1_Store16  =  8.0
route_DC1_Store17  =  13.0
route_DC1_Store18  =  0.0
route_DC1_Store19  =  15.0
route_DC1_Store2  =  1.0
route_DC1_Store20  =  0.0
route_DC1_Store3  =  9.0
route_DC1_Store4  =  0.0
route_DC1_Store5  =  0.0
route_DC1_Store6  =  7.0
route_DC1_Store7  =  0.0
route_DC1_Store8  =  18.0
route_DC1_Store9  =  0.0
route_DC2_Store1  =  12.0
route_DC2_Store10  =  16.0
route_DC2_Store11  =  0.0
route_DC2_Store12  =  0.0
route_DC2_Store13  =  5.0
route_DC2_Store14  =  14.0
route_DC2_Store15  =  11.0
route_DC2_Store16  =  0.0
route_DC2_Store17  =  0.0
route_DC2_Store18  =  15.0
route_DC2_Store19  =  0.0
route_DC2_Store2  =  14.0
route_DC2_Store20  =  13.0
route_DC2_Store3  =  0.0
route_DC2_Store4  =  15.0
route_DC2_Store5  =  13.0
route_DC2_Store6 

<h3>MicroMasters Practice Problem 3: WashCo</h3>
    
As a side project to your studies you have taken on a consulting role for WashCo, a company that collects, washes, and distributes work uniforms for security companies in the greater Boston area. Your assignment is to help them locate a new small distribution center (DC) for the distribution of clean uniforms.

WashCo currently has contracts with eight (8) work sites in the Boston Area. Clothes (uniforms and others) that have been collected are washed at a large central facility north of Boston. This is handled by another part of the organization. Your assignment concerns the distribution of the cleaned uniforms back to the work sites.  That is, the flow from the washing facility through a DC and then on to the eight work sites.  The uniforms are transported from the washing facility to the DC in large vehicles. At the DC the shipment is de-consolidated into smaller quantities and delivered by  electric vans to the eight work sites. There is always just one delivery per day per work site. 

The daily demand (in number of uniforms) for the work sites have been estimated as:

Site 1: 20

Site 2: 15

Site 3: 32

Site 4: 12

Site 5: 29

Site 6: 44

Site 7: 22

Site 8: 26

<b>Part 1</b>

You are aware of several methods for finding a good potential location for the new Distribution Center (DC). The first method that comes to mind is the "center-of-gravity".

What is the center of gravity for the distribution of uniforms? Remember to include the washing facility in the analysis.

Answer with one decimal, e.g., if your answer is 8.57, then write 8.6.

X-coordinate:

Y-coordinate:

<b>Part 2</b>

You also want to try using the Weber (or minimum weighted distance method) for location of the new DC.

What is the optimal DC location's coordinates according to the Weber method?

Answer with one decimal, e.g., if your answer is 8.57, then write 8.6.

X-coordinate:

Y-coordinate:

In [9]:
#include washing facility as a node with demand of 200 (the sum of all the sites' demand), this is because we need to send dirty clothes to the Washing Facility as well

clothes = weber_problem(['Washing Facility','Site 1','Site 2','Site 3','Site 4','Site 5','Site 6','Site 7','Site 8'],[200,20,15,32,12,29,44,22,26],[10,2,5,3,9,1,3,6,9],[10,3,2,5,3,8,8,6,6])

clothes.find_cog()
clothes.find_weber()

Center of Gravity x coordinate:  7.114999999999999
Center of Gravity y coordinate:  7.8950000000000005
Center of Gravity avg. weighted dist:  4.125240063050645
weber x coordinate:  9.900000000000007
weber y coordinate:  9.900000000000007
weber avg. weighted dist:  3.840104596670352


(9.900000000000007, 9.900000000000007, 3.840104596670352)

<b>Part 3</b>

Seeing the limits in your previous models, you decide to use a discrete approach, taking into account the actual transportation costs. While transportation costs are linear in distance and volume (i.e. number of units shipped), they are different for shipments from the washing facility to the DC and from the DC to the work sites.

Transportation from the washing facility to the DC costs 1dollar per mile per uniform. Transportation from the DC to the work sites costs 3dollar per mile per uniform. You have decided to co-locate the DC by one of the 8 work sites or at the washing facility. The distances between the work sites and the washing facility are provided in the data, under the tab "distances".

Taking the different transportation costs into account, at which site should WashCo locate the DC? Note that we assume no fixed facility costs at this point.



In [24]:
supply = ['Washing_Facility']

intermediate = ['Washing_Facility_as_DC','Site 1','Site 2','Site 3','Site 4','Site 5','Site 6','Site 7','Site 8']

demand = ['Site 1','Site 2','Site 3','Site 4','Site 5','Site 6','Site 7','Site 8']

demandcap = [20,15,32,12,29,44,22,26]

                #int1....                          #int8
cost_sup_int = np.array([[0,10.63,9.43,8.6,7.07,9.22,7.28,5.66,4.12]]) #sup1

                   #dem1 #dem2 #dem3 #dem4 #dem5 #dem6 #dem7 #dem8
cost_int_dem   = np.array([[10.63,9.43,8.6,7.07,9.22,7.28,5.66,4.12], #int1...
                  [0,3.16,2.24,7,5.1,5.1,5,7.62],
                  [3.16,0,3.61,4.12,7.21,6.32,4.12,5.66],
                  [2.24,3.61,0,6.32,3.61,3,3.16,6.08],
                  [7,4.12,6.32,0,9.43,7.81,4.24,3],
                  [5.1,7.21,3.61,9.43,0,2,5.39,8.25],
                  [5.1,6.32,3,7.81,2,0,3.61,6.32],
                  [5,4.12,3.16,4.24,5.39,3.61,0,3],
                  [7.62,5.66,6.08,3,8.25,6.32,3,0]]) #int8
                  

demandconstraint = dict(zip(demand,demandcap))

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*1) #1dollar per shirt

int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*3) #3dollar per shirt

prob = p.LpProblem('Clothes', p.LpMinimize)

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer') #amount of items travelling between DC and shelter
                  
int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary') #binary variables DC open or close

prob += p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for i in intermediate for d in demand])\
        + p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for s in supply for i in intermediate])
                  
prob += p.lpSum([int_open[i] for i in intermediate]) <= 1 #number of DC minimum must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1 #number of DC maximum must be open
                   
#for s in supply:
#    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) <= supplyconstraint[s] #constraint must be less than DC capacity
                  
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d] #constraint must be equal to cities demand
    #using '<=' will result in a higher optimal cost compared to using '==' because in this problem there is no constraint on the supply

for i in intermediate:
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 100000000000*int_open[i] <= 0 #linking constraint if any channel from DC is flowing, the DC must be open
                 
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow

                     
                 
prob.solve()

print('minimum cost:', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost: 3252.85
Solution Status: Optimal
open_Site_1  =  0.0
open_Site_2  =  0.0
open_Site_3  =  0.0
open_Site_4  =  0.0
open_Site_5  =  0.0
open_Site_6  =  0.0
open_Site_7  =  1.0
open_Site_8  =  0.0
open_Washing_Facility_as_DC  =  0.0
route_Site_1_Site_1  =  0.0
route_Site_1_Site_2  =  0.0
route_Site_1_Site_3  =  0.0
route_Site_1_Site_4  =  0.0
route_Site_1_Site_5  =  0.0
route_Site_1_Site_6  =  0.0
route_Site_1_Site_7  =  0.0
route_Site_1_Site_8  =  0.0
route_Site_2_Site_1  =  0.0
route_Site_2_Site_2  =  0.0
route_Site_2_Site_3  =  0.0
route_Site_2_Site_4  =  0.0
route_Site_2_Site_5  =  0.0
route_Site_2_Site_6  =  0.0
route_Site_2_Site_7  =  0.0
route_Site_2_Site_8  =  0.0
route_Site_3_Site_1  =  0.0
route_Site_3_Site_2  =  0.0
route_Site_3_Site_3  =  0.0
route_Site_3_Site_4  =  0.0
route_Site_3_Site_5  =  0.0
route_Site_3_Site_6  =  0.0
route_Site_3_Site_7  =  0.0
route_Site_3_Site_8  =  0.0
route_Site_4_Site_1  =  0.0
route_Site_4_Site_2  =  0.0
route_Site_4_Site_3  =  0.0
r

<b>Part 4</b>

After talking to the firms at the different sites, you realize that the costs for running a DC are very different between the sites: some have plenty of room and capacity and are happy to let you use the space, whereas others are only willing to let you rent space at a very high cost.

After your initial assessment, you estimate the daily fixed costs as specified in the data file under tab "fixed costs".

Taking the different transport costs as well as the different fixed costs into account, at wich site should WashCo locate the DC?


In [26]:
supply = ['Washing_Facility']

intermediate = ['Washing_Facility_as_DC','Site 1','Site 2','Site 3','Site 4','Site 5','Site 6','Site 7','Site 8']

intermediate_fixed_cost = np.array([1000,500,600,150,200,850,1100,1000,900])

demand = ['Site 1','Site 2','Site 3','Site 4','Site 5','Site 6','Site 7','Site 8']

demandcap = [20,15,32,12,29,44,22,26]

                #int1....                          #int8
cost_sup_int = np.array([[0,10.63,9.43,8.6,7.07,9.22,7.28,5.66,4.12]]) #sup1

                   #dem1 #dem2 #dem3 #dem4 #dem5 #dem6 #dem7 #dem8
cost_int_dem   = np.array([[10.63,9.43,8.6,7.07,9.22,7.28,5.66,4.12], #int1...
                  [0,3.16,2.24,7,5.1,5.1,5,7.62],
                  [3.16,0,3.61,4.12,7.21,6.32,4.12,5.66],
                  [2.24,3.61,0,6.32,3.61,3,3.16,6.08],
                  [7,4.12,6.32,0,9.43,7.81,4.24,3],
                  [5.1,7.21,3.61,9.43,0,2,5.39,8.25],
                  [5.1,6.32,3,7.81,2,0,3.61,6.32],
                  [5,4.12,3.16,4.24,5.39,3.61,0,3],
                  [7.62,5.66,6.08,3,8.25,6.32,3,0]]) #int8
                  

demandconstraint = dict(zip(demand,demandcap))

int_fixed_cost_dict = dict(zip(intermediate, intermediate_fixed_cost))

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*1) #1dollar per shirt

int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*3) #3dollar per shirt

prob = p.LpProblem('Clothes', p.LpMinimize)

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer') #amount of items travelling between DC and shelter
                  
int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary') #binary variables DC open or close

prob += p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for i in intermediate for d in demand])\
        + p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for s in supply for i in intermediate])\
        + p.lpSum([int_open[i]*int_fixed_cost_dict[i] for i in intermediate])
                  
prob += p.lpSum([int_open[i] for i in intermediate]) <= 1 #number of DC minimum must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1 #number of DC maximum must be open
                   
#for s in supply:
#    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) <= supplyconstraint[s] #constraint must be less than DC capacity
                  
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d] #constraint must be equal to cities demand
    #using '<=' will result in a higher optimal cost compared to using '==' because in this problem there is no constraint on the supply

for i in intermediate:
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 100000000000*int_open[i] <= 0 #linking constraint if any channel from DC is flowing, the DC must be open
                 
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow

                     
                 
prob.solve()

print('minimum cost:', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost: 3787.24
Solution Status: Optimal
open_Site_1  =  0.0
open_Site_2  =  0.0
open_Site_3  =  1.0
open_Site_4  =  0.0
open_Site_5  =  0.0
open_Site_6  =  0.0
open_Site_7  =  0.0
open_Site_8  =  0.0
open_Washing_Facility_as_DC  =  0.0
route_Site_1_Site_1  =  0.0
route_Site_1_Site_2  =  0.0
route_Site_1_Site_3  =  0.0
route_Site_1_Site_4  =  0.0
route_Site_1_Site_5  =  0.0
route_Site_1_Site_6  =  0.0
route_Site_1_Site_7  =  0.0
route_Site_1_Site_8  =  0.0
route_Site_2_Site_1  =  0.0
route_Site_2_Site_2  =  0.0
route_Site_2_Site_3  =  0.0
route_Site_2_Site_4  =  0.0
route_Site_2_Site_5  =  0.0
route_Site_2_Site_6  =  0.0
route_Site_2_Site_7  =  0.0
route_Site_2_Site_8  =  0.0
route_Site_3_Site_1  =  20.0
route_Site_3_Site_2  =  15.0
route_Site_3_Site_3  =  32.0
route_Site_3_Site_4  =  12.0
route_Site_3_Site_5  =  29.0
route_Site_3_Site_6  =  44.0
route_Site_3_Site_7  =  22.0
route_Site_3_Site_8  =  26.0
route_Site_4_Site_1  =  0.0
route_Site_4_Site_2  =  0.0
route_Site_4_Site_3  

<h3>MicroMasters Practice Problem 1: Harley Stevenson</h3>
    
HarleyStevenson is a bicycle manufacturer operating a small supply chain distribution network. After hearing about your expertise in Supply Chain Network Design, they have asked you to help them analyze their network to find opportunities for improvement.

All HarleyStevenson's bicycles are made to stock from a single manufacturing site. Once per week, finished bicycles are transported in large batches, using direct transports, from the manufacturing site to three (3) Distribution Centers (DCs). The DCs are all operated by third party providers, at the following weekly costs for HarleyStevenson:

DC1: Fixed cost: 2,000, Variable cost: 5 per bicycle

DC2: Fixed cost: 1,700, Variable cost: 6 per bicycle

DC3: Fixed cost: 1,700, Variable cost: 5 per bicycle

From the DCs, the bicycles are transported to five (5) sales regions using common carrier transport. Current company policy states that each sales region be supplied by only one (1) of the DCs, but in practice this is not always the case. Transport cost data as well as the actual flows for a few typical weeks are found in the following data file: Excel, LibreOffice.

(Note from the data that the inbound transports may differ in price per bicycle each week, whereas the outbound transports are paid for according to the common carrier's list price, specified per bicycle.)

There are currently no capacity restrictions.

<b>Part 4</b>

Since the policy is not adhered to in all weeks, HarleyStevenson is considering changing the policy.

Based on your network spreadsheet optimization model, how many DCs would you suggest supply sales Region 1 if HarleySevenson wants to minimize costs using the current network structure (i.e. with 3 DCs being open)?

<b>Part 5</b>

HarleyStevenson is also unsure whether they should use all three DCs in the network.

What is the cost optimal number of DCs in the network? Note that your solution does not have to adhere to the policy in Part 3.

In [3]:

supply = ['Manufacturing_Site']

demand = ['Region1', 'Region2', 'Region3','Region4', 'Region5']
demandcap = np.array([200,230,215,160,225])

intermediate = ['DC1','DC2','DC3']
intermediate_fixed_cost = np.array([2000,1700,1700])
intermediate_variable_cost = np.array([5,6,5])


demandconstraint = p.makeDict([demand], demandcap) 

                          #int1 #int2 #int3
cost_sup_int   = np.array([[20,25,20]]) #sup1
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 
cost_int_dem = np.array([[25,26,29,45,35],#int1
                         [31,33,23,32,44], #int2
                         [43,35,31,24,22]]) #int3

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost))


sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int) 
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem)


prob = p.LpProblem('Harvey_Bicycle', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])\
                       
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d] #demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - sum(demandcap)*int_open[i] <= 0 #linking constraint between flow and Intermediate open 
    

    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 3 #number of DC maximum must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1 #number of DC minimum must be open

prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  55455.0
Solution Status: Optimal
open_DC1  =  1.0
open_DC2  =  0.0
open_DC3  =  1.0
route_DC1_Region1  =  200.0
route_DC1_Region2  =  230.0
route_DC1_Region3  =  215.0
route_DC1_Region4  =  0.0
route_DC1_Region5  =  0.0
route_DC2_Region1  =  0.0
route_DC2_Region2  =  0.0
route_DC2_Region3  =  0.0
route_DC2_Region4  =  0.0
route_DC2_Region5  =  0.0
route_DC3_Region1  =  0.0
route_DC3_Region2  =  0.0
route_DC3_Region3  =  0.0
route_DC3_Region4  =  160.0
route_DC3_Region5  =  225.0
route_Manufacturing_Site_DC1  =  645.0
route_Manufacturing_Site_DC2  =  0.0
route_Manufacturing_Site_DC3  =  385.0


<b>Part 3</b>

The current policy is to have each sales region supplied by only one DC. However, as we can see in the data, this policy is not followed all the time.

How much lower would HarleyStevenson's costs per week have been by adhering to the policy? Assume each region is supplied by the DC that has the lowest outbound transport cost for that region . Make sure you compare with the 'current flow' baseline. Answer in dollars without the dollar sign and without the minus sign, e.g., if your solution is -$2,391.54, you write 2392.

In [4]:
supply = ['Manufacturing_Site']

demand = ['Region1', 'Region2', 'Region3','Region4', 'Region5']
demandcap = np.array([200,230,215,160,225])

intermediate = ['DC1','DC2','DC3']
intermediate_fixed_cost = np.array([2000,1700,1700])
intermediate_variable_cost = np.array([5,6,5])


demandconstraint = p.makeDict([demand], demandcap) 

                          #int1 #int2 #int3
cost_sup_int   = np.array([[20,25,20]]) #sup1
                  

#block outbound flow by inputting large number for the outbound flow cost which we would like to block

                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 
cost_int_dem = np.array([[25,26,290000,450000,350000],#int1
                         [310000,330000,23,320000,440000], #int2
                         [430000,350000,310000,24,22]]) #int3

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost))


sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int) 
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem)


prob = p.LpProblem('Harvey_Bicycle', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])\
                       
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d] #demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - sum(demandcap)*int_open[i] <= 0 #linking constraint between flow and Intermediate open 
    

    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 3 #number of DC maximum must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1 #number of DC minimum must be open

prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  57155.0
Solution Status: Optimal
open_DC1  =  1.0
open_DC2  =  1.0
open_DC3  =  1.0
route_DC1_Region1  =  200.0
route_DC1_Region2  =  230.0
route_DC1_Region3  =  0.0
route_DC1_Region4  =  0.0
route_DC1_Region5  =  0.0
route_DC2_Region1  =  0.0
route_DC2_Region2  =  0.0
route_DC2_Region3  =  215.0
route_DC2_Region4  =  0.0
route_DC2_Region5  =  0.0
route_DC3_Region1  =  0.0
route_DC3_Region2  =  0.0
route_DC3_Region3  =  0.0
route_DC3_Region4  =  160.0
route_DC3_Region5  =  225.0
route_Manufacturing_Site_DC1  =  430.0
route_Manufacturing_Site_DC2  =  215.0
route_Manufacturing_Site_DC3  =  385.0


<h3>MicroMasters Practice Problem 3: Summer Camp</h3>
    
You are planning a wildlife summer camp for kids. The camp has grown over the years and now has three (3) major camp sites, each with with 50 kids, located within a one mile radius.

A part of the camp is a two-day joint hike for all camp sites. The idea is to have the children at each site hike up to one of several possible land marks in the woods, where they meet up with the children from the other camp sites for some games and lunch. After this, the children are grouped in teams of 15 and continue their hike to each group's unique night camp. In the morning, all groups go back to the land mark where they regroup and go back to their original camp sites.

You are choosing between four (4) land marks for the meet-up: Ye Olde Tree; the Field by the Cabin; the Small Falls; and Little Peak.

Knowing that it will be two long days for the kids, you think it may be a good idea to try and select the land mark that minimizes the total man-distance covered.

The one-way distances are given in the matrices below.

<img src = "w2supp21.png">

Part 1

Which land mark should be chosen as meet up location in order to minimize total man-miles?

Part 2

Suppose there are actually 60 kids at Site 1 (and 50 at the other two sites). Which of the night camp(s) should these 10 extra kids be allocated to when the groups are created by the land mark? Note that each night camp should still have at least 15 kids.

Part 3

Suppose there are still 60 kids at Site 1, but that the tents on the night camp grounds fit no more than 20 kids. If we still want at least 15 kids at each night camp, which of the night camp group(s) should the extra 10 kids be allocated to at the land mark?

Part 4

Suppose there are still 60 kids at Site 1 and that there is only room for 20 kids in the tents at the camp grounds. You decide that maybe it is a better idea to use two different land marks. Still, we want at least 15 kids per night camp location.

Which of the following statements is/are true about the optimal solution with two landmarks?

In [40]:
supply = ['Site1','Site2','Site3']
#supplycap = np.array([50,50,50])
supplycap = np.array([60,50,50]) #for part2, part 3, part4

demand = ['Camp1', 'Camp2', 'Camp3','Camp4', 'Camp5','Camp6', 'Camp7', 'Camp8','Camp9', 'Camp10']

intermediate = ['Ye olde tree','The field','Small falls', 'Little Peak']

supplyconstraint = p.makeDict([supply], supplycap)

                          #int1 #int2 #int3
cost_sup_int   = np.array([[1.03,0.97,1.49,1.17], #sup1
                           [1.43,1.63,2,0.59],
                           [1.59,0.83,1.58,0.63]]) #sup3
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 ... #Dem10 
cost_int_dem = np.array([[2.99,3.01,3.33,2.81,3.74,2.96,3.67,2.93,3.22,3.33],
                         [3.76,3.62,3.9,4.03,4.04,3.66,3.67,4.3,4.15,3.83],
                         [2.94,3.09,3.68,3.73,3.38,3.67,3.78,2.85,3.25,3.78],
                         [3.91,3.03,3.02,3.85,3.09,3.61,3.57,3.52,3.72,3.63]]) #int4


sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int) 
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem)


prob = p.LpProblem('Summer_Camp', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
           
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) == supplyconstraint[s]

for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) >= 15 #demand constraint
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) <= 20 #for Part3, Part4

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 10000000000*int_open[i] <= 0 #linking constraint between flow and Intermediate open 
    

    
#prob += p.lpSum([int_open[i] for i in intermediate]) <= 1 #number of DC maximum must be open
#prob += p.lpSum([int_open[i] for i in intermediate]) >= 1 #number of DC minimum must be open
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 2 #for part4
prob += p.lpSum([int_open[i] for i in intermediate]) >= 2 #for part4


prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  629.3
Solution Status: Optimal
open_Little_Peak  =  1.0
open_Small_falls  =  0.0
open_The_field  =  0.0
open_Ye_olde_tree  =  1.0
route_Little_Peak_Camp1  =  0.0
route_Little_Peak_Camp10  =  15.0
route_Little_Peak_Camp2  =  20.0
route_Little_Peak_Camp3  =  20.0
route_Little_Peak_Camp4  =  0.0
route_Little_Peak_Camp5  =  15.0
route_Little_Peak_Camp6  =  0.0
route_Little_Peak_Camp7  =  15.0
route_Little_Peak_Camp8  =  0.0
route_Little_Peak_Camp9  =  15.0
route_Site1_Little_Peak  =  0.0
route_Site1_Small_falls  =  0.0
route_Site1_The_field  =  0.0
route_Site1_Ye_olde_tree  =  60.0
route_Site2_Little_Peak  =  50.0
route_Site2_Small_falls  =  0.0
route_Site2_The_field  =  0.0
route_Site2_Ye_olde_tree  =  0.0
route_Site3_Little_Peak  =  50.0
route_Site3_Small_falls  =  0.0
route_Site3_The_field  =  0.0
route_Site3_Ye_olde_tree  =  0.0
route_Small_falls_Camp1  =  0.0
route_Small_falls_Camp10  =  0.0
route_Small_falls_Camp2  =  0.0
route_Small_falls_Camp3  =  0.0
route_Small_fal

<h3>MicroMasters Practice Problem 5: Inventory Considerations at SandyCo</h3>
    
Remember where we left SandyCo in Week 1? They had just invested in a packaging machine at Plant 2 when they suffered a whimsical breakdown in one of the distribution centers and had to re-plan the network.

SandyCo managed to make it through the breakdown, thanks to swift execution of the new plans, and their operations are now back to normal. However, they are interested in exploring how inventory costs could be proxied into their supply chain network design for the future.

Let us recap. SandyCo is a sand distributor who has two (2) facilities that mine, clean, and sort sand for use in cement, children’s playboxes, and small beaches. They distribute the sand from their two (2) plants, through two (2) packaging centers (DCs), to three (3) different customer regions where it is sold. They also have a packaging machine at Plant 2, so Plant 2 can supply Region 1 directly, without going through the DCs.

Each plant has a specific maximum weekly supply of available sand and each region has an expected minimum weekly required demand. The cost to distribute a ton of sand differs between each plant and region pairing due to distance and other factors.

The maximum weekly supply from Plant 1 is 100, from Plant 2 it is 125. The expected minimum weekly required demand (tons) is:

Region 1: 25
    
Region 2: 95
    
Region 3: 80
    
The transportation costs per ton of sand are given in the table below.

<img src = "w2supp51.png">

<b>Part 2</b>

SandyCo uses an inventory holding cost of $5 per ton sand and week. Use this and your linear approximation from Part 1 to update the previous optimization model to account for inventory costs.

Based on the optimization model, what is the total cost of the current network when inventory costs are included? No optimization is needed, only include the extra cost. Answer in dollars without the dollar sign.

<b>Part 3</b>

There is plenty of capacity in both distribution centers still, so SandCo wants to explore what the cost would be of using only one DC.

When inventory costs are taken into account, what is the difference in cost of using only DC A compared to using both DCs? Answer in dollars without the dollar sign. For instance, if your total cost from (Part 2) is 65,000 and you get a cost of 66,000, your answer is 65,000-66,000=-1000 (note the minus sign).

In [88]:
#IMPORTANT: CREATE INV FUNCTION OUTSIDE OF THE OBJECTIVE FUNCTION

def inv_cost(T, holdingcost):
    if T <= 10:
        return 2.06*T*holdingcost
    elif T > 10 and T <=25:
        return (7.97 + 1.26*T)*holdingcost
    elif T > 25 and T <=50:
        return (14.32 + T)*holdingcost
    else:
        return (29.27 + 0.7*T)*holdingcost

supply = ['Plant1','Plant2']
supplycap = np.array([100,125])

demand = ['Region1', 'Region2', 'Region3']
demandcap = [25,95,80]

intermediate = ['DC A','DC B']

supplyconstraint = p.makeDict([supply], supplycap)
demandconstraint = p.makeDict([demand], demandcap)

                          #int1 #int2
cost_sup_int   = np.array([[180,210], #sup1
                           [190,205]]) #sup2
                  
                         #Dem1#Dem2#Dem3
cost_int_dem = np.array([[175,180,165],#int1
                         [235,130,145]]) #int2

                        #Dem1#Dem2#Dem3
cost_sup_dem = np.array([[99999999,99999999,99999999],#sup1
                         [275,99999999,99999999]]) #sup2



sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int) 
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem)
sup_dem_dict = p.makeDict([supply, demand], cost_sup_dem)


prob = p.LpProblem('Sandy_Co', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

sup_to_dem_route = [(s,d) for s in supply for d in demand]

sup_to_dem_vars = p.LpVariable.dicts("route", (supply,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([sup_to_dem_vars[s][d]*sup_dem_dict[s][d] for (s,d) in sup_to_dem_route])\
           
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) + p.lpSum([sup_to_dem_vars[s][d] for d in demand])  <= supplyconstraint[s]

for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) + p.lpSum([sup_to_dem_vars[s][d] for s in supply]) == demandconstraint[d] #demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 10000000000*int_open[i] <= 0 #linking constraint between flow and Intermediate open 
    
    
#prob += p.lpSum([int_open[i] for i in intermediate]) <= 2    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 1 #for part 3
prob += p.lpSum([int_open[i] for i in intermediate]) >= 0 

prob += int_open['DC A'] == 1 #For part 3, force DC A to open


prob.solve()

print('minimum cost excl. inv cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

#INV COST, TAKE NOTE THE FORMULATION, SUM UP ALL FLOW IN A DC FIRST, THEN PUT INTO THE INV COST FUNCTION
invcost = inv_cost(sum([sup_to_int_vars[s]['DC A'].varValue for s in supply]),5) +\
            inv_cost(sum([sup_to_int_vars[s]['DC B'].varValue for s in supply]),5) 

print('minimum cost incl. inv cost: ', p.value(prob.objective)+invcost)


minimum cost excl. inv cost:  69425.0
Solution Status: Optimal
open_DC_A  =  1.0
open_DC_B  =  0.0
route_DC_A_Region1  =  0.0
route_DC_A_Region2  =  95.0
route_DC_A_Region3  =  80.0
route_DC_B_Region1  =  0.0
route_DC_B_Region2  =  0.0
route_DC_B_Region3  =  0.0
route_Plant1_DC_A  =  100.0
route_Plant1_DC_B  =  0.0
route_Plant1_Region1  =  0.0
route_Plant1_Region2  =  0.0
route_Plant1_Region3  =  0.0
route_Plant2_DC_A  =  75.0
route_Plant2_DC_B  =  0.0
route_Plant2_Region1  =  25.0
route_Plant2_Region2  =  0.0
route_Plant2_Region3  =  0.0
minimum cost incl. inv cost:  70183.85


<b>Part 4</b>

Located next to SandyCo's DC A is third party logistics provider's DC. The third party can provide the same services as performed by SandyCo at their own DC. Since much of the inventory costs at DC A is due to handling, SandyCo is thinking about using the third party to reduce inventory costs. Instead of 5dollars per ton and week, the inventory holding cost with the third party DC would by 2dollars per ton. On the other hand, SandyCo would have to pay 2dollars per ton going through the third party's DC. Assume Plant 2 cannot supply Region 1 directly.

With this third DC-option in the network - what is the cost-optimal combination of DCs in the network, considering transportation as well as inventory holding costs?

<b>Explanation</b>

Ok, this is to show that even though inventory costs are not normally used in these types of models -- for several reasons -- it is possible to estimate them and include them into the supply chain network model. However, as you see in this problem, even a very small problem becomes quite complicated pretty fast.

There are many things to consider in this setup, so an Excel file is provided below. Note that there are many other ways to model the problem in Excel, the spreadsheet shows one way, chosen for its relative clarity. The key points are the following:

- The decision variables: we need to include binary variables for each segment at each DC to calculate the total costs. A simple way to handle the rest of the model is to consider the segments as "separate" DCs (but with similar costs), and introduce decision variables for each such DC. That is, for every arc in the previous model we will have four arcs, one for each segment. We then introduce constraints to make sure that not more than one of the four arcs are used.

- The objective function: This needs to be modified to include not just transportation costs as before, but inventory costs. To avoid problems with the solver, the linear approximation should be entered in two components - the "constant" which is multiplied by the binary variable indicating if a segment is "on"; and the "slope" which is multiplied by the throughput. Both components are multiplied by the holding cost (make sure you use the right cost for the different facilities!). For the third party, there is also a variable cost based only on throughput.

- Constraints: We need supply, demand, conservation of flow (balance), and linking constraints as before. We also need to make sure that the binary variables are binary and that the sum of the binary variables is not more than 1 for each DC.

In [96]:
#IMPORTANT: CREATE INV FUNCTION OUTSIDE OF THE OBJECTIVE FUNCTION

def inv_cost(T, holdingcost):
    if T <= 10:
        return 2.06*T*holdingcost
    elif T > 10 and T <=25:
        return (7.97 + 1.26*T)*holdingcost
    elif T > 25 and T <=50:
        return (14.32 + T)*holdingcost
    else:
        return (29.27 + 0.7*T)*holdingcost

supply = ['Plant1','Plant2']
supplycap = np.array([100,125])

demand = ['Region1', 'Region2', 'Region3']
demandcap = [25,95,80]

intermediate = ['DC A','DC B', 'Third Party']
intermediatefixcost = [0,0,2]

supplyconstraint = p.makeDict([supply], supplycap)
demandconstraint = p.makeDict([demand], demandcap)
intfixcostdict = p.makeDict([intermediate], intermediatefixcost)

                          #int1 #int2 #int3
cost_sup_int   = np.array([[180,210, 180], #sup1
                           [190,205, 190]]) #sup2
                  
                         #Dem1#Dem2#Dem3
cost_int_dem = np.array([[175,180,165],#int1
                         [235,130,145], #int2
                         [175,180,165]]) #int3

                        #Dem1#Dem2#Dem3
cost_sup_dem = np.array([[99999999,99999999,99999999],#sup1
                         [99999999,99999999,99999999]]) #sup2



sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int) 
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem)
sup_dem_dict = p.makeDict([supply, demand], cost_sup_dem)


prob = p.LpProblem('Sandy_Co', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

sup_to_dem_route = [(s,d) for s in supply for d in demand]

sup_to_dem_vars = p.LpVariable.dicts("route", (supply,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([sup_to_dem_vars[s][d]*sup_dem_dict[s][d] for (s,d) in sup_to_dem_route])\
            + p.lpSum([int_to_dem_vars[i][d]*intfixcostdict[i] for (i,d) in int_to_dem_route])\

           
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) + p.lpSum([sup_to_dem_vars[s][d] for d in demand])  <= supplyconstraint[s]

for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) + p.lpSum([sup_to_dem_vars[s][d] for s in supply]) == demandconstraint[d] #demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 10000000000*int_open[i] <= 0 #linking constraint between flow and Intermediate open 
    
        
prob += p.lpSum([int_open[i] for i in intermediate]) <= 3
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1 


prob.solve()

print('minimum cost excl. inv cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

#INV COST, TAKE NOTE THE FORMULATION, SUM UP ALL FLOW IN A DC FIRST, THEN PUT INTO THE INV COST FUNCTION
invcost = inv_cost(sum([sup_to_int_vars[s]['DC A'].varValue for s in supply]),5) +\
            inv_cost(sum([sup_to_int_vars[s]['DC B'].varValue for s in supply]),5) +\
                inv_cost(sum([sup_to_int_vars[s]['Third Party'].varValue for s in supply]),2)

print('minimum cost incl. inv cost: ', p.value(prob.objective)+invcost)


TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

<h3>MIDTERM Question 1</h3>

GoOnline is a Brazilian e-commerce platform that connects sellers and buyers over the internet. GoOnline strategically splits the country into 4 markets (final customer region):

-Southeast (M1),

-South (M2),

-Northeast (M3),

-and Others (M4).

The company also opened 3 Distribution Centers (DCs):

-São Paulo (DC1),

-Santa Catarina (DC2),

-and Bahia (DC3).

The distances (in km) between the DCs and the markets are presented in Table 1, and the monthly outbound capacity (in units) at each DC to the markets is presented in Table 2.

TABLE 1: Distance between DCs and markets (in km):

<img src = "midterm1.png">

The monthly capacity is the maximum quantity of units that each DC can handle in inventory every month

The logistics manager of GoOnline, tells you to consider that the transportation cost is $1 per unit per km for now just for your initial analysis.

Question 2

Now that you've built the model, Frank Gonzalez, the finance coordinator of GoOnline, informs you that the outbound transportation cost per unit per km is actually $1.2. And Ruby Mitchell, the logistics manager, finally handles you the monthly demand (in units) of a specific product under analysis, of each market.

TABLE 3: Monthly demand (in units):

Monthly Demand

Southeast (M1)	16212

South (M2)	1463

Northeast (M3)	1707

Others (M4)	1219

Use your model to decide how many units GoOnline should send from each DC to each market every month to fulfill the demand at a minimum transportation cost.

What is the optimal monthly transportation cost in this network?

In [41]:
#supply = ['Supply1', 'Supply2']
#supplycap = [100, 125]
demand = ['Southeast(M1)', 'South (M2)', 'Northeast (M3)','Others (M4)']
demandcap = [16212,1463,1707,1219]

intermediate = ['Sao Paulo (DC1)','Santa Catarina (DC2)','Bahia (DC3)']
intermediatecap = [15000, 4000, 7000] 

                   #int1      #int2
#cost_sup_int   = np.array([[180,       210], #sup1
#                  [190,       205]]) #sup2


                         #Dem1#Dem2#Dem3#Dem4
cost_int_dem = np.array([[150,750,2700,2800],#int1
                         [750,200,3400,3500], #int2
                         [2000,2850,800,3000]]) #int3
      
#supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))
intermediateconstraint = dict(zip(intermediate,intermediatecap)) #added the constraint for the DCs (Intermediate)

#sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int)
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem * 1.2)

prob = p.LpProblem('Go Online', p.LpMinimize)

#sup_to_int_route = [(s,i) for s in supply for i in intermediate]

#sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')


#prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route]) \
#            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\

prob += p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
                   
#for s in supply:
#   prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s]

for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d]

#for i in intermediate:
#    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand])
    
for i in intermediate: #constraint for DCs (Intermediate) capacity
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) <= intermediateconstraint[i]
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

minimum cost:  10169040.0
Solution Status: Optimal
route_Bahia_(DC3)_Northeast_(M3)  =  1707.0
route_Bahia_(DC3)_Others_(M4)  =  1219.0
route_Bahia_(DC3)_South_(M2)  =  0.0
route_Bahia_(DC3)_Southeast(M1)  =  0.0
route_Santa_Catarina_(DC2)_Northeast_(M3)  =  0.0
route_Santa_Catarina_(DC2)_Others_(M4)  =  0.0
route_Santa_Catarina_(DC2)_South_(M2)  =  1463.0
route_Santa_Catarina_(DC2)_Southeast(M1)  =  1212.0
route_Sao_Paulo_(DC1)_Northeast_(M3)  =  0.0
route_Sao_Paulo_(DC1)_Others_(M4)  =  0.0
route_Sao_Paulo_(DC1)_South_(M2)  =  0.0
route_Sao_Paulo_(DC1)_Southeast(M1)  =  15000.0


<b>Question 3</b>

You realize that your network is missing the inbound costs and details about the product flowing into DCs before reaching the markets.

After gathering some additional data with your team you observe that Capricious Co. (S1) and Guaranteed Inc. (S2) are two companies who sell this product using the GoOnline platform. Although both companies compete in the market, they chose to sell throughout GoOnline as a form of horizontal collaboration, in order to lower the costs and gain a competitive advantage.

The product commercialized by both companies must be delivered to one of GoOnline's DCs before moving to the markets, to fulfill the previously stated demand. The data you got, includes:

TABLE 4: Distances from the Sellers to the DCs (in km):

<img src = 'midterm2.png'>

Ruby tells you the inbound transportation cost (from the sellers to DCs) is $0.5 per unit per km. She adds that due to some transportation asset restrictions, there is some new inbound transportation constraints from the Sellers to the DCs that you must consider in your model.

Monthly Inbound Transportation capacity from the Sellers (in units) to some DCs:

You cannot transport more than 11000 units per month from Capricious Co. to São Paulo (DC1).

You cannot transport more than 3000 units per month from Garanteed Inc. to Santa Catarina (DC2).

In order to keep low handling costs, GoOnline has an internal rule that every unit that moves from the sellers into the DCs must move out from the DC to the market in the same month. Assuming that the outbound transportation per unit per km (from Question 2), the demand in the final markets, and DC's capacity remain unchanged, what is the optimal monthly transportation cost in this network?

In [44]:
supply = ['Capricious Co. (S1)', 'Guaranteed Inc. (S2)']
                        #int1 #int2 #int3
supplycap   = np.array([[11000,999999999,999999999], #sup1
                           [999999999,3000,999999999]]) #sup2

demand = ['Southeast(M1)', 'South (M2)', 'Northeast (M3)','Others (M4)']
demandcap = [16212,1463,1707,1219]

intermediate = ['Sao Paulo (DC1)','Santa Catarina (DC2)','Bahia (DC3)']
intermediatecap = [15000, 4000, 7000] 

                           #int1 #int2 #int3
cost_sup_int   = np.array([[100,750,1900], #sup1
                           [500,300,2500]]) #sup2


                         #Dem1#Dem2#Dem3
cost_int_dem = np.array([[150,750,2700,2800],#int1
                         [750,200,3400,3500], #int2
                         [2000,2850,800,3000]]) #int3
      
supplyconstraint = p.makeDict([supply, intermediate], supplycap)
demandconstraint = dict(zip(demand,demandcap))
intermediateconstraint = dict(zip(intermediate,intermediatecap)) #added the constraint for the DCs (Intermediate)

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*0.5)
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem * 1.2)

prob = p.LpProblem('Go Online', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route_inbound", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route_outbound", (intermediate,demand), lowBound = 0, cat = 'Integer')


prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route]) \
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
                   
for s in supply:
    for i in intermediate:
        prob += sup_to_int_vars[s][i] <= supplyconstraint[s][i]

for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d]

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand])
    
for i in intermediate: #constraint for DCs (Intermediate) capacity
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) <= intermediateconstraint[i]
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

minimum cost:  14711060.0
Solution Status: Optimal
route_inbound_Capricious_Co._(S1)_Bahia_(DC3)  =  1707.0
route_inbound_Capricious_Co._(S1)_Santa_Catarina_(DC2)  =  894.0
route_inbound_Capricious_Co._(S1)_Sao_Paulo_(DC1)  =  11000.0
route_inbound_Guaranteed_Inc._(S2)_Bahia_(DC3)  =  0.0
route_inbound_Guaranteed_Inc._(S2)_Santa_Catarina_(DC2)  =  3000.0
route_inbound_Guaranteed_Inc._(S2)_Sao_Paulo_(DC1)  =  4000.0
route_outbound_Bahia_(DC3)_Northeast_(M3)  =  1707.0
route_outbound_Bahia_(DC3)_Others_(M4)  =  0.0
route_outbound_Bahia_(DC3)_South_(M2)  =  0.0
route_outbound_Bahia_(DC3)_Southeast(M1)  =  0.0
route_outbound_Santa_Catarina_(DC2)_Northeast_(M3)  =  0.0
route_outbound_Santa_Catarina_(DC2)_Others_(M4)  =  0.0
route_outbound_Santa_Catarina_(DC2)_South_(M2)  =  1463.0
route_outbound_Santa_Catarina_(DC2)_Southeast(M1)  =  2431.0
route_outbound_Sao_Paulo_(DC1)_Northeast_(M3)  =  0.0
route_outbound_Sao_Paulo_(DC1)_Others_(M4)  =  1219.0
route_outbound_Sao_Paulo_(DC1)_South_(M2)  =

<b>Question 4</b>

GoOnline is studying the project to change the original 3 DCs to only 1 very large automated DC in 5 years from now. They now must decide where to locate this facility.

This new modern DC should be located in a point that minimizes the transportation costs to the final markets. GoOnline would like to explore two different methods to decide the location of this new facility: Center of Gravity and Weber method.

The team provides you with a new set of data of demand for your 4 markets, as it's now considering the next 5 years expected variations. This analysis is not considering the inbound costs from the sellers to the DCs, but only analyzing the scenario between DCs and Markets.

Table 5: X and Y coordinates in custom grid for each market, considering future demand.

<img src = "midterm3.png">

Assuming an Euclidean surface, and including only the markets (and their demand) in the calculation, use the center of gravity method to find the X and Y coordinates for the new distribution center.

What is the X coordinate?

What is the Y coordinate?

<b>Question 5</b>

Now, find the coordinates for the new DC facility using the Weber method. Use the coordinates and demand from Table 5 and remember to include only the markets in the calculation.

What is the X coordinate?

What is the Y coordinate?

In [38]:
goonline = weber_problem(['Southeast(M1)','South(M2)','Northeast(M3)','Others(M4)'],[18280,1649,1924,1374],[19,20.45,6.52,1.21],[37.52,39.69,28.1,39.05])

goonline.find_cog()
goonline.find_weber()


Center of Gravity x coordinate:  17.01679381753993
Center of Gravity y coordinate:  36.98426443363327
Center of Gravity avg. weighted dist:  4.009231385470313
weber x coordinate:  19.000000000000018
weber y coordinate:  37.520000000001474
weber avg. weighted dist:  2.536753221517014


(19.000000000000018, 37.520000000001474, 2.536753221517014)

<h3>Midterm Question 3: 1</h3>

Arioca Inc. is a 3D Printers manufacturer with a plant in Lyon, France.

You are hired as the Director of Network Optimization for the European region, and are now in charge of defining the flows for the new model T-Q21 which utilizes titanium as the printing material. Pieces printed with this material are mainly used for the aerospace and medical industries due to the titanium strength and resistance to corrosion.

Arioca’s warehouses are located in the cities of Paris, Marseilles, Nantes and Bordeaux, from where the new models are distributed to the retailers stores. Your regional manager Amanda Lee gathered the following data about the retailers demand of your product:

<img src = 'midterm4.png'>

Your assistant Mr. Bennett after a conference call with the local warehouse managers and the logistics team, got details on capacity and costs. The information he prepared shows:

<img src = 'midterm5.png'>

After a few phone calls, your carrier confirms that:

your current cost per kilometer from the plant to the warehouses is 2.4 Euros per 3D printer.

your current cost per kilometer from warehouses to the retailers is 5.6 Euros per 3D printer

You and your team were getting ready to begin analyzing the provided information, when you notice you are missing the distances from Warehouses to Retailers. A few hours later, your team gets together again with the following information:

Distance in Km from Warehouses to Retailers

<img src = 'midterm6.png'>

Considering the current flow of products from warehouses to retailers, what’s the current total monthly cost (including inbound and outbound costs)?. Please, remember you want to send enough products to EXACTLY satisfy the customer demand.

In [46]:
supply = ['Lyon Plant']

demand = ['1_Nice', '2_Montpellier', '3_Strasbourg','4_Lille', '5_Rennes', '6_Dijon','7_Limoges']
demandcap = np.array([135,105,120,95,45,62,25])

intermediate = ['1_Paris','2_Marseille','3_Nantes','4_Bordeaux']
#intermediate_fixed_cost = np.array([10000,7000,7000,5500])
intermediatecap = np.array([200,150,150,100])


                          #int1 #int2 #int3 #int4
cost_sup_int   = np.array([[465,314,684,550]]) #sup1
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7
cost_int_dem = np.array([[932,748,491,225,354,314,392], #int1
                         [199,170,801,999,1052,506,691], #int2
                         [1143,825,866,600,113,639,327], #int3
                         [803,485,945,800,466,649,221]]) #int4

                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7
forced_flow = np.array([[135,60,0,0,0,0,0], #int1
                         [0,40,110,0,0,0,0], #int2
                         [0,0,10,95,0,45,0], #int3
                         [0,5,0,0,45,17,25]]) #int4

#dist_within_limit = cost_int_dem <= 50 #remember to use less than or equal to sign

#dist_within_limit = dist_within_limit.astype(int) / sum(demandcap)

#dist_within_limit_dict = p.makeDict([intermediate, demand], dist_within_limit)


demandconstraint = dict(zip(demand,demandcap))
intermediateconstraint = dict(zip(intermediate,intermediatecap))

#intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost))

#int_dem_dist = p.makeDict([intermediate, demand], cost_int_dem/sum(demandcap)) #maximum avg. distance constraint
sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*2.4) 
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*5.6)

forced_flow_dict = p.makeDict([intermediate, demand], forced_flow)


prob = p.LpProblem('Printer', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("inbound_route", (supply,intermediate), lowBound = 0, cat = 'Integer')

#int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("outbound_route", (intermediate,demand), lowBound = 0, cat = 'Integer')

#prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
#            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
#            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\

for i in intermediate:
    for d in demand:
        prob += int_to_dem_vars[i][d] == forced_flow_dict[i][d]
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d]

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
    #prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - sum(demandcap)*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) <= intermediateconstraint[i]
    

#prob += p.lpSum([int_to_dem_vars[i][d]*dist_within_limit_dict[i][d] for (i,d) in int_to_dem_route]) >= 0.5 #pct of demand within certain limit constraint
 
#prob += p.lpSum([int_to_dem_vars[i][d]*int_dem_dist[i][d] for (i,d) in int_to_dem_route]) <= 100  
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  2938216.8
Solution Status: Optimal
inbound_route_Lyon_Plant_1_Paris  =  195.0
inbound_route_Lyon_Plant_2_Marseille  =  150.0
inbound_route_Lyon_Plant_3_Nantes  =  150.0
inbound_route_Lyon_Plant_4_Bordeaux  =  92.0
outbound_route_1_Paris_1_Nice  =  135.0
outbound_route_1_Paris_2_Montpellier  =  60.0
outbound_route_1_Paris_3_Strasbourg  =  0.0
outbound_route_1_Paris_4_Lille  =  0.0
outbound_route_1_Paris_5_Rennes  =  0.0
outbound_route_1_Paris_6_Dijon  =  0.0
outbound_route_1_Paris_7_Limoges  =  0.0
outbound_route_2_Marseille_1_Nice  =  0.0
outbound_route_2_Marseille_2_Montpellier  =  40.0
outbound_route_2_Marseille_3_Strasbourg  =  110.0
outbound_route_2_Marseille_4_Lille  =  0.0
outbound_route_2_Marseille_5_Rennes  =  0.0
outbound_route_2_Marseille_6_Dijon  =  0.0
outbound_route_2_Marseille_7_Limoges  =  0.0
outbound_route_3_Nantes_1_Nice  =  0.0
outbound_route_3_Nantes_2_Montpellier  =  0.0
outbound_route_3_Nantes_3_Strasbourg  =  10.0
outbound_route_3_Nantes_4_Lille  =

<b>Question 2</b>

You are preparing a proposal about a new network design considering the flow of this product model. Aiming to convince the CFO, you need to define the new minimum monthly total inbound+outbound cost for the optimized network design. What would that cost be?

In [47]:
supply = ['Lyon Plant']

demand = ['1_Nice', '2_Montpellier', '3_Strasbourg','4_Lille', '5_Rennes', '6_Dijon','7_Limoges']
demandcap = np.array([135,105,120,95,45,62,25])

intermediate = ['1_Paris','2_Marseille','3_Nantes','4_Bordeaux']
#intermediate_fixed_cost = np.array([10000,7000,7000,5500])
intermediatecap = np.array([200,150,150,100])


                          #int1 #int2 #int3 #int4
cost_sup_int   = np.array([[465,314,684,550]]) #sup1
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7
cost_int_dem = np.array([[932,748,491,225,354,314,392], #int1
                         [199,170,801,999,1052,506,691], #int2
                         [1143,825,866,600,113,639,327], #int3
                         [803,485,945,800,466,649,221]]) #int4

#dist_within_limit = cost_int_dem <= 50 #remember to use less than or equal to sign

#dist_within_limit = dist_within_limit.astype(int) / sum(demandcap)

#dist_within_limit_dict = p.makeDict([intermediate, demand], dist_within_limit)


demandconstraint = dict(zip(demand,demandcap))
intermediateconstraint = dict(zip(intermediate,intermediatecap))

#intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost))

#int_dem_dist = p.makeDict([intermediate, demand], cost_int_dem/sum(demandcap)) #maximum avg. distance constraint
sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*2.4) 
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*5.6)

prob = p.LpProblem('Printer', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("inbound_route", (supply,intermediate), lowBound = 0, cat = 'Integer')

#int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("outbound_route", (intermediate,demand), lowBound = 0, cat = 'Integer')

#prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
#            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
#            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d] #RW demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
    #prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - sum(demandcap)*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) <= intermediateconstraint[i]
    

#prob += p.lpSum([int_to_dem_vars[i][d]*dist_within_limit_dict[i][d] for (i,d) in int_to_dem_route]) >= 0.5 #pct of demand within certain limit constraint
 
#prob += p.lpSum([int_to_dem_vars[i][d]*int_dem_dist[i][d] for (i,d) in int_to_dem_route]) <= 100  
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  1873636.0
Solution Status: Optimal
inbound_route_Lyon_Plant_1_Paris  =  200.0
inbound_route_Lyon_Plant_2_Marseille  =  150.0
inbound_route_Lyon_Plant_3_Nantes  =  137.0
inbound_route_Lyon_Plant_4_Bordeaux  =  100.0
outbound_route_1_Paris_1_Nice  =  0.0
outbound_route_1_Paris_2_Montpellier  =  0.0
outbound_route_1_Paris_3_Strasbourg  =  105.0
outbound_route_1_Paris_4_Lille  =  95.0
outbound_route_1_Paris_5_Rennes  =  0.0
outbound_route_1_Paris_6_Dijon  =  0.0
outbound_route_1_Paris_7_Limoges  =  0.0
outbound_route_2_Marseille_1_Nice  =  135.0
outbound_route_2_Marseille_2_Montpellier  =  15.0
outbound_route_2_Marseille_3_Strasbourg  =  0.0
outbound_route_2_Marseille_4_Lille  =  0.0
outbound_route_2_Marseille_5_Rennes  =  0.0
outbound_route_2_Marseille_6_Dijon  =  0.0
outbound_route_2_Marseille_7_Limoges  =  0.0
outbound_route_3_Nantes_1_Nice  =  0.0
outbound_route_3_Nantes_2_Montpellier  =  0.0
outbound_route_3_Nantes_3_Strasbourg  =  15.0
outbound_route_3_Nantes_4_Lille  

<b>Question 3</b>

Reviewing old information presented to Ms. Hall (the CFO) to compare strategies and get ready for the meeting, you notice that you missed adding the warehouse’s fixed costs to the total cost equation.

Warehouse	Fixed Cost (Euros/month)

Paris	10000

Marseilles	7000

Nantes	7000

Bordeaux	5500

Once you consider these new data points, what’s the difference between this new total cost and the previous answer?.

In [49]:
supply = ['Lyon Plant']

demand = ['1_Nice', '2_Montpellier', '3_Strasbourg','4_Lille', '5_Rennes', '6_Dijon','7_Limoges']
demandcap = np.array([135,105,120,95,45,62,25])

intermediate = ['1_Paris','2_Marseille','3_Nantes','4_Bordeaux']
intermediate_fixed_cost = np.array([10000,7000,7000,5500])
intermediatecap = np.array([200,150,150,100])


                          #int1 #int2 #int3 #int4
cost_sup_int   = np.array([[465,314,684,550]]) #sup1
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7
cost_int_dem = np.array([[932,748,491,225,354,314,392], #int1
                         [199,170,801,999,1052,506,691], #int2
                         [1143,825,866,600,113,639,327], #int3
                         [803,485,945,800,466,649,221]]) #int4

#dist_within_limit = cost_int_dem <= 50 #remember to use less than or equal to sign

#dist_within_limit = dist_within_limit.astype(int) / sum(demandcap)

#dist_within_limit_dict = p.makeDict([intermediate, demand], dist_within_limit)


demandconstraint = dict(zip(demand,demandcap))
intermediateconstraint = dict(zip(intermediate,intermediatecap))

intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost))

#int_dem_dist = p.makeDict([intermediate, demand], cost_int_dem/sum(demandcap)) #maximum avg. distance constraint
sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*2.4) 
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*5.6)


prob = p.LpProblem('Printer', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("inbound_route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("outbound_route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d] #RW demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - sum(demandcap)*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) <= intermediateconstraint[i]
    
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  1903136.0
Solution Status: Optimal
inbound_route_Lyon_Plant_1_Paris  =  200.0
inbound_route_Lyon_Plant_2_Marseille  =  150.0
inbound_route_Lyon_Plant_3_Nantes  =  137.0
inbound_route_Lyon_Plant_4_Bordeaux  =  100.0
open_1_Paris  =  1.0
open_2_Marseille  =  1.0
open_3_Nantes  =  1.0
open_4_Bordeaux  =  1.0
outbound_route_1_Paris_1_Nice  =  0.0
outbound_route_1_Paris_2_Montpellier  =  0.0
outbound_route_1_Paris_3_Strasbourg  =  120.0
outbound_route_1_Paris_4_Lille  =  80.0
outbound_route_1_Paris_5_Rennes  =  0.0
outbound_route_1_Paris_6_Dijon  =  0.0
outbound_route_1_Paris_7_Limoges  =  0.0
outbound_route_2_Marseille_1_Nice  =  135.0
outbound_route_2_Marseille_2_Montpellier  =  15.0
outbound_route_2_Marseille_3_Strasbourg  =  0.0
outbound_route_2_Marseille_4_Lille  =  0.0
outbound_route_2_Marseille_5_Rennes  =  0.0
outbound_route_2_Marseille_6_Dijon  =  0.0
outbound_route_2_Marseille_7_Limoges  =  0.0
outbound_route_3_Nantes_1_Nice  =  0.0
outbound_route_3_Nantes_2_Montpel

<b>Question 4</b>

Arioca Inc. defines its service level considering how long it takes for the product to arrive from the warehouses to its customers (the retailers).

According to their experience, longer distances usually cause the latest deliveries.

Considering that, and aiming to assure the expected service level, the team wants to explore a new scenario to present to the CFO. This new scenario considers all the previously gathered information but limits the MAXIMUM DISTANCE from Warehouses to Retailers to 600 km (Note that this is not an average distance but that warehouses should be located at 600 km or less from the retailers they serve. If they are further away, they shouldn't be used!).

Is this a feasible scenario?

In [54]:
supply = ['Lyon Plant']

demand = ['1_Nice', '2_Montpellier', '3_Strasbourg','4_Lille', '5_Rennes', '6_Dijon','7_Limoges']
demandcap = np.array([135,105,120,95,45,62,25])

intermediate = ['1_Paris','2_Marseille','3_Nantes','4_Bordeaux']
intermediate_fixed_cost = np.array([10000,7000,7000,5500])
intermediatecap = np.array([200,150,150,100])


                          #int1 #int2 #int3 #int4
cost_sup_int   = np.array([[465,314,684,550]]) #sup1
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7
cost_int_dem = np.array([[932,748,491,225,354,314,392], #int1
                         [199,170,801,999,1052,506,691], #int2
                         [1143,825,866,600,113,639,327], #int3
                         [803,485,945,800,466,649,221]]) #int4


dist_within_limit = cost_int_dem <= 600 #remember to use less than or equal to sign

dist_within_limit = dist_within_limit.astype(int)

dist_within_limit_dict = p.makeDict([intermediate, demand], dist_within_limit)


demandconstraint = dict(zip(demand,demandcap))
intermediateconstraint = dict(zip(intermediate,intermediatecap))

intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost))

#int_dem_dist = p.makeDict([intermediate, demand], cost_int_dem/sum(demandcap)) #maximum avg. distance constraint
sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*2.4) 
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*5.6)


prob = p.LpProblem('Printer', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("inbound_route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("outbound_route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d] #RW demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - sum(demandcap)*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) <= intermediateconstraint[i]
    
for i in intermediate:
    for d in demand:
        prob += int_to_dem_vars[i][d] - sum(demandcap)*dist_within_limit_dict[i][d] <= 0 
    

#prob += p.lpSum([int_to_dem_vars[i][d]*dist_within_limit_dict[i][d] for (i,d) in int_to_dem_route]) >= 0.5 #pct of demand within certain limit constraint
 
#prob += p.lpSum([int_to_dem_vars[i][d]*int_dem_dist[i][d] for (i,d) in int_to_dem_route]) <= 100  
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)
    

#for i in intermediate:
#    print([int_to_dem_vars[i][d].varValue for d in demand])
#    print('\n')
    
#for i in intermediate:
#    print([dist_within_limit_dict[i][d] for d in demand])
#    print('\n')


minimum cost:  1920496.0
Solution Status: Optimal
inbound_route_Lyon_Plant_1_Paris  =  200.0
inbound_route_Lyon_Plant_2_Marseille  =  150.0
inbound_route_Lyon_Plant_3_Nantes  =  137.0
inbound_route_Lyon_Plant_4_Bordeaux  =  100.0
open_1_Paris  =  1.0
open_2_Marseille  =  1.0
open_3_Nantes  =  1.0
open_4_Bordeaux  =  1.0
outbound_route_1_Paris_1_Nice  =  0.0
outbound_route_1_Paris_2_Montpellier  =  0.0
outbound_route_1_Paris_3_Strasbourg  =  120.0
outbound_route_1_Paris_4_Lille  =  18.0
outbound_route_1_Paris_5_Rennes  =  0.0
outbound_route_1_Paris_6_Dijon  =  62.0
outbound_route_1_Paris_7_Limoges  =  0.0
outbound_route_2_Marseille_1_Nice  =  135.0
outbound_route_2_Marseille_2_Montpellier  =  15.0
outbound_route_2_Marseille_3_Strasbourg  =  0.0
outbound_route_2_Marseille_4_Lille  =  0.0
outbound_route_2_Marseille_5_Rennes  =  0.0
outbound_route_2_Marseille_6_Dijon  =  0.0
outbound_route_2_Marseille_7_Limoges  =  0.0
outbound_route_3_Nantes_1_Nice  =  0.0
outbound_route_3_Nantes_2_Montpe

<h3>MicroMasters Practice Problem 2: LogCo</h3>
    
LogCo is a logistics service provider. They just signed a contract with a large manufacturer, making them the exclusive provider of transport services from three of the manufacturer's plants to its six regional warehouses. LogCo has also been contracted to do the logistics planning for this network.

The six regional warehouses face the following demand in number of truckloads per week:

Providence: 4

Philadelphia: 15

Columbus: 5

Knoxville: 12

Jacksonville: 3

Richmond: 8

Since transportation costs are a large part of total costs for the client, the three plants (located in Cincinnati, Atlanta, and Detroit) can adjust production planning to fit the transport network. They do, however, have a capacity limit of producing enough goods to fill 20 truckloads per week.

LogCo owns and operates two large terminals, one in Scranton, PA, and one in Charlotte, NC.

The transportation costs are given in the table below 

<img src="w2mmproblem2.png">

<b>Part 1</b>

LogCo is not sure whether it would be optimal to route the traffic through the terminals, or have all shipments going straight from a plant to a warehouse.

In choosing between using both terminals or using none, what should LogCo do to minimize costs?

In [82]:
supply = ['Cincinnati', 'Atlanta', 'Detroit']
supplycap = np.array([20,20,20])

demand = ['Providence','Philadelphia','Columbus','Knoxville','Jacksonville','Richmond']
demandcap = np.array([4,15,5,12,3,8])

intermediate = ['Scranton','Charlotte']

                          #int1 #int2
cost_sup_int   = np.array([[83,72],
                           [122,35],
                           [75,95]]) #supn
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6....#Dem20
cost_int_dem = np.array([[44,21,73,94,143,61],
                         [124,84,64,35,52,41]]) #intn

cost_sup_dem = np.array([[125,85,15,34,113,76],
                         [163,114,82,31,45,75],
                         [106,85,32,75,152,93]])

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))


sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*1)
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*1)
sup_dem_dict = p.makeDict([supply, demand], cost_sup_dem*1)

prob = p.LpProblem('LogCo', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("supply_to_intermediate_route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("intermediate_to_demand_route", (intermediate,demand), lowBound = 0, cat = 'Integer')

sup_to_dem_route = [(s,d) for s in supply for d in demand]

sup_to_dem_vars = p.LpVariable.dicts("direct_route", (supply,demand), lowBound = 0, cat = 'Integer')

direct_route_open = p.LpVariable('use_direct_route', lowBound = 0, cat = 'Binary')

intermediate_route_open = p.LpVariable('use_intermediate_route', lowBound = 0, cat = 'Binary')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([sup_to_dem_vars[s][d]*sup_dem_dict[s][d] for (s,d) in sup_to_dem_route])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) +\
            p.lpSum([sup_to_dem_vars[s][d] for d in demand]) <= supplyconstraint[s] #plant cap constraint
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) +\
            p.lpSum([sup_to_dem_vars[s][d] for s in supply]) == demandconstraint[d] #demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint

#linking constraint if direct route flow exists, then direct route is open
prob += p.lpSum([sup_to_dem_vars[s][d] for s in supply for d in demand]) - sum(demandcap)*direct_route_open <= 0
#linking constraint if intermediat route flow exists, then intermediate route is open    
prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate for d in demand]) - sum(demandcap)*intermediate_route_open <= 0
#constraint: there could be only 1 type of flow
prob += direct_route_open + intermediate_route_open <= 1

    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

minimum cost:  2884.0
Solution Status: Optimal
direct_route_Atlanta_Columbus  =  0.0
direct_route_Atlanta_Jacksonville  =  3.0
direct_route_Atlanta_Knoxville  =  12.0
direct_route_Atlanta_Philadelphia  =  0.0
direct_route_Atlanta_Providence  =  0.0
direct_route_Atlanta_Richmond  =  5.0
direct_route_Cincinnati_Columbus  =  5.0
direct_route_Cincinnati_Jacksonville  =  0.0
direct_route_Cincinnati_Knoxville  =  0.0
direct_route_Cincinnati_Philadelphia  =  12.0
direct_route_Cincinnati_Providence  =  0.0
direct_route_Cincinnati_Richmond  =  3.0
direct_route_Detroit_Columbus  =  0.0
direct_route_Detroit_Jacksonville  =  0.0
direct_route_Detroit_Knoxville  =  0.0
direct_route_Detroit_Philadelphia  =  3.0
direct_route_Detroit_Providence  =  4.0
direct_route_Detroit_Richmond  =  0.0
intermediate_to_demand_route_Charlotte_Columbus  =  0.0
intermediate_to_demand_route_Charlotte_Jacksonville  =  0.0
intermediate_to_demand_route_Charlotte_Knoxville  =  0.0
intermediate_to_demand_route_Charlotte_Phil

<h3>MicroMasters Practice Problem 4: Fredex</h3>
    
After finding that some of the large U.S. airlines have spare cargo capacity on certain flights you see a business opportunity. With great confidence in your recently acquired supply chain skills, you decide to launch a small express logistics company to deliver packages and parcels from business to business. 

The basic business idea is to use the capacity to provide express parcel logistics from cities on the east coast to cities on the west coast -- at a lower price than any of the larger competitors!

After weeks of intense negotiations with the airlines, you manage to secure capacity on the following routes (presented with the amount of capacity and the price you pay for it):

Boston (BOS) - Minneapolis (MSP): 1,100 kgs per day, $1 per kg

BOS - Dallas (DFW): 1,000 kgs per day, $0.8 per kg

New York (EWR) - MSP: 1,200 kgs per day, $1.1 per kg

EWR - DFW: 800 kgs per day, $0.9 per kg

Atlanta (ATL) - MSP: 700 kgs per day, $1 per kg

ATL - DFW: 600 kgs per day, $0.7 per kg

MSP - Seattle (SEA): 1,300 kgs per day, $0.5 per kg

MSP - Los Angeles (LAX): 900 kgs per day, $0.6 per kg

DFW - SEA: 1,200 kgs per day, $0.8 per kg

DFW - LAX: 900 kgs per day, $0.9 per kg

You have contracted with local delivery companies to do the handling and the last mile deliveries at a very low cost. For all practical reasons, we assume the cost of the last mile delivery is 0.

In your earlier efforts to understand the business case of your venture, you conducted an extensive survey of the market. Based on the survey, you now estimate that the demand in kgs per day from the three east coast cities to the two west coast cities is as follows.

<img src = "w2mmproblem4.png">

<b>Part 1</b>

Suppose you want to minimize costs for your network.

In the optimal solution, how large share of SEA-bound shipments should go through DFW? Answer in fraction with one decimal, e.g. 0.4.

In [33]:
types = ['product_for_SEA', 'product_for_LAX']
supply = ['BOS', 'EWR','ATL']

                      #sup1 #sup2 #sup3
supplycap = np.array([[700, 700, 300], #type1
                      [700, 900, 200]])#type2

demand = ['SEA', 'LAX']

                      #dem1 #dem2
demandcap = np.array([[1700, 0], #type1
                      [0, 1800]])#type2


intermediate = ['MSP','DFW']

                          #int1 int2
cost_sup_int = np.array([[1,  0.8],#sup1
                           [1.1,0.9],#sup2
                           [1,  0.7]])#sup3

#broadcast the cost to be the same for either product types
cost_sup_int = np.broadcast_to(cost_sup_int,(len(types),)+cost_sup_int.shape)

#capacity does not need to be broadcasted as it is total capacity regardless of product types
                        #int1 int2
cap_sup_int = np.array([[1100,1000],#sup1
                          [1200, 800],#sup2
                          [700,  600]])#sup3
                  
                         #dem1 dem2
cost_int_dem = np.array([[0.5, 0.6],#int1
                         [0.8, 0.9]])#int2

#broadcast the cost to be the same for either product types                               
cost_int_dem = np.broadcast_to(cost_int_dem,(len(types),)+cost_int_dem.shape)

#capacity does not need to be broadcasted as it is total capacity regardless of product types
                        #dem1 dem2
cap_int_dem = np.array([[1300, 900],#int1
                        [1200, 900]])#int2

supplyconstraint = p.makeDict([types,supply], supplycap)
demandconstraint = p.makeDict([types, demand], demandcap)
                               
supintcapdict = p.makeDict([supply, intermediate], cap_sup_int)
intdemcapdict = p.makeDict([intermediate, demand], cap_int_dem)
                               
sup_int_dict = p.makeDict([types,supply, intermediate], cost_sup_int*1)
int_dem_dict = p.makeDict([types,intermediate, demand], cost_int_dem*1) 

prob = p.LpProblem('East_to_West', p.LpMinimize)

sup_to_int_route = [(t,s,i) for t in types for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route_to_int", (types, supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(t,i,d) for t in types for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route_from_int", (types, intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[t][s][i]*sup_int_dict[t][s][i] for (t,s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[t][i][d]*int_dem_dict[t][i][d] for (t,i,d) in int_to_dem_route])

for t in types:
    for s in supply:
        prob += p.lpSum([sup_to_int_vars[t][s][i] for i in intermediate]) == supplyconstraint[t][s] 
        
    for d in demand:
        prob += p.lpSum([int_to_dem_vars[t][i][d] for i in intermediate]) == demandconstraint[t][d]

    for i in intermediate:
        prob += p.lpSum([sup_to_int_vars[t][s][i] for s in supply]) == p.lpSum([int_to_dem_vars[t][i][d] for d in demand]) #conservation of flow constraint

#flow to intermediate capacity constraint
for s in supply:
    for i in intermediate:     
        prob += p.lpSum([sup_to_int_vars[t][s][i] for t in types]) <= supintcapdict[s][i]
                               
#flow from intermediate capacity constraint
for i in intermediate:
    for d in demand:     
        prob += p.lpSum([int_to_dem_vars[t][i][d] for t in types]) <= intdemcapdict[i][d]
        
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  5670.0
Solution Status: Optimal
route_from_int_product_for_LAX_DFW_LAX  =  900.0
route_from_int_product_for_LAX_DFW_SEA  =  0.0
route_from_int_product_for_LAX_MSP_LAX  =  900.0
route_from_int_product_for_LAX_MSP_SEA  =  0.0
route_from_int_product_for_SEA_DFW_LAX  =  0.0
route_from_int_product_for_SEA_DFW_SEA  =  400.0
route_from_int_product_for_SEA_MSP_LAX  =  0.0
route_from_int_product_for_SEA_MSP_SEA  =  1300.0
route_to_int_product_for_LAX_ATL_DFW  =  200.0
route_to_int_product_for_LAX_ATL_MSP  =  0.0
route_to_int_product_for_LAX_BOS_DFW  =  200.0
route_to_int_product_for_LAX_BOS_MSP  =  500.0
route_to_int_product_for_LAX_EWR_DFW  =  500.0
route_to_int_product_for_LAX_EWR_MSP  =  400.0
route_to_int_product_for_SEA_ATL_DFW  =  300.0
route_to_int_product_for_SEA_ATL_MSP  =  0.0
route_to_int_product_for_SEA_BOS_DFW  =  100.0
route_to_int_product_for_SEA_BOS_MSP  =  600.0
route_to_int_product_for_SEA_EWR_DFW  =  0.0
route_to_int_product_for_SEA_EWR_MSP  =  700.0


<h2>Final Exam Problem 2</h2>

<b>Question 1</b>

Jean Hall is part of the supply chain team at FreshGo, a pure-play online grocer in the UK that offers deliveries to 80% of the zip codes in the country. Jean is an expert in network design, and she is in charge of determining the flows that will optimize the cost operation.

The current network has a hub-and-spoke structure, with a big central hub (Hub A) that serves four distribution centers (DCs). All customer orders are served from one of these four DCs. FreshGo has divided its market in 8 delivery areas. Orders in the same delivery area are all served from one single DC.

<img src='finalexam2.png'>
<img src='finalexam22.png'>

Considering that the transportation cost between Hub A and DCs is 0.027 pounds per box per mile and the transportation cost between DCs and delivery areas is 0.087 pounds per box per mile.

What is the optimal daily cost of operation (in pounds) for this network to satisfy all the expected demand?

In [32]:
supply = ['Hub_A']

demand = ['Area1', 'Area2', 'Area3','Area4', 'Area5', 'Area6','Area7','Area8']
demandcap = np.array([359,258,583,988,244,299,503,707])

intermediate = ['DC1','DC2','DC3','DC4']
#intermediate_fixed_cost = np.array([10000,7000,7000,5500])
intermediatecap = np.array([1250,1340,1060,895])


                          #int1 #int2 #int3 #int4
cost_sup_int   = np.array([[91,66,82,27]]) #sup1
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5...#DemN
cost_int_dem = np.array([[75,43,65,86,45,78,2,44],#int1...
                         [90,18,59,51,69,23,7,24],
                         [27,83,41,10,36,55,21,21],
                         [42,89,45,76,15,18,55,11]]) #intN

#dist_within_limit = cost_int_dem <= 50 #remember to use less than or equal to sign

#dist_within_limit = dist_within_limit.astype(int) / sum(demandcap)

#dist_within_limit_dict = p.makeDict([intermediate, demand], dist_within_limit)


demandconstraint = dict(zip(demand,demandcap))
intermediateconstraint = dict(zip(intermediate,intermediatecap))

#intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost))

#int_dem_dist = p.makeDict([intermediate, demand], cost_int_dem/sum(demandcap)) #maximum avg. distance constraint
sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*0.027) 
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*0.087)


prob = p.LpProblem('Fresh_Go', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("inbound_route", (supply,intermediate), lowBound = 0, cat = 'Integer')

#int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("outbound_route", (intermediate,demand), lowBound = 0, cat = 'Integer')

int_to_dem_open = p.LpVariable.dicts("outbound_route_open", (intermediate,demand), lowBound = 0, cat = 'Binary')

#prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
#            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
#            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\

for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d]

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
    #prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - sum(demandcap)*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) <= intermediateconstraint[i]
    
for i in intermediate:
    for d in demand:
        prob += int_to_dem_vars[i][d] - sum(demandcap)*int_to_dem_open[i][d] <= 0
        prob += int_to_dem_open[i][d] <= int_to_dem_vars[i][d] 
for d in demand:
    prob += p.lpSum([int_to_dem_open[i][d] for i in intermediate]) == 1
    
    

#prob += p.lpSum([int_to_dem_vars[i][d]*dist_within_limit_dict[i][d] for (i,d) in int_to_dem_route]) >= 0.5 #pct of demand within certain limit constraint
 
#prob += p.lpSum([int_to_dem_vars[i][d]*int_dem_dist[i][d] for (i,d) in int_to_dem_route]) <= 100  
    
prob.solve()

print('minimum cost: ', round(p.value(prob.objective),0))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  15530.0
Solution Status: Optimal
inbound_route_Hub_A_DC1  =  862.0
inbound_route_Hub_A_DC2  =  1264.0
inbound_route_Hub_A_DC3  =  988.0
inbound_route_Hub_A_DC4  =  827.0
outbound_route_DC1_Area1  =  359.0
outbound_route_DC1_Area2  =  0.0
outbound_route_DC1_Area3  =  0.0
outbound_route_DC1_Area4  =  0.0
outbound_route_DC1_Area5  =  0.0
outbound_route_DC1_Area6  =  0.0
outbound_route_DC1_Area7  =  503.0
outbound_route_DC1_Area8  =  0.0
outbound_route_DC2_Area1  =  0.0
outbound_route_DC2_Area2  =  258.0
outbound_route_DC2_Area3  =  0.0
outbound_route_DC2_Area4  =  0.0
outbound_route_DC2_Area5  =  0.0
outbound_route_DC2_Area6  =  299.0
outbound_route_DC2_Area7  =  0.0
outbound_route_DC2_Area8  =  707.0
outbound_route_DC3_Area1  =  0.0
outbound_route_DC3_Area2  =  0.0
outbound_route_DC3_Area3  =  0.0
outbound_route_DC3_Area4  =  988.0
outbound_route_DC3_Area5  =  0.0
outbound_route_DC3_Area6  =  0.0
outbound_route_DC3_Area7  =  0.0
outbound_route_DC3_Area8  =  0.0
outbound_ro

<b>Question 2</b>

FreshGo is considering opening one additional hub. In other words, Hub A and the four DCs (from Question 1) will remain open, and FreshGo is considering to add one (only one) more hub to its network.

Jean Hall has identified three potential locations, each one with a different daily fixed cost (see Table 6). The new potential hubs have a maximum capacity of 2000 boxes each.

<img src = 'finalexam23.png'>

Which additional hub should FreshGo open to minimize the daily cost of operation? (Remember that Hub A remains open.)

In [39]:
supply = ['Hub_A','Hub_B','Hub_C','Hub_D']
supplycap = np.array([9999999,2000,2000,2000])
supply_fixed_cost = np.array([0,1000,800,500])

demand = ['Area1', 'Area2', 'Area3','Area4', 'Area5', 'Area6','Area7','Area8']
demandcap = np.array([359,258,583,988,244,299,503,707])

intermediate = ['DC1','DC2','DC3','DC4']
#intermediate_fixed_cost = np.array([10000,7000,7000,5500])
intermediatecap = np.array([1250,1340,1060,895])


                          #int1 #int2 #int3 #int4
cost_sup_int   = np.array([[91,66,82,27],
                           [69,12,66,13],
                           [22,50,34,56],
                           [56,23,65,80]]) #supN

                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5...#DemN
cost_int_dem = np.array([[75,43,65,86,45,78,2,44],#int1...
                         [90,18,59,51,69,23,7,24],
                         [27,83,41,10,36,55,21,21],
                         [42,89,45,76,15,18,55,11]]) #intN

#dist_within_limit = cost_int_dem <= 50 #remember to use less than or equal to sign

#dist_within_limit = dist_within_limit.astype(int) / sum(demandcap)

#dist_within_limit_dict = p.makeDict([intermediate, demand], dist_within_limit)

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))
intermediateconstraint = dict(zip(intermediate,intermediatecap))

#intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost))

supplyfixedcost = p.makeDict([supply], supply_fixed_cost)

#int_dem_dist = p.makeDict([intermediate, demand], cost_int_dem/sum(demandcap)) #maximum avg. distance constraint
sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*0.027) 
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*0.087)


prob = p.LpProblem('Fresh_Go', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("inbound_route", (supply,intermediate), lowBound = 0, cat = 'Integer')

#int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("outbound_route", (intermediate,demand), lowBound = 0, cat = 'Integer')

int_to_dem_open = p.LpVariable.dicts("outbound_route_open", (intermediate,demand), lowBound = 0, cat = 'Binary')

sup_open = p.LpVariable.dicts("open_Hub", (supply), lowBound = 0, cat = 'Binary')

#prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
#            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
#            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([sup_open[s]*supplyfixedcost[s] for s in supply])

for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d]
    
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s]
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) - sum(demandcap)*sup_open[s] <= 0

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
    #prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - sum(demandcap)*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) <= intermediateconstraint[i]
    
for i in intermediate:
    for d in demand:
        prob += int_to_dem_vars[i][d] - sum(demandcap)*int_to_dem_open[i][d] <= 0
        prob += int_to_dem_open[i][d] <= int_to_dem_vars[i][d]
for d in demand:
    prob += p.lpSum([int_to_dem_open[i][d] for i in intermediate]) == 1

prob += sup_open['Hub_A'] == 1
prob += p.lpSum([sup_open[s] for s in supply]) == 2
    

#prob += p.lpSum([int_to_dem_vars[i][d]*dist_within_limit_dict[i][d] for (i,d) in int_to_dem_route]) >= 0.5 #pct of demand within certain limit constraint
 
#prob += p.lpSum([int_to_dem_vars[i][d]*int_dem_dist[i][d] for (i,d) in int_to_dem_route]) <= 100  
    
prob.solve()

print('minimum cost: ', round(p.value(prob.objective),0))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  13379.0
Solution Status: Optimal
inbound_route_Hub_A_DC1  =  0.0
inbound_route_Hub_A_DC2  =  1114.0
inbound_route_Hub_A_DC3  =  0.0
inbound_route_Hub_A_DC4  =  827.0
inbound_route_Hub_B_DC1  =  0.0
inbound_route_Hub_B_DC2  =  0.0
inbound_route_Hub_B_DC3  =  0.0
inbound_route_Hub_B_DC4  =  0.0
inbound_route_Hub_C_DC1  =  862.0
inbound_route_Hub_C_DC2  =  150.0
inbound_route_Hub_C_DC3  =  988.0
inbound_route_Hub_C_DC4  =  0.0
inbound_route_Hub_D_DC1  =  0.0
inbound_route_Hub_D_DC2  =  0.0
inbound_route_Hub_D_DC3  =  0.0
inbound_route_Hub_D_DC4  =  0.0
open_Hub_Hub_A  =  1.0
open_Hub_Hub_B  =  0.0
open_Hub_Hub_C  =  1.0
open_Hub_Hub_D  =  0.0
outbound_route_DC1_Area1  =  359.0
outbound_route_DC1_Area2  =  0.0
outbound_route_DC1_Area3  =  0.0
outbound_route_DC1_Area4  =  0.0
outbound_route_DC1_Area5  =  0.0
outbound_route_DC1_Area6  =  0.0
outbound_route_DC1_Area7  =  503.0
outbound_route_DC1_Area8  =  0.0
outbound_route_DC2_Area1  =  0.0
outbound_route_DC2_Area2  =  258.0
o

<b>Question 3</b>

Jean Hall has been tasked to analyze another option. Instead of introducing a new hub, she should explore the option of delivering directly to consumers from Hub A (in addition to delivering from the four DCs).

<img src = 'finalexam24.png'>

The cost of serving from Hub A directly to the delivery areas would be 0.087 pounds per box per mile.

What would be the optimal daily cost of operation for this network?

In [41]:
supply = ['Hub_A']

demand = ['Area1', 'Area2', 'Area3','Area4', 'Area5', 'Area6','Area7','Area8']
demandcap = np.array([359,258,583,988,244,299,503,707])

intermediate = ['DC1','DC2','DC3','DC4']
intermediatecap = np.array([1250,1340,1060,895])


                          #int1 #int2 #int3 #int4
cost_sup_int   = np.array([[91,66,82,27]]) #sup1
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5...#DemN
cost_int_dem = np.array([[75,43,65,86,45,78,2,44],#int1...
                         [90,18,59,51,69,23,7,24],
                         [27,83,41,10,36,55,21,21],
                         [42,89,45,76,15,18,55,11]]) #intN

                        #dem1....                   #demN
cost_sup_dem = np.array([[186,224,152,49,122,294,193,226]]) #sup1

#dist_within_limit = cost_int_dem <= 50 #remember to use less than or equal to sign

#dist_within_limit = dist_within_limit.astype(int) / sum(demandcap)

#dist_within_limit_dict = p.makeDict([intermediate, demand], dist_within_limit)


demandconstraint = dict(zip(demand,demandcap))
intermediateconstraint = dict(zip(intermediate,intermediatecap))


#int_dem_dist = p.makeDict([intermediate, demand], cost_int_dem/sum(demandcap)) #maximum avg. distance constraint
sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*0.027) 
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*0.087)
sup_dem_dict = p.makeDict([supply, demand], cost_sup_dem*0.087)


prob = p.LpProblem('Fresh_Go', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("inbound_route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("outbound_route", (intermediate,demand), lowBound = 0, cat = 'Integer')

sup_to_dem_route = [(s,d) for s in supply for d in demand]

sup_to_dem_vars = p.LpVariable.dicts("direct_route", (supply,demand), lowBound = 0, cat = 'Integer')

int_to_dem_open = p.LpVariable.dicts("outbound_route_open", (intermediate,demand), lowBound = 0, cat = 'Binary')


prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([sup_to_dem_vars[s][d]*sup_dem_dict[s][d] for (s,d) in sup_to_dem_route])

for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) + p.lpSum([sup_to_dem_vars[s][d] for s in supply]) == demandconstraint[d]

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint  
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) <= intermediateconstraint[i]
    
for i in intermediate:
    for d in demand:
        prob += int_to_dem_vars[i][d] - sum(demandcap)*int_to_dem_open[i][d] <= 0
        prob += int_to_dem_open[i][d] <= int_to_dem_vars[i][d]
for d in demand:
    prob += p.lpSum([int_to_dem_open[i][d] for i in intermediate]) == 1   

#prob += p.lpSum([int_to_dem_vars[i][d]*dist_within_limit_dict[i][d] for (i,d) in int_to_dem_route]) >= 0.5 #pct of demand within certain limit constraint
 
#prob += p.lpSum([int_to_dem_vars[i][d]*int_dem_dist[i][d] for (i,d) in int_to_dem_route]) <= 100  
    
prob.solve()

print('minimum cost: ', round(p.value(prob.objective),0))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  14282.0
Solution Status: Optimal
direct_route_Hub_A_Area1  =  0.0
direct_route_Hub_A_Area2  =  0.0
direct_route_Hub_A_Area3  =  0.0
direct_route_Hub_A_Area4  =  287.0
direct_route_Hub_A_Area5  =  0.0
direct_route_Hub_A_Area6  =  0.0
direct_route_Hub_A_Area7  =  0.0
direct_route_Hub_A_Area8  =  0.0
inbound_route_Hub_A_DC1  =  503.0
inbound_route_Hub_A_DC2  =  1264.0
inbound_route_Hub_A_DC3  =  1060.0
inbound_route_Hub_A_DC4  =  827.0
outbound_route_DC1_Area1  =  0.0
outbound_route_DC1_Area2  =  0.0
outbound_route_DC1_Area3  =  0.0
outbound_route_DC1_Area4  =  0.0
outbound_route_DC1_Area5  =  0.0
outbound_route_DC1_Area6  =  0.0
outbound_route_DC1_Area7  =  503.0
outbound_route_DC1_Area8  =  0.0
outbound_route_DC2_Area1  =  0.0
outbound_route_DC2_Area2  =  258.0
outbound_route_DC2_Area3  =  0.0
outbound_route_DC2_Area4  =  0.0
outbound_route_DC2_Area5  =  0.0
outbound_route_DC2_Area6  =  299.0
outbound_route_DC2_Area7  =  0.0
outbound_route_DC2_Area8  =  707.0
outbound_rou

<b>Question 4</b>

FreshGo management team recognizes that Jean Hall did a great analysis, but they are reluctant to introduce any major changes to their network right now, in the middle of the COVID pandemic. However, they want to better understand what is their current level of service. (To answer this question, just consider FreshGo’s original network from Question 1.)

What is the level of service FreshGo is currently providing, measured in percentage of demand within 30 miles of DC?

In [45]:
supply = ['Hub_A']

demand = ['Area1', 'Area2', 'Area3','Area4', 'Area5', 'Area6','Area7','Area8']
demandcap = np.array([359,258,583,988,244,299,503,707])

intermediate = ['DC1','DC2','DC3','DC4']

intermediatecap = np.array([1250,1340,1060,895])


                          #int1 #int2 #int3 #int4
cost_sup_int   = np.array([[91,66,82,27]]) #sup1
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5...#DemN
cost_int_dem = np.array([[75,43,65,86,45,78,2,44],#int1...
                         [90,18,59,51,69,23,7,24],
                         [27,83,41,10,36,55,21,21],
                         [42,89,45,76,15,18,55,11]]) #intN

dist_within_limit = cost_int_dem <= 30 #remember to use less than or equal to sign

dist_within_limit = dist_within_limit.astype(int) / sum(demandcap)

dist_within_limit_dict = p.makeDict([intermediate, demand], dist_within_limit)


demandconstraint = dict(zip(demand,demandcap))
intermediateconstraint = dict(zip(intermediate,intermediatecap))

#int_dem_dist = p.makeDict([intermediate, demand], cost_int_dem/sum(demandcap)) #maximum avg. distance constraint

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*0.027) 
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*0.087)


prob = p.LpProblem('Fresh_Go', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("inbound_route", (supply,intermediate), lowBound = 0, cat = 'Integer')

#int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("outbound_route", (intermediate,demand), lowBound = 0, cat = 'Integer')

int_to_dem_open = p.LpVariable.dicts("outbound_route_open", (intermediate,demand), lowBound = 0, cat = 'Binary')


prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\

for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d]

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
    #prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - sum(demandcap)*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) <= intermediateconstraint[i]
    
for i in intermediate:
    for d in demand:
        prob += int_to_dem_vars[i][d] - sum(demandcap)*int_to_dem_open[i][d] <= 0
        prob += int_to_dem_open[i][d] <= int_to_dem_vars[i][d]
for d in demand:
    prob += p.lpSum([int_to_dem_open[i][d] for i in intermediate]) == 1   
    

#prob += p.lpSum([int_to_dem_vars[i][d]*dist_within_limit_dict[i][d] for (i,d) in int_to_dem_route]) >= 0.77 #pct of demand within certain limit constraint
 
#prob += p.lpSum([int_to_dem_vars[i][d]*int_dem_dist[i][d] for (i,d) in int_to_dem_route]) <= 100  
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])
print('%demand within 30 mile',p.lpSum([int_to_dem_vars[i][d].varValue*dist_within_limit_dict[i][d] for (i,d) in int_to_dem_route]))

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  15529.662
Solution Status: Optimal
%demand within 30 mile 0.7609743719868055
inbound_route_Hub_A_DC1  =  862.0
inbound_route_Hub_A_DC2  =  1264.0
inbound_route_Hub_A_DC3  =  988.0
inbound_route_Hub_A_DC4  =  827.0
outbound_route_DC1_Area1  =  359.0
outbound_route_DC1_Area2  =  0.0
outbound_route_DC1_Area3  =  0.0
outbound_route_DC1_Area4  =  0.0
outbound_route_DC1_Area5  =  0.0
outbound_route_DC1_Area6  =  0.0
outbound_route_DC1_Area7  =  503.0
outbound_route_DC1_Area8  =  0.0
outbound_route_DC2_Area1  =  0.0
outbound_route_DC2_Area2  =  258.0
outbound_route_DC2_Area3  =  0.0
outbound_route_DC2_Area4  =  0.0
outbound_route_DC2_Area5  =  0.0
outbound_route_DC2_Area6  =  299.0
outbound_route_DC2_Area7  =  0.0
outbound_route_DC2_Area8  =  707.0
outbound_route_DC3_Area1  =  0.0
outbound_route_DC3_Area2  =  0.0
outbound_route_DC3_Area3  =  0.0
outbound_route_DC3_Area4  =  988.0
outbound_route_DC3_Area5  =  0.0
outbound_route_DC3_Area6  =  0.0
outbound_route_DC3_Area7  =  0.0
